# Code for downloading and processing AIS data
This notebook does not access or process any Cuebiq data. It is contained in this folder to maintain all of the code used for Ahmadiani and Woodward, "Fishing for Anglers" in a single place. The code was run  on the Texas A&M HPRC system.

## Key Inputs: 
* Spatial wkts with Gulf and island areas to be excluded
* AIS data downloaded from **https://marinecadastre.gov/accessais**

## Key Output:
* **Indicators2019_All.C.csv.csv** -- a csv file labeled data with features for AIS trips

## Preliminaries

In [ ]:
import csv
import datetime
import datetime as dt
import geopandas as gpd
import h3.api as h3
import heapq
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pytz
import requests
import seaborn as sns
import time

from scipy import interpolate
from shapely import wkt
from shapely.geometry import Polygon
from zipfile import ZipFile
from shapely.geometry import Point  # Import the Point class from shapely.geometry
from datetime import datetime

In [ ]:
# !pip freeze t:/files.txt
# !pip install sklearn
!pip install scikit-learn
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


# 1. Download all the zip files and exctracts them to CSV files

## Unzip the files, create csvs, process the data to create Coastline and Gulf data sets for each day

In [ ]:
###############################################################################################
# Values that need to be adjusted depending on ron and computer system in which we're operating
###############################################################################################

## Directorey woul
zipdir = r"/scratch/user/r-woodward/AIS Data/ZIP Files"               #D:\AIS Data\Zip Files"

csvdir = r"/scratch/user/r-woodward/AIS Data/CSV Files"               #"D:\AIS Data\CSV Files"
extract_folder = csvdir
Gulfdir =  r"/scratch/user/r-woodward/AIS Data/CSV Files/Gulf"         # r"D:\AIS Data\CSV Files\Gulf"
Coastdir =  r"/scratch/user/r-woodward/AIS Data/CSV Files/Coastline"   #r"D:\AIS Data\CSV Files\Coastline"


# Period to be processed 2019-2022, 1-13, and 1-32 for complete series
startyr = 2019
endyr = 2020
startmo = 8
endmo = 9 #
startday = 1 #1
endday = 32

###############################################################################################

import zipfile
import pandas as pd
import time

# Record the start time
start_time = time.time()
print("Starting at ", start_time)

date_format = "AIS_{:04d}_{:02d}_{:02d}"
for yr in range(startyr, endyr):
    for mo in range(startmo, endmo):
        for day in range(startday, endday):

            date_str = date_format.format(yr, mo, day)

            # New style using paths read in
            zip_file_path = os.path.join(zipdir, date_str + ".zip")
            csv_file_path = os.path.join(csvdir, date_str + ".csv")
            Gulf_file_path = os.path.join(Gulfdir, date_str + "_Gulf.csv")
            Coastline_file_path = os.path.join(Coastdir, date_str + "_Coastline.csv")

            if os.path.exists(zip_file_path):
                print(zip_file_path, "Exits")
            if os.path.exists(csv_file_path):
                print(csv_file_path, "Exits")
            if os.path.exists(Gulf_file_path):
                print(Gulf_file_path, "Exits")
            if os.path.exists(Coastline_file_path):
                print(Coastline_file_path, "Exits")
            # If the zip file exits and has not already been unzipped
            if os.path.exists(zip_file_path) and not os.path.exists(csv_file_path):
                print(csv_file_path)

                # Extract the ZIP file
                with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                    zip_ref.extractall(extract_folder)
    
            try:
                # Run the rest if the csv_file_path exist and the other files have not already been processed
                if os.path.exists(csv_file_path) and not os.path.exists(Coastline_file_path):
                    print("Processing ", date_str)
                    # Read the CSV file and create a DataFrame
                    AIS_Data_df = pd.read_csv(csv_file_path)
    
                    # Rename LAT and LON for later compatibility
                    AIS_Data_df.rename(columns={'LAT': 'lat'}, inplace=True)
                    AIS_Data_df.rename(columns={'LON': 'lon'}, inplace=True)
    
                    # Drop a bunch of columns not needed
                    columns_to_drop = ['SOG', 'COG', 'Heading', 'IMO', 'CallSign', 'Status', 'Length', 'Width', 'Draft', 'Cargo', 'TransceiverClass']
                    AIS_Data_df.drop(columns=columns_to_drop, inplace=True)
    
                    # Quickly eliminate a lot of points using the Gulf Rectangle
                    filtered_data = AIS_Data_df[
                        (AIS_Data_df['lat'] >= lat_south) &
                        (AIS_Data_df['lat'] <= lat_north) &
                        (AIS_Data_df['lon'] >= lon_west) &
                        (AIS_Data_df['lon'] <= lon_east)
                    ]
    
                    geometry = [Point(lon, lat) for lon, lat in zip(filtered_data['lon'], filtered_data['lat'])]
                    # AIS_Data_gdf = AIS_Data_df.set_geometry('geometry')
                    AIS_Data_gdf = gpd.GeoDataFrame(filtered_data, geometry=geometry)
                    
                    end_time = time.time()
                    elapsed_time = end_time - start_time
                    print(date_str, f"Elapsed time: {elapsed_time} seconds")
                    
                    # Use the function process_data to create two data frames, coastline points and Gulf (in water)
                    Just_Gulf, Coastline = process_data(AIS_Data_gdf)
    
                    # Save the two data frames as csv files
                    Coastline.to_csv(Coastline_file_path,  index=False)
                    Just_Gulf.to_csv(Gulf_file_path, index=False)

                    # Delete the csv file
                    os.remove(csv_file_path)
            except Exception as e:
                print(f"Error processing {date_str}: {e}")
                # Rename the file with "HAS ERRORS" at the end
                base_name, file_extension = os.path.splitext(csv_file_path)
                new_name = f"{csv_file_path} HAS ERRORS{file_extension}"
                os.rename(csv_file_path, new_name)

                base_name, file_extension = os.path.splitext(zip_file_path)
                new_name = f"{zip_file_path} HAS ERRORS{file_extension}"
                os.rename(zip_file_path, new_name)



# Record the end time
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds")

## Area Processing cells
Because the AIS data are global, the first step is to eliminate all data outside of the polygon used for analysis.

### Areas for processing

In [ ]:
# Gulf Rectangle boundaries
lon_west  =-98.1429
lat_south = 25.0856
lon_east  = -85.4564
lat_north = 30.9213

# wkt string
# GULF
# Gulf=  'POLYGON ((-97.1461459379788 25.957168007570218, -96.289339604698 24.85679572942162, -92.46567751398405 24.93226258740249, -86.46809930536585 26.63685607835417, -87.4518747436496 30.2870218479717, -87.56970870420537 30.265986917324028, -87.68688306481553 30.240398581969316, -87.80866540589953 30.223904790285445, -87.86330288865585 30.223335989499247, -87.95809562982397 30.227886303681714, -88.02919018569986 30.21821663439117, -88.04103927834595 30.236417575256624, -87.96007047859824 30.256320996255198, -87.9264980494346 30.259732606445226, -87.87712683007626 30.243810745100816, -87.84223783506322 30.25688960618315, -87.78825863523141 30.25802681616385, -87.76653529871386 30.268829654214812, -87.75929418654121 30.287589653053473, -87.77443469381102 30.308051015443525, -87.80142429372691 30.321121316921534, -87.83236359119138 30.34441622544655, -87.84487096676237 30.372249210920785, -87.87910145959437 30.384175375165654, -87.9185984350809 30.414267849534085, -87.92386469847926 30.462509996885288, -87.94032177159842 30.47215556160502, -87.93505550820048 30.49938493987453, -87.91991500093027 30.529441859299524, -87.9172818692315 30.552120171744733, -87.91991500093027 30.575926698627413, -87.92452298140375 30.609925886488114, -87.92978924480211 30.64051495391115, -87.9475628837711 30.650709160976575, -87.96928622028864 30.651841784313817, -87.99298440558087 30.665432229337696, -88.01141632747473 30.659203512573512, -88.0423556249392 30.638249428634225, -88.05091330296123 30.624088693607646, -88.04696360541267 30.61559125795084, -88.05354643466042 30.590094476474817, -88.07790290287716 30.565157802896906, -88.07592805410286 30.539080789195197, -88.09370063563347 30.456267467088836, -88.09896689903184 30.398370320370773, -88.10620801120405 30.353504810736837, -88.15887064518647 30.323393627447288, -88.19178479142525 30.31032362908067, -88.20429216699586 30.330780246575657, -88.19705105482359 30.353504810736837, -88.28855238136764 30.381335210261483, -88.2957934935403 30.364865114151982, -88.31817511298274 30.363161152750493, -88.32014996175704 30.38190309511809, -88.3517475421464 30.390988804016374, -88.39058623470804 30.37224860380745, -88.39256108348229 30.342143188113525, -88.43732432236718 30.34725608152752, -88.44259058576552 30.331348424972347, -88.48077099540268 30.308618718161995, -88.5018360489957 30.312596797505776, -88.5288256489116 30.337030027619335, -88.55384040005282 30.331348424972347, -88.58807111214138 30.335893733464147, -88.60584475111035 30.368840908630105, -88.65784982716818 30.350096668434418, -88.70377316315728 30.333940318201655, -88.76345157602148 30.340304903230376, -88.81215810808295 30.366323547786152, -88.96059706293589 30.3823315816322, -89.10671665911954 30.334299618169936, -89.33169445006897 30.2782325348214, -89.41287200350473 30.192067121425936, -89.44534302487904 30.08977311890679, -89.359526754104 30.069703042406346, -89.20876844058175 30.176028017449696, -89.16006190852028 30.16199165952355, -89.16006190852028 30.08977311890679, -89.13918768049385 30.055651566395866, -89.16238126718959 29.921058052351142, -89.21108779925103 29.7782302647352, -89.24587817929464 29.66946405486304, -89.31545893938255 29.693644509171918, -89.27371048332964 29.629150377050877, -89.30386214603394 29.59688781400496, -89.3850396994697 29.61503677626405, -89.46157853556547 29.532331728567527, -89.3827203407997 29.439460132099583, -89.18979122539224 29.427128425449155, -89.19596940111164 29.394836842780364, -88.95749181835598 29.20950068771296, -89.04027937299146 29.04327521681688, -89.10329676532581 29.036793464759455, -89.13913018449634 28.94816874403513, -89.19473376596763 28.98276303815456, -89.26022242858967 29.027070073382717, -89.29358457747252 29.016265231135534, -89.4010848349841 28.903827915763785, -89.44186079473003 28.94816874403513, -89.39243538897739 29.025989640029877, -89.44680333530505 29.0864764893534, -89.45298151102406 29.14692783767063, -89.42209063242889 29.17066685855066, -89.376372132108 29.156639918576943, -89.36525141581359 29.180376694261838, -89.34671688865649 29.20195081707395, -89.37390086182032 29.206265096965808, -89.40232047012778 29.203029404070804, -89.4356826190106 29.20195081707395, -89.46780913274982 29.179297868997324, -89.49499310591362 29.193321712569713, -89.54441851166625 29.2278337721458, -89.63214860687673 29.254788228681946, -89.68404528291667 29.282813330704627, -89.80143062157865 29.305443379957012, -89.85579856790665 29.300055727855437, -89.9076952439466 29.27526886673428, -89.98924716343802 29.21489311194422, -90.05473582606008 29.165272108427317, -90.1572935429963 29.102672297767754, -90.26232567572231 29.06703684192044, -90.2833314731671 29.10914859304988, -90.26479694600961 29.13181701694839, -90.28827401374208 29.15232223739727, -90.29939473003648 29.19547774319095, -90.269739486585 29.23538042243976, -90.29568782460515 29.2310673685935, -90.30927981118687 29.24724038352842, -90.33893505463838 29.26341084252479, -90.393303000966 29.251552755878066, -90.40565935240406 29.30220951059583, -90.44767094729364 29.334530475168393, -90.50203889362164 29.32698983267116, -90.54899302908619 29.3032870410304, -90.57123446167503 29.309751984787326, -90.56629192109963 29.291433580776584, -90.59100462397578 29.241849662499888, -90.72198194921984 29.166349767170132, -90.74669465209598 29.108069019686837, -90.77634989554745 29.09943202507891, -90.84801673388853 29.115625795370804, -90.88755705849043 29.13721352427197, -90.89126396392177 29.16742872854931, -90.91474103165422 29.217048688862008, -90.92956865337999 29.198713674444775, -90.93080428852403 29.16742872854931, -90.97652278884493 29.160954790180483, -91.02965510002853 29.163112814994165, -91.08772995178788 29.17713887061521, -91.15445424955357 29.210577881865433, -91.21005783102524 29.21165637807694, -91.28790284508534 29.240771484217632, -91.33609261569393 29.292511224930067, -91.36327658885774 29.33237606278429, -91.36698404442399 29.461560546417502, -91.37316222014341 29.483075264842185, -91.42505889618336 29.473394206552683, -91.48189811279867 29.486302078781378, -91.50784645081843 29.51963980456361, -91.52885224826323 29.514263494469418, -91.5424442348453 29.538992155608895, -91.5424442348453 29.565864276975645, -91.56097876200242 29.62710598028727, -91.58692710002259 29.62603188493061, -91.6351168706312 29.61636451146687, -91.65118012750057 29.642142112800144, -91.63758814091887 29.656102225576717, -91.64129504635021 29.723727639327834, -91.71296188469128 29.718362205888425, -91.76980110130658 29.702264184791886, -91.83034722335326 29.685090118027063, -91.85505992622939 29.662544701481323, -91.845174845079 29.634624327137942, -91.79451380418274 29.609919080630874, -91.75744474986853 29.590580315799386, -91.69689862782151 29.57123784364269, -91.71914006041031 29.53254178333445, -91.74879530386181 29.517489314800784, -91.77103673645024 29.48199963736188, -91.8019276150454 29.470166981654714, -91.8525886559417 29.47662132872088, -91.94649692687157 29.517489314800784, -92.01816376521265 29.556191136499905, -92.081181157547 29.575536491098305, -92.14666982016863 29.57338719024669, -92.2282217396605 29.543292175193457, -92.2620745477254 29.513938983582918, -92.40184398133182 29.53587031357371, -92.53015690398729 29.55580375974114, -92.65388722226176 29.58171136156176, -92.76845233177524 29.613588513778055, -92.90134785881101 29.66735835342932, -92.99758255080252 29.705179169035475, -93.10985635812567 29.739006777997886, -93.2015084457365 29.758900043967557, -93.27024751144474 29.76088915343348, -93.3389865771526 29.740996282222838, -93.43980387352478 29.750943211286994, -93.61623414217547 29.740996282222838, -93.76058618016258 29.715129649330578, -93.8293252458708 29.663376387168597, -93.95534686633557 29.6852752506843, -94.10199020651301 29.65541198179467, -94.53962892485461 29.488014011072636, -94.71376789131534 29.402214927729204, -94.72293005770678 29.324334651639106, -94.9360211614021 29.206402439154715, -95.13765575414574 29.060298314788838, -95.37824248412433 28.86583930097359, -95.42635983011992 28.85781253647636, -95.88003908572986 28.63281185841163, -96.17332576608472 28.508049911739334, -96.3657951500674 28.385156078353774, -96.42995161139497 28.326681104162276, -96.5811775559527 28.237899002445175, -96.8538425165949 28.05404703841444, -97.01423366991399 27.871905659410203, -97.18378864850824 27.632633899635536, -97.28460594488008 27.443685488256804, -97.34647110401767 27.286998971872194, -97.37396673030096 27.111736817761667, -97.37167542811062 26.942327753778045, -97.30062980709232 26.653886203530476, -97.27907391837897 26.589650805967054, -97.227579295341 26.438555761793396, -97.19764056101677 26.3183955849306, -97.16770182669217 26.152971688301193, -97.15093613547081 26.036819954392655, -97.1461459379788 25.957168007570218))'
Gulf_poly=Polygon([(-97.146145,25.9571),(-96.289339,24.8567),(-92.465677,24.9322),(-86.468099,26.6368),(-87.451874,30.2870),(-87.569708,30.2659),(-87.686883,30.2403),(-87.808665,30.2239),(-87.863302,30.2233),(-87.958095,30.2278),(-88.029190,30.2182),(-88.041039,30.2364),(-87.960070,30.2563),(-87.926498,30.2597),(-87.877126,30.2438),(-87.842237,30.2568),(-87.788258,30.2580),(-87.766535,30.2688),(-87.759294,30.2875),(-87.774434,30.3080),(-87.801424,30.3211),(-87.832363,30.3444),(-87.844870,30.3722),(-87.879101,30.3841),(-87.918598,30.4142),(-87.923864,30.4625),(-87.940321,30.4721),(-87.935055,30.4993),(-87.919915,30.5294),(-87.917281,30.5521),(-87.919915,30.5759),(-87.924522,30.6099),(-87.929789,30.6405),(-87.947562,30.6507),(-87.969286,30.6518),(-87.992984,30.6654),(-88.011416,30.6592),(-88.042355,30.6382),(-88.050913,30.6240),(-88.046963,30.6155),(-88.053546,30.5900),(-88.077902,30.5651),(-88.075928,30.5390),(-88.093700,30.4562),(-88.098966,30.3983),(-88.106208,30.3535),(-88.158870,30.3233),(-88.191784,30.3103),(-88.204292,30.3307),(-88.197051,30.3535),(-88.288552,30.3813),(-88.295793,30.3648),(-88.318175,30.3631),(-88.320149,30.3819),(-88.351747,30.3909),(-88.390586,30.3722),(-88.392561,30.3421),(-88.437324,30.3472),(-88.442590,30.3313),(-88.480770,30.3086),(-88.501836,30.3125),(-88.528825,30.3370),(-88.553840,30.3313),(-88.588071,30.3358),(-88.605844,30.3688),(-88.657849,30.3500),(-88.703773,30.3339),(-88.763451,30.3403),(-88.812158,30.3663),(-88.960597,30.3823),(-89.106716,30.3342),(-89.331694,30.2782),(-89.412872,30.1920),(-89.445343,30.0897),(-89.359526,30.0697),(-89.208768,30.1760),(-89.160061,30.1619),(-89.160061,30.0897),(-89.139187,30.0556),(-89.162381,29.9210),(-89.211087,29.7782),(-89.245878,29.6694),(-89.315458,29.6936),(-89.273710,29.6291),(-89.303862,29.5968),(-89.385039,29.6150),(-89.461578,29.5323),(-89.382720,29.4394),(-89.189791,29.4271),(-89.195969,29.3948),(-88.957491,29.2095),(-89.040279,29.0432),(-89.103296,29.0367),(-89.139130,28.9481),(-89.194733,28.9827),(-89.260222,29.0270),(-89.293584,29.0162),(-89.401084,28.9038),(-89.441860,28.9481),(-89.392435,29.0259),(-89.446803,29.0864),(-89.452981,29.1469),(-89.422090,29.1706),(-89.376372,29.1566),(-89.365251,29.1803),(-89.346716,29.2019),(-89.373900,29.2062),(-89.402320,29.2030),(-89.435682,29.2019),(-89.467809,29.1792),(-89.494993,29.1933),(-89.544418,29.2278),(-89.632148,29.2547),(-89.684045,29.2828),(-89.801430,29.3054),(-89.855798,29.3000),(-89.907695,29.2752),(-89.989247,29.2148),(-90.054735,29.1652),(-90.157293,29.1026),(-90.262325,29.0670),(-90.283331,29.1091),(-90.264796,29.1318),(-90.288274,29.1523),(-90.299394,29.1954),(-90.269739,29.2353),(-90.295687,29.2310),(-90.309279,29.2472),(-90.338935,29.2634),(-90.393303,29.2515),(-90.405659,29.3022),(-90.447670,29.3345),(-90.502038,29.3269),(-90.548993,29.3032),(-90.571234,29.3097),(-90.566291,29.2914),(-90.591004,29.2418),(-90.721981,29.1663),(-90.746694,29.1080),(-90.776349,29.0994),(-90.848016,29.1156),(-90.887557,29.1372),(-90.891263,29.1674),(-90.914741,29.2170),(-90.929568,29.1987),(-90.930804,29.1674),(-90.976522,29.1609),(-91.029655,29.1631),(-91.087729,29.1771),(-91.154454,29.2105),(-91.210057,29.2116),(-91.287902,29.2407),(-91.336092,29.2925),(-91.363276,29.3323),(-91.366984,29.4615),(-91.373162,29.4830),(-91.425058,29.4733),(-91.481898,29.4863),(-91.507846,29.5196),(-91.528852,29.5142),(-91.542444,29.5389),(-91.542444,29.5658),(-91.560978,29.6271),(-91.586927,29.6260),(-91.635116,29.6163),(-91.651180,29.6421),(-91.637588,29.6561),(-91.641295,29.7237),(-91.712961,29.7183),(-91.769801,29.7022),(-91.830347,29.6850),(-91.855059,29.6625),(-91.845174,29.6346),(-91.794513,29.6099),(-91.757444,29.5905),(-91.696898,29.5712),(-91.719140,29.5325),(-91.748795,29.5174),(-91.771036,29.4819),(-91.801927,29.4701),(-91.852588,29.4766),(-91.946496,29.5174),(-92.018163,29.5561),(-92.081181,29.5755),(-92.146669,29.5733),(-92.228221,29.5432),(-92.262074,29.5139),(-92.401843,29.5358),(-92.530156,29.5558),(-92.653887,29.5817),(-92.768452,29.6135),(-92.901347,29.6673),(-92.997582,29.7051),(-93.109856,29.7390),(-93.201508,29.7589),(-93.270247,29.7608),(-93.338986,29.7409),(-93.439803,29.7509),(-93.616234,29.7409),(-93.760586,29.7151),(-93.829325,29.6633),(-93.955346,29.6852),(-94.101990,29.6554),(-94.539628,29.4880),(-94.713767,29.4022),(-94.722930,29.3243),(-94.936021,29.2064),(-95.137655,29.0602),(-95.378242,28.8658),(-95.426359,28.8578),(-95.880039,28.6328),(-96.173325,28.5080),(-96.365795,28.3851),(-96.429951,28.3266),(-96.581177,28.2378),(-96.853842,28.0540),(-97.014233,27.8719),(-97.183788,27.6326),(-97.284605,27.4436),(-97.346471,27.2869),(-97.373966,27.1117),(-97.371675,26.9423),(-97.300629,26.6538),(-97.279073,26.5896),(-97.227579,26.4385),(-97.197640,26.3183),(-97.167701,26.1529),(-97.150936,26.0368),(-97.146145,25.9571)])
Gulf= gpd.GeoDataFrame({'geometry': [Gulf_poly]})
# Gulf_9_points =  'POLYGON ((-86.46548116885579 26.634102610424407, -87.40061559690446 30.106586599968125, -88.7880360341276 30.345100465820636, -88.98500765154343 29.03427800965217, -89.40594788057383 28.894932882450135, -89.73082325193461 29.282245762224036, -90.26144456780166 29.061893210578702, -90.44200934056991 29.29836721229654, -90.74748011763224 29.107574757965764, -91.23958578659675 29.205956552434742, -91.56907754519192 29.516482959643398, -91.9838883621404 29.432203016438365, -93.34672004272255 29.754346820033902, -94.41852173053665 29.530847871103077, -95.32114214435019 28.899812819711684, -96.78381464810607 28.100841256451787, -97.36631695022126 27.16235521630348, -97.15924369852006 26.306987683560095, -97.14527180449294 25.954696710703956, -96.30275760729239 24.857559393634787, -92.47510121995658 24.93070297513802, -86.46548116885579 26.634102610424407))'
Gulf_9_points_poly=Polygon([(-86.4654,26.6341),(-87.4006,30.1065),(-88.7880,30.3451),(-88.9850,29.0342),(-89.4059,28.8949),(-89.7308,29.2822),(-90.2614,29.0618),(-90.4420,29.2983),(-90.7474,29.1075),(-91.2395,29.2059),(-91.5690,29.5164),(-91.9838,29.4322),(-93.3467,29.7543),(-94.4185,29.5308),(-95.3211,28.8998),(-96.7838,28.1008),(-97.3663,27.1623),(-97.1592,26.3069),(-97.1452,25.9546),(-96.3027,24.8575),(-92.4751,24.9307),(-86.4654,26.6341)])
Gulf_9_points= gpd.GeoDataFrame({'geometry': [Gulf_9_points_poly]})
# Gulf_12_points =  'POLYGON ((-86.4654694978406 26.636149213954525, -87.45090136370695 30.286097273699895, -89.19586847435528 28.863951459820456, -90.77164956359688 29.105476683866655, -93.37406823961514 29.74571184680657, -96.07159845917135 28.552059076480816, -97.20115939069258 27.600884380644146, -97.14867513711442 25.954238051503978, -96.29658010773115 24.857874649532846, -92.47327569683304 24.931742615282246, -86.4654694978406 26.636149213954525))'
Gulf_12_points_poly=Polygon([(-86.46546,26.6361),(-87.45090,30.2860),(-89.19586,28.8639),(-90.77164,29.1054),(-93.37406,29.7457),(-96.07159,28.5520),(-97.20115,27.6008),(-97.14867,25.9542),(-96.29658,24.8578),(-92.47327,24.9317),(-86.46546,26.6361)])
Gulf_12_points= gpd.GeoDataFrame({'geometry': [Gulf_12_points_poly]})

# GULF & COAST
# Gulf_withorigin = 'POLYGON ((-97.14617780059835 25.956613984790593, -95.41903951326633 24.305964880613757, -84.84960080963211 25.682219361927096, -87.12006891302346 30.334018523366254, -87.20659151697032 30.415772954557582, -87.23344493600807 30.406509435290076, -87.26602708444021 30.414846642191108, -87.26280467415597 30.39045058636019, -87.27594032871419 30.369325745202858, -87.30425895388599 30.362344676102396, -87.3989240723187 30.344889823334427, -87.39811496874212 30.380494419080563, -87.33581399336356 30.490019289317203, -87.38355110436821 30.443294423171082, -87.43047911179625 30.455849574210532, -87.45556132266289 30.424458664637186, -87.4693160834608 30.361646541783685, -87.51009860743945 30.35323737894953, -87.58291792931071 30.34066902900635, -87.59586358653203 30.30155717675538, -87.72855657305323 30.285488857736567, -87.84506748804681 30.410472761777044, -87.8996195460641 30.470212281326056, -87.88825197383167 30.528983054230267, -87.89668625540148 30.56624880203526, -87.89668625540148 30.601290808639575, -87.91062115754396 30.62874730823566, -87.90842091017784 30.661243210561125, -87.93299035323106 30.686790565652288, -87.9645272321436 30.686790565652288, -87.99826435842223 30.69404352191559, -88.0287011136517 30.6893133948561, -88.04923675573464 30.671652206131824, -88.06500519519065 30.644523378919814, -88.06867156178721 30.598450225941264, -88.08774037229242 30.589611736390836, -88.10534235122034 30.614862424655243, -88.12734482488052 30.611390845320187, -88.13101190382389 30.590243083807238, -88.11451004857902 30.56940644790174, -88.12504475661486 30.51713260569427, -88.13012046212697 30.409944780424098, -88.3999788052048 30.42088771574832, -88.5179977549948 30.346541163717603, -88.54002576052524 30.346499391493396, -88.53992872075061 30.384000583673313, -88.54809915214372 30.394647379626292, -88.56443998792192 30.41323898564835, -88.5453177016835 30.415037986915436, -88.52289247509448 30.410540421575135, -88.51254905662934 30.419760207474354, -88.49394828728789 30.41676199868199, -88.48056268811374 30.424931901248428, -88.48577785708768 30.43422527040208, -88.5066385329844 30.42748017121707, -88.52723838528212 30.42680562793501, -88.53554651817782 30.429207802296204, -88.54614620592992 30.438918186466523, -88.61801283428964 30.43273637512739, -88.62308085092758 30.41753599921266, -88.62043666833374 30.377433650652904, -88.66715056082187 30.369069001140858, -88.72553604830324 30.348759355724425, -88.80555530745305 30.393975338307627, -88.81836711770535 30.401951705381578, -88.82164231731885 30.407352000708883, -88.82511017573282 30.40452726810132, -88.83434946387757 30.40828857362186, -88.83512722585296 30.41521965260938, -88.81179436659222 30.421479558978064, -88.80920182667478 30.427515516940574, -88.84267739878256 30.43033440069395, -88.85673680050274 30.43111495029609, -88.88947482037187 30.424727521295466, -88.90446179708928 30.432054832619542, -88.91803197819463 30.429365946606964, -88.91934412068707 30.430949894266178, -88.92774183264007 30.434796517207886, -88.93823897258041 30.42665054792141, -88.95582168198199 30.427103119617797, -88.97944024684898 30.429818505705526, -88.99728538474864 30.44316805443853, -89.01749237913509 30.44316805443853, -89.0293016615686 30.44113180072472, -89.02352823460126 30.430723617604286, -89.01007026596929 30.42068290029553, -89.04921906210062 30.428118568112566, -89.06474061122745 30.427821152280586, -89.0840563168074 30.433025798414278, -89.08785047326053 30.431836189497318, -89.08129693029612 30.42960563366917, -89.06543045785502 30.4238059498026, -89.04835675381543 30.42291366014365, -89.04448170127748 30.422681700436, -89.04131435052804 30.40862475012409, -89.03096252124925 30.402828166939912, -89.00492868194398 30.413554896730375, -88.9948858624943 30.41155621476223, -88.98886017082455 30.40895786702113, -88.98476579038577 30.410889978565734, -88.97958987574616 30.413088541485777, -88.97093760052802 30.414620843932852, -88.96529817114457 30.41362151897826, -88.96213082039513 30.414021250187915, -88.94176637817542 30.414338746102473, -88.93621624811303 30.4109705087064, -88.92609239955304 30.412122813839602, -88.92270065413021 30.415889883235977, -88.91463240950287 30.41721939750194, -88.91103510298132 30.415756930812663, -88.90800308748445 30.411856912501477, -88.9020418366769 30.410793244098684, -88.89587502549703 30.41190123176689, -88.8814859162461 30.40981820098206, -88.87377740227055 30.40667141600015, -88.86545220717754 30.406361164100318, -88.86231741149453 30.40192888646436, -88.86221463130781 30.400155919077037, -88.86308826289208 30.397806687708197, -88.86154656009697 30.395546053228756, -88.86113543935178 30.39164522756994, -88.89771071582754 30.397594375784465, -88.96367134185533 30.392562177625777, -88.9988847218127 30.3889551588163, -89.02876310223307 30.381896512306966, -89.06249091772342 30.375298643695828, -89.09132142629706 30.36661173071002, -89.15228397327252 30.34467726544534, -89.20950545756013 30.32739195834955, -89.28962017895877 30.306601213601184, -89.28753864280388 30.329960313888193, -89.26464174509995 30.330730299285577, -89.26434438279222 30.345872114992346, -89.27594151279825 30.358702328641783, -89.2955674251156 30.377687956788307, -89.31608542435659 30.378201030709576, -89.34433484360159 30.374609456707674, -89.3719895382307 30.358958915753803, -89.40196486833926 30.362527468094967, -89.4069144962265 30.358078659729742, -89.40423344874931 30.338056520999984, -89.39113755829835 30.330758544446724, -89.37061722601652 30.33093655034706, -89.36185229185972 30.334051599177485, -89.34195066306378 30.341260336838317, -89.33287634527125 30.33333959628669, -89.32937035885126 30.323994078474982, -89.33586674545325 30.318653382216766, -89.45022735285862 30.241841361294732, -89.72519080019674 30.1853827101053, -89.74442426096313 30.189538988209947, -89.74151676565447 30.217371823584145, -89.74789064509477 30.23920738698216, -89.77360980774695 30.244230812668988, -89.79821074593612 30.24799821342394, -89.81285951253513 30.229642843318544, -89.85960603939652 30.2600242229622, -89.9498196387077 30.278116873447544, -90.04814272832806 30.35570209277128, -90.14612761646801 30.385436523941024, -90.22129593004662 30.398120990850558, -90.36816857121208 30.25738122356246, -90.38559413880824 30.34550155043857, -90.42044527400016 30.335833677463015, -90.40799844000294 30.269206841682944, -90.45693196431618 30.199325738388467, -90.45391345914385 30.10927978338424, -90.37769620353772 30.04528421934232, -90.24940925464897 30.030911956043212, -90.11810427964417 30.01392413180284, -90.03509538739952 30.02437852194518, -89.89548952317001 30.08446986687585, -89.85775820851362 30.059653949337033, -90.00943793114463 30.00934993897242, -90.01396568890365 29.988436888690757, -89.76310217811502 29.879737537766758, -89.84672013275741 29.83171270786133, -89.89870304367031 29.86097285426864, -89.89718467836123 29.896116034913007, -89.90466372576462 29.915087917481216, -89.92442408001551 29.927983974771934, -89.96174058575956 29.93398793227847, -89.97433687612305 29.932009395497445, -90.00417433892103 29.94838230847124, -90.03102003432386 29.9600462573987, -90.04385250513126 29.96311551844147, -90.05723606364236 29.962501673815296, -90.06558110600795 29.955885552228423, -90.06518747193438 29.94708611041341, -90.06510874511973 29.9316681571307, -90.07920090058741 29.921297170288227, -90.11202998234704 29.91747602500874, -90.12997959361982 29.91931843146682, -90.13297117239817 29.943197790827156, -90.13627769861874 29.957045164422198, -90.15399123194192 29.96161505656535, -90.16768969771238 29.955135296505773, -90.18563943593048 29.941287631680694, -90.20547859325302 29.933305603958516, -90.21626416687651 29.953839268552485, -90.23956732577103 29.972868428681068, -90.27586038738045 29.97723300824896, -90.29711303133598 29.967813413693122, -90.3286533798177 29.94268719873503, -90.35490123252325 29.949657148026233, -90.3834780583288 29.984316227341296, -90.40252938288317 29.997149861188376, -90.44698222302496 30.02244558120175, -90.47915761208075 30.062758748347363, -90.54583552569359 30.061842662870333, -90.61209142552444 30.05579667772109, -90.6573909864317 30.067521888328727, -90.69739854255963 30.052315488489953, -90.72661037657474 30.03307514980024, -90.80472036711068 30.01768017369831, -90.81657445781515 30.04352037904073, -90.838589197695 30.07320080686489, -90.88325373278816 30.079062575629138, -90.90865535572625 30.14534940238937, -90.94866291185417 30.145715504381627, -90.97639321341677 30.127225862392365, -90.98528378144515 30.15651510824216, -91.01618908935335 30.205738049189108, -91.03883886980725 30.2172625433504, -91.08477347128742 30.227139606797067, -91.10234292715336 30.270660026213363, -91.13218979116772 30.282907960627483, -91.18172295589748 30.29533727307505, -91.13578835354788 30.30685125984286, -91.14088882772371 30.348857643032105, -91.17475765827429 30.357077564142884, -91.22788945283354 30.35689497595287, -91.19634910435123 30.408573413207023, -91.18851693727851 30.450735899595514, -91.19084541938115 30.471901527982737, -91.20771763207254 30.471682825157856, -91.20363122195644 30.456764999260855, -91.21079332664378 30.426257909586653, -91.24936044346556 30.379407629405804, -91.2466444493232 30.337691982639384, -91.22165730321335 30.327377508055392, -91.23958286455287 30.31284163186575, -91.23632367158221 30.28188710067529, -91.2069909348445 30.272974136665383, -91.13800468362845 30.25561498930867, -91.17439912339707 30.220418543618493, -91.17711511753947 30.183800430696962, -91.1445231878311 30.159380789801645, -91.09889448623937 30.186617692184385, -91.05000659167659 30.198355414324766, -91.02501944556678 30.165486267240397, -91.02501944556678 30.149047582114633, -91.05163618816174 30.13119676467194, -91.0347970244791 30.10394455600519, -90.99242751585803 30.102534754503196, -90.94026287584776 30.09652246997878, -90.90223895785421 30.05233581931772, -90.86421503986104 30.042461637410085, -90.81587116790041 29.969079692188146, -90.77295846045108 29.985548294119525, -90.74090972957093 29.996369031357588, -90.65888670647155 30.031645848267246, -90.60728281776639 30.022239923979058, -90.50787750803998 30.039640652223, -90.4975567273352 30.005834127061178, -90.43291604072398 29.9799589380859, -90.41444728055583 29.96019517614171, -90.400324110403 29.935720310799624, -90.2982027469401 29.930071478209186, -90.27104280551615 29.951723857277713, -90.24822845472046 29.948429234068968, -90.23247568869472 29.923480688357486, -90.19608136142347 29.91453534563776, -90.15860064225852 29.937603273681376, -90.1732670106274 29.895229502665174, -90.15534144928789 29.889578304350366, -90.13307029732009 29.90229304994476, -90.0988487711262 29.90182216237106, -90.05104727422069 29.91924352022633, -90.04561528593591 29.949841227460226, -89.99346819840248 29.922539109973798, -89.93262992961327 29.905118328651355, -89.92719794132847 29.878745945394286, -89.97391304057709 29.89711316423103, -89.98694981246044 29.873564835985363, -90.0086777655996 29.834462335653157, -90.03855370116558 29.784030498511598, -90.0331217128808 29.74819431367912, -90.10265116292585 29.736874961024327, -90.15642784694478 29.85189546279588, -90.25909242552645 29.898525887039852, -90.32210348962944 29.84388602551222, -90.55458044594869 29.662632108329603, -90.65282882872005 29.631846768389707, -90.80433144006541 29.620558495655658, -91.00831876822983 29.702944243523902, -91.22063742310216 29.742496848784356, -91.34923421091148 29.754708472880495, -91.48921717264089 29.82852800680294, -91.73234584895138 30.098926382015065, -91.77521144488752 30.07980225252291, -91.73770404844312 29.989347240882395, -91.56825098950735 29.872677846829674, -91.59880920488436 29.772216840335815, -91.71540451906269 29.772216840335815, -91.79187865087634 29.864529052696398, -91.8344381571927 29.854724930186904, -91.83421649309709 29.903735908801167, -91.83687646224197 29.933515121047336, -91.8317781880478 29.950994250040253, -91.84485636967601 29.94696087751116, -91.85150629253764 29.93908572635995, -91.84374804919919 29.921412386585033, -91.84086641595869 29.852033433970064, -91.7943617536407 29.861712380635836, -91.71771737122155 29.77191795370274, -91.59788398748788 29.771247772939518, -91.55878757829488 29.80615847406702, -91.59780350127383 29.771199379379738, -91.71759903726709 29.771963967201955, -91.79423281887331 29.861635013533444, -91.86029653357544 29.84762920793392, -91.89641652855059 29.913205196481158, -91.89908477060337 29.92497867221121, -91.90684692930212 29.919933067375197, -91.91776246497282 29.893649741775633, -91.93171009388601 29.88902315887033, -91.9787681809997 29.947785818817636, -91.98580263732102 29.947680728751436, -91.98568135359156 29.9270809320804, -91.95499656998359 29.889969522844112, -91.93838071954443 29.847374286986124, -91.89675014615965 29.865556978900315, -92.08674042680882 29.778804130863392, -92.0185620964324 29.8939324587662, -92.09401278204915 29.949871534684164, -92.15491875718523 29.931753867739573, -92.13764691348997 29.863191991162438, -92.2349146648271 29.782749075164077, -92.21946090994177 29.713295359822467, -92.15128257956538 29.627200808963096, -92.37945308904833 29.604281757713025, -92.34854557927765 29.80010423032735, -92.61327279545509 29.94377323768117, -92.75767414115234 30.012976938706316, -92.87676810909444 29.985902955897842, -92.8732949956842 29.8409529251081, -92.63262593546887 29.738888856747025, -92.52742626378718 29.683722391675786, -92.73152664119964 29.728682960421352, -93.03239384687718 29.79076317007413, -93.22403986737179 29.782306124050905, -93.2498469619112 29.796594603078987, -93.24970172096623 29.79651451494965, -93.19599869562323 29.914173687803995, -93.26673534938253 30.031112286672737, -93.27641448374197 30.111525106588672, -93.30228419742978 30.112286280846178, -93.31970665766852 30.10436978192152, -93.32252240881854 30.11517868953682, -93.30597987081396 30.136336366122578, -93.30826766862289 30.153380979336816, -93.29350951940168 30.161429290760395, -93.28477358274677 30.16075623046477, -93.28589801023686 30.16389713920165, -93.29506641900313 30.167411847003137, -93.29506641900313 30.171823749614845, -93.28200576123254 30.172496734347845, -93.27396177995641 30.180347883321716, -93.27647011820363 30.186703117144503, -93.27335631899979 30.18647882175442, -93.26859912577186 30.172721061570954, -93.26782070840352 30.18266575965488, -93.26660978649076 30.189693711212797, -93.26323650401991 30.200608471390808, -93.25363562314234 30.20666339866071, -93.24256433708454 30.205841146444186, -93.2419588761284 30.200608471390808, -93.23910456019154 30.195674551570804, -93.23893157134718 30.192385134325505, -93.23893157134718 30.186927449698672, -93.23564478329851 30.182964831638458, -93.2310605789154 30.181020847517672, -93.22716832991105 30.17877774125077, -93.22318958648408 30.1793011372812, -93.21955682074625 30.177730940849713, -93.21912434863444 30.180048841071923, -93.22440050839637 30.181469462642653, -93.2280332741342 30.182291918399187, -93.2327039729395 30.184235877437928, -93.23564478329851 30.186777919836217, -93.23469334465324 30.198515323484173, -93.2382396159684 30.201655028655566, -93.23694219963345 30.20621489830313, -93.22950367931348 30.204196621401977, -93.22872522951252 30.21122303627476, -93.23192552313853 30.213839126403712, -93.23979651557032 30.20920486207953, -93.24957038529271 30.211522021524488, -93.24801348569079 30.215857205548602, -93.2336804043053 30.215916064896405, -93.22670111800174 30.218034263063103, -93.22108364429816 30.221799834947177, -93.21914306225261 30.221946924718097, -93.21774720499191 30.22206459637583, -93.21624921183405 30.220623108865226, -93.21536403405902 30.220299506724743, -93.21441076568564 30.22044659873869, -93.21468312807826 30.221387982420126, -93.21519380753209 30.222417610569977, -93.21658966470385 30.222947129544636, -93.2200622852064 30.223005964792492, -93.22064105529005 30.225065176308732, -93.22111534650244 30.228819430997305, -93.21634977982517 30.238177060462547, -93.20393042424291 30.244664493999124, -93.20234190290043 30.259758598742565, -93.17923614013823 30.265745751403472, -93.16002946229594 30.25751339461324, -93.15381978456779 30.266369386901133, -93.1821243624071 30.28133544462517, -93.16465061795974 30.28545071062092, -93.14515511774934 30.305401066861748, -93.15916299555764 30.315250058356426, -93.19439930674488 30.299167016542413, -93.20320840382101 30.271857229470214, -93.21476129273505 30.275598748660514, -93.22703623720614 30.29168565579421, -93.25331905948514 30.303406236731746, -93.2627057817277 30.29929172427771, -93.26674929284755 30.286947151004, -93.2551964039339 30.285575435844464, -93.24407674835412 30.292184432456082, -93.24104411501403 30.282956653646593, -93.22284831497474 30.26736721818581, -93.21057337050365 30.26237807591461, -93.24104411501403 30.259259733265097, -93.24941996314429 30.25252377986827, -93.2518749520385 30.234309432650008, -93.2696375187435 30.22682310582205, -93.27758012987204 30.227821315665864, -93.28480068544313 30.215467755730856, -93.29664239657991 30.21459421296585, -93.33057890713481 30.173029647739597, -93.34566293738251 30.14275483650289, -93.69262987049676 30.062135005872378, -93.70708034193744 30.092462535574853, -93.68067085964903 30.108125721095533, -93.70758269976227 30.132165930944126, -93.70528049926865 30.124201195538262, -93.72939805691013 30.114690468453162, -93.72595467091689 30.101753955065092, -93.72831584988394 30.093752895321927, -93.73293982536067 30.095114823565993, -93.73854762540739 30.085921443844086, -93.72988996662475 30.07689747970319, -93.73166085085003 30.072385189475142, -93.72497084377711 30.064296607108048, -93.72595466834653 30.05390819496634, -93.74002999605626 30.063065987356026, -93.74541454787031 30.06722614833754, -93.74454806826803 30.06924367417443, -93.73893423973793 30.07528588742892, -93.74555346297119 30.075190419686862, -93.7579093463391 30.066502469604828, -93.75140044349348 30.062492389240248, -93.73452135080838 30.046736932988424, -93.74985588463124 30.030214880426286, -93.78074559305165 30.039670033815966, -93.80634004624282 30.048837956020733, -93.80843613359981 30.04282170136628, -93.82575643439266 30.051225257388083, -93.8425251332492 30.081586708395292, -93.8490340360948 30.081395784980728, -93.8500269195798 30.073663076998315, -93.82895572562164 30.045304644395287, -93.80115498701988 30.034512943120433, -93.76629374466015 30.02066345857564, -93.83160341389097 29.989805583760514, -93.874187076058 30.029259893728266, -93.88775648368494 30.0272541198362, -93.877386367287 30.01607835095632, -93.8864968421437 29.980623194316834, -93.96729519944624 30.018790631320115, -94.01234640473022 30.04719463330929, -94.04466574765178 30.083217842433967, -94.09559318693952 30.094657862467486, -94.10636630124627 30.08152302444813, -94.08286132457641 30.073471777602364, -94.06670165311564 30.070081582913843, -94.04499622058584 30.055640438095892, -94.03156100989943 30.020560553680113, -93.97933704577943 30.00536166953762, -93.95875132692545 29.99654167362405, -93.9349146628044 29.987157650074863, -93.91346166509481 29.96463237502013, -93.89850957578284 29.97307995172639, -93.87965694143264 29.968762391138114, -93.86817200326476 29.96557103011959, -93.88507420812198 29.931022628966787, -93.93903174781434 29.879929185149194, -93.95441688826256 29.853056940961906, -94.00317370123788 29.88086852819245, -94.02939403177098 29.890262582476396, -94.0534473928388 29.90040716680214, -94.07121654245634 29.88443837309022, -94.05468578804707 29.87167504810246, -94.03428967676004 29.884026652382644, -93.99641118436914 29.873640178379063, -93.98702249822128 29.827870674185206, -93.9630651611536 29.77533627130596, -94.02295848388682 29.759036582660002, -94.02004475370276 29.733457577137074, -93.92518664882718 29.74976143372963, -93.88050945267436 29.709278015231533, -93.99090770640181 29.710684166216694, -94.32566292975143 29.738798990571325, -94.32274919956737 29.714620715783127, -94.39779525181304 29.638502027968116, -94.40625827009087 29.580252553267556, -94.45421507656677 29.542832561693302, -94.53249751066701 29.63911494322545, -94.58891728299156 29.589452066737593, -94.67143116944828 29.604169469795565, -94.67354691091056 29.732243309936507, -94.6340530702833 29.8050928037475, -94.69611481984049 29.845473872345863, -94.72291421169449 29.808152546747806, -94.78471050949359 29.801568300143316, -94.81091700383959 29.822434090916317, -94.83989830346987 29.783106471914465, -94.84020661516794 29.752061521762712, -94.88645327275546 29.715919373380146, -94.89909405238139 29.702530140439848, -94.88244522067892 29.683246509064844, -94.90402703955257 29.671728131025972, -94.93300833918242 29.700923312518327, -94.92530054672777 29.730110010774, -94.94534080711027 29.736267587514817, -94.95705665164152 29.713777216088985, -94.98048834070444 29.716990434884508, -94.97093067755875 29.739212382453985, -94.98357145718464 29.74483402255065, -94.99374574322471 29.731716372127664, -95.00854470473793 29.730377739909073, -95.05263333655003 29.812536743889837, -95.0504751546627 29.839551472502222, -95.07976476599107 29.839551472502222, -95.089322428635 29.898370146551343, -95.1035328223992 29.888157387730217, -95.10236897530885 29.85730435824358, -95.10824304262674 29.84254647444662, -95.10319496607534 29.83438048990567, -95.09459959248773 29.81982207937142, -95.09473602698927 29.815205555173478, -95.09801045502239 29.811299098966444, -95.09787402052126 29.804551224247504, -95.08750499841538 29.797566102502813, -95.07249720326264 29.795434951318725, -95.08641354523554 29.790817278144274, -95.09419031181477 29.78904120219987, -95.10305855440504 29.78347596015547, -95.1041500304161 29.777910408718157, -95.10401359591492 29.767370538264572, -95.10142134038864 29.759553738373242, -95.11397331451624 29.747116663630678, -95.1287082406663 29.740482926156176, -95.14849124336777 29.742141401668135, -95.17250367925624 29.750670282933044, -95.20511152508836 29.74462907541779, -95.21234255366187 29.733256403242237, -95.22666818994684 29.727924976561567, -95.24222172310529 29.72626626598607, -95.25968533928312 29.73005756415354, -95.26882645087606 29.732663998539905, -95.27605747944996 29.73989057604433, -95.29188388161106 29.75410527432048, -95.29584048215105 29.74640589625679, -95.28519859104289 29.74249675492871, -95.27876807686125 29.723687450331255, -95.25719509575421 29.721605811990997, -95.24075853872014 29.71446843847592, -95.23014326230232 29.721308431561155, -95.21336427699637 29.72309270092596, -95.20206414403528 29.73260826841573, -95.19076401107465 29.740636326492904, -95.17124559959652 29.741230971899057, -95.15549389910574 29.732013571886682, -95.14282405305853 29.730824168251658, -95.12193592849431 29.731716222299724, -95.10481451491695 29.736473709922407, -95.08015967936586 29.754609561261205, -95.06166855270241 29.72933739387878, -95.0644079788746 29.71625282943326, -95.04009557159539 29.696325330233748, -95.01711638030143 29.685376874720674, -95.00908919208999 29.677485291546546, -94.98874749292001 29.680453203130924, -95.00191746695495 29.673411676595208, -95.01731620582653 29.651931975386347, -95.01853189573751 29.625868478144724, -95.00515900567457 29.617061728512383, -94.99198903163962 29.600855279814496, -95.00414593074892 29.589579692301058, -95.01670805982837 29.566319813514426, -95.0278567810222 29.562813812877465, -95.03932500845706 29.561883913345525, -95.04748883137677 29.566026127212268, -95.04467036870174 29.572112337346198, -95.04360129665302 29.580057670710474, -95.03776999456767 29.584368175007967, -95.03738124109493 29.592059009672056, -95.04010251540149 29.59696055445555, -95.04389286175733 29.59163645155833, -95.05169479969452 29.584076680593896, -95.05179317811212 29.57885781042063, -95.05789264001456 29.571670727914196, -95.06812399546259 29.57141403693386, -95.08307757492632 29.555840231237, -95.08878352315728 29.544543440796446, -95.08691381464658 29.537503858731966, -95.09251153154948 29.538016510007964, -95.10246957530299 29.551498305403484, -95.10447296872115 29.551088239284425, -95.10665313214615 29.545808489382566, -95.09828601845983 29.539913297130553, -95.10310485864841 29.536160926964726, -95.11475016288787 29.518907781224172, -95.11600911469748 29.50685622011136, -95.10460168058005 29.50897205501309, -95.09814069617579 29.513337118354073, -95.08751415050634 29.52206652934612, -95.08951194109257 29.525654246852625, -95.08900186690042 29.5275405215189, -95.0840711497098 29.530758203007622, -95.06991657853949 29.530314400578305, -95.06949151671301 29.53286629327495, -95.0727219865963 29.538450645402804, -95.07004413024237 29.54070649646617, -95.06456083267724 29.541667987966363, -95.05916254747733 29.545033136204097, -95.06103281951505 29.547806525817265, -95.04947110934211 29.548398163531544, -95.04777086203525 29.54122421490601, -95.04037478624959 29.540595545662256, -95.03960967496135 29.544034688427203, -95.03433890830946 29.541039312593142, -95.02762293144634 29.537193267816377, -95.0271128572542 29.541594018517983, -95.03353129083841 29.545291980233316, -95.0317460311659 29.549840287614145, -95.03000327767622 29.545033127316273, -95.02490253575519 29.544700315448736, -95.02213963388121 29.546808105434877, -95.02171457205426 29.543886770716398, -95.01268773806747 29.531514619958756, -95.0006468517448 29.517590823524188, -94.96183083662575 29.504078773675772, -94.91556848391212 29.49290928953721, -94.92174735978816 29.481738573690407, -94.9360063041176 29.487944678984718, -94.93901652569812 29.482703992810798, -94.96278143291526 29.4701528269555, -94.9712215590741 29.468865564606375, -94.97596527961582 29.464360017693878, -94.98083221367794 29.45894236886963, -94.98927238661152 29.461087961702717, -95.01397669748499 29.468704610397623, -95.03165783768594 29.461248882730843, -95.0497086511259 29.460068762337826, -95.0513720336536 29.456367471040082, -95.06030501389452 29.459049579656778, -95.07034691582054 29.45384622428398, -95.07829418789667 29.443921559270265, -95.08482447577919 29.444243464749505, -95.10047259289075 29.433995932967832, -95.09794671571927 29.430025407059418, -95.09067711800607 29.4353372917725, -95.0834691843731 29.43726892608018, -95.06985409035096 29.445638445480782, -95.06135235704521 29.453631765227797, -95.0514953916715 29.450949558712324, -95.04576596296565 29.45438282837111, -95.0307339134565 29.45615306264456, -95.01557903819992 29.454425947064337, -95.01557904009124 29.46125852776464, -95.00750814936504 29.458916242486566, -94.97881031223054 29.448569347820865, -94.96715181589423 29.461063578960008, -94.9568385306737 29.4349020209783, -94.98082812890358 29.421037567103184, -94.98015552334631 29.412249257408533, -94.95863214549453 29.415569374846683, -94.94473163063235 29.405413379983415, -94.93445000956034 29.414830148956128, -94.93673852867671 29.419258156169064, -94.92923452713282 29.42135789497378, -94.92581301347566 29.416565685658185, -94.92281918902552 29.42127322892614, -94.92095290876074 29.41595606011161, -94.91283893603901 29.41503146037266, -94.89348026395686 29.419960539684197, -94.89243787392151 29.39336661998813, -94.89511830544069 29.380521211963256, -94.89427188103559 29.36710067567698, -94.91554161955098 29.368985724230072, -94.9213096842327 29.36490140825572, -94.94191671643284 29.34657216162735, -94.94859074770557 29.341474742762884, -94.96168456144107 29.34601810670938, -94.97655810382496 29.348622129893442, -94.99340208751386 29.35316517518936, -95.00541534380568 29.354993416740015, -95.00865701613814 29.351613914205586, -95.01234362388907 29.351558511590525, -95.01355130573843 29.34939778611303, -95.00764001809746 29.348012683092108, -95.00261860409185 29.35205713555284, -94.97808368580792 29.34330308455523, -94.97223596316857 29.33632154853987, -94.96168463753676 29.332664362659017, -94.94903575921973 29.332165645330193, -94.94998115218893 29.329875010613918, -94.9534445992452 29.30378027015867, -95.00247489617824 29.312014471624238, -95.0133705177191 29.343678284820072, -95.04387825803336 29.349376726989064, -95.02426613925994 29.3237312282709, -95.0093754564876 29.305680529456026, -95.02281338972125 29.30124653606886, -95.07438589402585 29.256579375847053, -95.16808840358807 29.332913986554658, -95.16046146850961 29.297762844271972, -95.13917171778971 29.27860008091794, -95.09494426156525 29.233896880486412, -95.1575099313462 29.21177333846613, -95.23355958168354 29.27860008091794, -95.25189779524005 29.25083933962945, -95.22816598946099 29.233426216586253, -95.22600855257213 29.21789309411875, -95.1970187850246 29.187406554834695, -95.28968450816338 29.11162498262354, -95.30606033508963 29.080750056288935, -95.27109460019105 28.989952019080036, -95.28796073866363 29.004703823464766, -95.33141322934401 29.027784535849563, -95.38142534200924 28.977402739874762, -95.39693926153996 28.934457767202716, -95.39270819257717 28.891494992829365, -95.41865874888319 28.86481949682384, -95.48776620861116 28.910262531120296, -95.54252642124848 28.96127247245775, -95.56450157851525 28.990638839752933, -95.57834419726616 28.983222348195696, -95.57090378968753 28.973383324482597, -95.56056935907125 28.93979983178373, -95.5248274072886 28.894293423252662, -95.49575501286823 28.894443148141022, -95.45693481561273 28.86913657722907, -95.46634059027804 28.86179801626291, -95.69665746768966 28.76901717121686, -95.76389924024677 28.78804405798941, -95.94921167034799 28.721202895943165, -96.0127473390786 28.8432484879893, -95.99792234467064 28.991089077121742, -96.01486519542287 28.993404649392247, -96.02492501305687 28.94893660419274, -96.05986964273322 28.887298278919744, -96.03445554272298 28.84649479894243, -96.01168858702454 28.770409421506926, -95.99262787992845 28.701234936578786, -96.0863430231512 28.662681650853273, -96.17426924696058 28.678784221377157, -96.17980513691288 28.72661579650969, -96.16034386452678 28.752867940513582, -96.14468272225176 28.75912709300914, -96.14468272225176 28.784664582931214, -96.13754303589086 28.80545342285859, -96.14099769962819 28.810498611518867, -96.1380036577223 28.8159471408447, -96.13535508219077 28.833501574136136, -96.14019161142238 28.82381673557559, -96.14698578343871 28.818267724194655, -96.1507858430672 28.792132922427797, -96.1640031322208 28.781322652695067, -96.1614908809851 28.772190856813793, -96.15299356062914 28.77186701958928, -96.15232855294906 28.76610254883957, -96.19363291885371 28.765649149993223, -96.18890397393926 28.798677348710342, -96.19562794048169 28.797900335057705, -96.1990507500516 28.773980030839553, -96.20587480477533 28.77163526581961, -96.19883240663079 28.76077217086791, -96.18654915405268 28.760915743371456, -96.18152666855403 28.75321040660988, -96.19320940656182 28.74813701731759, -96.20467373081209 28.73387276594427, -96.20740334249602 28.728319714545137, -96.20472829956829 28.716542396536326, -96.21242580451701 28.709599885542588, -96.21122477537638 28.70160345711942, -96.22858510568642 28.70289633426843, -96.23768651772347 28.71202544891497, -96.24509796718925 28.716472844103805, -96.27981475679401 28.751703211523974, -96.29502773201362 28.746573314427167, -96.29580788458922 28.73289235712599, -96.2731834599033 28.711341217486236, -96.29970864746616 28.6966291593139, -96.31609185154889 28.664118443793555, -96.3375460473717 28.65898425059204, -96.37889413386664 28.68499490490315, -96.38006436272977 28.73768089595177, -96.41375913433784 28.778081723945476, -96.38638509184312 28.805380642677264, -96.39928550267406 28.8271594991389, -96.40730892892257 28.82274895929541, -96.399757468924 28.805518497860263, -96.42225452683628 28.77477227843255, -96.40107724009474 28.755114970260095, -96.42232938912177 28.73757804854074, -96.4288925527918 28.776210176145113, -96.4598331815223 28.77018340956033, -96.45139482823254 28.71400817460679, -96.43233040042885 28.69920577090633, -96.43108027401516 28.672336801595677, -96.4348306532554 28.642169544706732, -96.46327102915933 28.69262625265351, -96.48358558337631 28.678369211134466, -96.47921014092962 28.653414718835094, -96.55327976796738 28.67590113232761, -96.55168617306086 28.74303799241541, -96.53861398528062 28.878850157993355, -96.60024001338537 28.86494982428961, -96.58996900870157 28.81996542173853, -96.59650510259138 28.736488358148932, -96.65133943568301 28.746760855121423, -96.69541766318602 28.811843797831614, -96.73135837176504 28.834420164047415, -96.73949589068857 28.823726706046557, -96.72186459968765 28.813329235504597, -96.70965832130211 28.77678132147763, -96.67507386587695 28.742598921141493, -96.66863166339559 28.714352846932357, -96.68900881217041 28.698050471877337, -96.6234818497111 28.64354470118782, -96.63036054028906 28.622139073408135, -96.62973520478178 28.59386597997232, -96.65318528629815 28.599631014584205, -96.67263422667068 28.580446443718074, -96.6231545927881 28.55659504815449, -96.57888334141941 28.57391235251461, -96.51935895154382 28.52554749407821, -96.5688385854264 28.509856816579116, -96.5744189952626 28.481738515346578, -96.52568341602507 28.45230436698101, -96.47545972749742 28.495471644847626, -96.42858428487179 28.45688355072768, -96.63319931220614 28.362315932317813, -96.73215915491556 28.435294446414815, -96.81995760300786 28.565417665881697, -96.88245819317572 28.53600686065073, -96.83335058661542 28.45099620469415, -96.86757710027858 28.439547236091045, -96.86311277240922 28.397338911166628, -96.80544846809254 28.359697139466448, -96.82590997082586 28.311561713759716, -96.81214495989654 28.2509521455748, -96.88617839705903 28.1883406959019, -96.91134804921221 28.287335478437242, -96.98249188827423 28.20720201314404, -96.96891481975119 28.194997311077444, -96.99281033365365 28.1468827362184, -96.99335341639474 28.180158223006345, -97.00312890573129 28.21868739587191, -97.12369327421825 28.160529331144147, -97.2062418508397 28.214141197918266, -97.23230982240433 28.181833693268842, -97.20732801632148 28.17369544897433, -97.17392842775426 28.131079243082382, -97.23231014444083 28.073113115557362, -97.21520303810132 28.065445836207488, -97.16415326045386 28.033093480048905, -97.20053980409611 27.967160443588426, -97.18288961501591 27.962603580669565, -97.14541690589184 27.990661206498743, -97.13672758203671 28.017033512384955, -97.11011652773136 28.02590274599298, -97.09979795565336 28.048671755420358, -97.11147423458353 28.058496891732418, -97.07943235286847 28.0769465500414, -97.06961212785326 28.083807665990662, -97.05544383951 28.08913165631141, -97.04521118681781 28.100704637432543, -97.04337455684743 28.115458378243787, -97.02592657212847 28.109383554253895, -97.03530650376301 28.09231435051592, -97.03878213282049 28.075964090240717, -97.03655675830132 28.066993246252267, -97.0348550099247 28.05513364846543, -97.03660039287476 28.051667934538116, -97.03825850667745 28.05482558952876, -97.04044023536575 28.05405543832569, -97.03948027214011 28.05247660725334, -97.03677492856718 28.04920335503128, -97.03908756097627 28.050551176837715, -97.04026569446778 28.04893378864361, -97.03572769879743 28.046315108562226, -97.03324052809313 28.05039714091437, -97.03411321956818 28.051744947756575, -97.0332841626666 28.05047415890361, -97.03550952592832 28.04666170223463, -97.04476005556468 28.035993807411458, -97.04838172518654 28.028444766229555, -97.04746539913809 28.027096667436993, -97.05017075085699 28.026557409945084, -97.05130524977459 28.021126299438606, -97.05636686033051 28.01103372116536, -97.0589867994674 28.002711326953587, -97.06618650413724 27.993618790264577, -97.07548067726512 27.99681665906334, -97.08054228782056 27.990151077824763, -97.08032939169178 27.98786719984929, -97.10368336254795 27.95805690163671, -97.16195931291257 27.99093899262411, -97.16413977159758 27.994373422025532, -97.16761671922983 27.99359287949777, -97.16402190896618 27.990886954973845, -97.11073620425427 27.95845213552674, -97.13416792551133 27.918146366939084, -97.15008268735903 27.89580940616173, -97.18136778241599 27.852956215761353, -97.19565338851794 27.84916696595708, -97.21632904823488 27.873173689882975, -97.23903255322799 27.885716431938718, -97.32275247409505 27.894495892870253, -97.38234917470041 27.887389093677086, -97.43485102999585 27.876518969047808, -97.4830959781043 27.894495892870253, -97.52140814277931 27.886134904214018, -97.52802999840206 27.861048064691943, -97.51667824590585 27.83762844129403, -97.47931206060579 27.81880543690494, -97.45566257623838 27.82173367419078, -97.41876938062616 27.816713790495314, -97.40694463844213 27.82298860885217, -97.39886375307738 27.784516647225516, -97.38295426527458 27.74731145159018, -97.34545332973906 27.719483042104173, -97.33143782857924 27.680578424352717, -97.34318054576708 27.675546642349588, -97.35605965494092 27.642331069625968, -97.37159034541521 27.648035471456012, -97.36325680418483 27.627229746708494, -97.3351429834788 27.620746208074365, -97.43364662828488 27.350267337536664, -97.45009007152534 27.40418148388663, -97.52408556610567 27.48332021501315, -97.59681618043724 27.519783480652322, -97.62590842616979 27.510809028521777, -97.56456173408156 27.45638574407164, -97.52851264697796 27.39295153682572, -97.60503790205705 27.33509925203768, -97.65500067190236 27.376104477528216, -97.77895893632798 27.472098318799127, -97.79919702031593 27.411480337628674, -97.77326697520651 27.39239000952699, -97.71761224423975 27.363186661263654, -97.70116880099972 27.315995746990637, -97.75432882031383 27.293758853353538, -97.74595633243622 27.24042350979643, -97.67060394153732 27.254069870405573, -97.6050194531627 27.21064382409361, -97.4682688178282 27.225534662367977, -97.51431750115526 26.827766677022623, -97.44657622147527 26.5357353184787, -97.38020364099295 26.36815722433265, -97.43210005657703 26.355186178091273, -97.46487664736803 26.337807760275822, -97.4859082931258 26.345151072188003, -97.52578641993 26.330708674203947, -97.5457255126611 26.29937005511303, -97.59161273976888 26.28737099680103, -97.5932515693085 26.234217393542508, -97.57986779473522 26.23495240179338, -97.57659013565635 26.265328676390155, -97.57030795575461 26.280513834670515, -97.55665104292487 26.273901187363975, -97.5436650685455 26.287103691223138, -97.52148376471 26.294726962989188, -97.52148376471 26.319913340032628, -97.48821180895727 26.331510488240895, -97.47675146864243 26.32123878717998, -97.44421888968391 26.32057606550231, -97.41944976706766 26.339462147089762, -97.35290585556173 26.347744544118, -97.3240701605763 26.246327257697217, -97.30410696289663 26.164067790057246, -97.28858005021209 26.12756316713029, -97.30373727449953 26.121588596635007, -97.29708284896466 26.10266704763758, -97.26763355051382 26.084571236050152, -97.24482695096206 26.08039488207578, -97.22622739404532 26.074030627590375, -97.2127205729516 26.0497637342838, -97.30416845742035 26.013751896001523, -97.30948261653897 25.999025665465695, -97.40646602046021 25.953840443474945, -97.40358751760375 25.944482744034687, -97.39468339036422 25.949004139493397, -97.34253761233443 25.96524790249275, -97.29614019156341 25.989978670049794, -97.18030945184529 26.053846163306005, -97.16768298993466 26.00772373752909, -97.1660627359436 25.96045460847111, -97.15840335344006 25.961778932402723, -97.16061279070078 25.950654148207846, -97.15589932454472 25.949594590115268, -97.14617780059835 25.956613984790593))'
Gulf_withorigin_poly=Polygon([(-97.146177,25.9566),(-95.419039,24.3059),(-84.849600,25.6822),(-87.120068,30.3340),(-87.206591,30.4157),(-87.233444,30.4065),(-87.266027,30.4148),(-87.262804,30.3904),(-87.275940,30.3693),(-87.304258,30.3623),(-87.398924,30.3448),(-87.398114,30.3804),(-87.335813,30.4900),(-87.383551,30.4432),(-87.430479,30.4558),(-87.455561,30.4244),(-87.469316,30.3616),(-87.510098,30.3532),(-87.582917,30.3406),(-87.595863,30.3015),(-87.728556,30.2854),(-87.845067,30.4104),(-87.899619,30.4702),(-87.888251,30.5289),(-87.896686,30.5662),(-87.896686,30.6012),(-87.910621,30.6287),(-87.908420,30.6612),(-87.932990,30.6867),(-87.964527,30.6867),(-87.998264,30.6940),(-88.028701,30.6893),(-88.049236,30.6716),(-88.065005,30.6445),(-88.068671,30.5984),(-88.087740,30.5896),(-88.105342,30.6148),(-88.127344,30.6113),(-88.131011,30.5902),(-88.114510,30.5694),(-88.125044,30.5171),(-88.130120,30.4099),(-88.399978,30.4208),(-88.517997,30.3465),(-88.540025,30.3464),(-88.539928,30.3840),(-88.548099,30.3946),(-88.564439,30.4132),(-88.545317,30.4150),(-88.522892,30.4105),(-88.512549,30.4197),(-88.493948,30.4167),(-88.480562,30.4249),(-88.485777,30.4342),(-88.506638,30.4274),(-88.527238,30.4268),(-88.535546,30.4292),(-88.546146,30.4389),(-88.618012,30.4327),(-88.623080,30.4175),(-88.620436,30.3774),(-88.667150,30.3690),(-88.725536,30.3487),(-88.805555,30.3939),(-88.818367,30.4019),(-88.821642,30.4073),(-88.825110,30.4045),(-88.834349,30.4082),(-88.835127,30.4152),(-88.811794,30.4214),(-88.809201,30.4275),(-88.842677,30.4303),(-88.856736,30.4311),(-88.889474,30.4247),(-88.904461,30.4320),(-88.918031,30.4293),(-88.919344,30.4309),(-88.927741,30.4347),(-88.938238,30.4266),(-88.955821,30.4271),(-88.979440,30.4298),(-88.997285,30.4431),(-89.017492,30.4431),(-89.029301,30.4411),(-89.023528,30.4307),(-89.010070,30.4206),(-89.049219,30.4281),(-89.064740,30.4278),(-89.084056,30.4330),(-89.087850,30.4318),(-89.081296,30.4296),(-89.065430,30.4238),(-89.048356,30.4229),(-89.044481,30.4226),(-89.041314,30.4086),(-89.030962,30.4028),(-89.004928,30.4135),(-88.994885,30.4115),(-88.988860,30.4089),(-88.984765,30.4108),(-88.979589,30.4130),(-88.970937,30.4146),(-88.965298,30.4136),(-88.962130,30.4140),(-88.941766,30.4143),(-88.936216,30.4109),(-88.926092,30.4121),(-88.922700,30.4158),(-88.914632,30.4172),(-88.911035,30.4157),(-88.908003,30.4118),(-88.902041,30.4107),(-88.895875,30.4119),(-88.881485,30.4098),(-88.873777,30.4066),(-88.865452,30.4063),(-88.862317,30.4019),(-88.862214,30.4001),(-88.863088,30.3978),(-88.861546,30.3955),(-88.861135,30.3916),(-88.897710,30.3975),(-88.963671,30.3925),(-88.998884,30.3889),(-89.028763,30.3818),(-89.062490,30.3752),(-89.091321,30.3666),(-89.152283,30.3446),(-89.209505,30.3273),(-89.289620,30.3066),(-89.287538,30.3299),(-89.264641,30.3307),(-89.264344,30.3458),(-89.275941,30.3587),(-89.295567,30.3776),(-89.316085,30.3782),(-89.344334,30.3746),(-89.371989,30.3589),(-89.401964,30.3625),(-89.406914,30.3580),(-89.404233,30.3380),(-89.391137,30.3307),(-89.370617,30.3309),(-89.361852,30.3340),(-89.341950,30.3412),(-89.332876,30.3333),(-89.329370,30.3239),(-89.335866,30.3186),(-89.450227,30.2418),(-89.725190,30.1853),(-89.744424,30.1895),(-89.741516,30.2173),(-89.747890,30.2392),(-89.773609,30.2442),(-89.798210,30.2479),(-89.812859,30.2296),(-89.859606,30.2600),(-89.949819,30.2781),(-90.048142,30.3557),(-90.146127,30.3854),(-90.221295,30.3981),(-90.368168,30.2573),(-90.385594,30.3455),(-90.420445,30.3358),(-90.407998,30.2692),(-90.456931,30.1993),(-90.453913,30.1092),(-90.377696,30.0452),(-90.249409,30.0309),(-90.118104,30.0139),(-90.035095,30.0243),(-89.895489,30.0844),(-89.857758,30.0596),(-90.009437,30.0093),(-90.013965,29.9884),(-89.763102,29.8797),(-89.846720,29.8317),(-89.898703,29.8609),(-89.897184,29.8961),(-89.904663,29.9150),(-89.924424,29.9279),(-89.961740,29.9339),(-89.974336,29.9320),(-90.004174,29.9483),(-90.031020,29.9600),(-90.043852,29.9631),(-90.057236,29.9625),(-90.065581,29.9558),(-90.065187,29.9470),(-90.065108,29.9316),(-90.079200,29.9212),(-90.112029,29.9174),(-90.129979,29.9193),(-90.132971,29.9431),(-90.136277,29.9570),(-90.153991,29.9616),(-90.167689,29.9551),(-90.185639,29.9412),(-90.205478,29.9333),(-90.216264,29.9538),(-90.239567,29.9728),(-90.275860,29.9772),(-90.297113,29.9678),(-90.328653,29.9426),(-90.354901,29.9496),(-90.383478,29.9843),(-90.402529,29.9971),(-90.446982,30.0224),(-90.479157,30.0627),(-90.545835,30.0618),(-90.612091,30.0557),(-90.657390,30.0675),(-90.697398,30.0523),(-90.726610,30.0330),(-90.804720,30.0176),(-90.816574,30.0435),(-90.838589,30.0732),(-90.883253,30.0790),(-90.908655,30.1453),(-90.948662,30.1457),(-90.976393,30.1272),(-90.985283,30.1565),(-91.016189,30.2057),(-91.038838,30.2172),(-91.084773,30.2271),(-91.102342,30.2706),(-91.132189,30.2829),(-91.181722,30.2953),(-91.135788,30.3068),(-91.140888,30.3488),(-91.174757,30.3570),(-91.227889,30.3568),(-91.196349,30.4085),(-91.188516,30.4507),(-91.190845,30.4719),(-91.207717,30.4716),(-91.203631,30.4567),(-91.210793,30.4262),(-91.249360,30.3794),(-91.246644,30.3376),(-91.221657,30.3273),(-91.239582,30.3128),(-91.236323,30.2818),(-91.206990,30.2729),(-91.138004,30.2556),(-91.174399,30.2204),(-91.177115,30.1838),(-91.144523,30.1593),(-91.098894,30.1866),(-91.050006,30.1983),(-91.025019,30.1654),(-91.025019,30.1490),(-91.051636,30.1311),(-91.034797,30.1039),(-90.992427,30.1025),(-90.940262,30.0965),(-90.902238,30.0523),(-90.864215,30.0424),(-90.815871,29.9690),(-90.772958,29.9855),(-90.740909,29.9963),(-90.658886,30.0316),(-90.607282,30.0222),(-90.507877,30.0396),(-90.497556,30.0058),(-90.432916,29.9799),(-90.414447,29.9601),(-90.400324,29.9357),(-90.298202,29.9300),(-90.271042,29.9517),(-90.248228,29.9484),(-90.232475,29.9234),(-90.196081,29.9145),(-90.158600,29.9376),(-90.173267,29.8952),(-90.155341,29.8895),(-90.133070,29.9022),(-90.098848,29.9018),(-90.051047,29.9192),(-90.045615,29.9498),(-89.993468,29.9225),(-89.932629,29.9051),(-89.927197,29.8787),(-89.973913,29.8971),(-89.986949,29.8735),(-90.008677,29.8344),(-90.038553,29.7840),(-90.033121,29.7481),(-90.102651,29.7368),(-90.156427,29.8518),(-90.259092,29.8985),(-90.322103,29.8438),(-90.554580,29.6626),(-90.652828,29.6318),(-90.804331,29.6205),(-91.008318,29.7029),(-91.220637,29.7424),(-91.349234,29.7547),(-91.489217,29.8285),(-91.732345,30.0989),(-91.775211,30.0798),(-91.737704,29.9893),(-91.568250,29.8726),(-91.598809,29.7722),(-91.715404,29.7722),(-91.791878,29.8645),(-91.834438,29.8547),(-91.834216,29.9037),(-91.836876,29.9335),(-91.831778,29.9509),(-91.844856,29.9469),(-91.851506,29.9390),(-91.843748,29.9214),(-91.840866,29.8520),(-91.794361,29.8617),(-91.717717,29.7719),(-91.597883,29.7712),(-91.558787,29.8061),(-91.597803,29.7711),(-91.717599,29.7719),(-91.794232,29.8616),(-91.860296,29.8476),(-91.896416,29.9132),(-91.899084,29.9249),(-91.906846,29.9199),(-91.917762,29.8936),(-91.931710,29.8890),(-91.978768,29.9477),(-91.985802,29.9476),(-91.985681,29.9270),(-91.954996,29.8899),(-91.938380,29.8473),(-91.896750,29.8655),(-92.086740,29.7788),(-92.018562,29.8939),(-92.094012,29.9498),(-92.154918,29.9317),(-92.137646,29.8631),(-92.234914,29.7827),(-92.219460,29.7132),(-92.151282,29.6272),(-92.379453,29.6042),(-92.348545,29.8001),(-92.613272,29.9437),(-92.757674,30.0129),(-92.876768,29.9859),(-92.873294,29.8409),(-92.632625,29.7388),(-92.527426,29.6837),(-92.731526,29.7286),(-93.032393,29.7907),(-93.224039,29.7823),(-93.249846,29.7965),(-93.249701,29.7965),(-93.195998,29.9141),(-93.266735,30.0311),(-93.276414,30.1115),(-93.302284,30.1122),(-93.319706,30.1043),(-93.322522,30.1151),(-93.305979,30.1363),(-93.308267,30.1533),(-93.293509,30.1614),(-93.284773,30.1607),(-93.285898,30.1638),(-93.295066,30.1674),(-93.295066,30.1718),(-93.282005,30.1724),(-93.273961,30.1803),(-93.276470,30.1867),(-93.273356,30.1864),(-93.268599,30.1727),(-93.267820,30.1826),(-93.266609,30.1896),(-93.263236,30.2006),(-93.253635,30.2066),(-93.242564,30.2058),(-93.241958,30.2006),(-93.239104,30.1956),(-93.238931,30.1923),(-93.238931,30.1869),(-93.235644,30.1829),(-93.231060,30.1810),(-93.227168,30.1787),(-93.223189,30.1793),(-93.219556,30.1777),(-93.219124,30.1800),(-93.224400,30.1814),(-93.228033,30.1822),(-93.232703,30.1842),(-93.235644,30.1867),(-93.234693,30.1985),(-93.238239,30.2016),(-93.236942,30.2062),(-93.229503,30.2041),(-93.228725,30.2112),(-93.231925,30.2138),(-93.239796,30.2092),(-93.249570,30.2115),(-93.248013,30.2158),(-93.233680,30.2159),(-93.226701,30.2180),(-93.221083,30.2217),(-93.219143,30.2219),(-93.217747,30.2220),(-93.216249,30.2206),(-93.215364,30.2202),(-93.214410,30.2204),(-93.214683,30.2213),(-93.215193,30.2224),(-93.216589,30.2229),(-93.220062,30.2230),(-93.220641,30.2250),(-93.221115,30.2288),(-93.216349,30.2381),(-93.203930,30.2446),(-93.202341,30.2597),(-93.179236,30.2657),(-93.160029,30.2575),(-93.153819,30.2663),(-93.182124,30.2813),(-93.164650,30.2854),(-93.145155,30.3054),(-93.159162,30.3152),(-93.194399,30.2991),(-93.203208,30.2718),(-93.214761,30.2755),(-93.227036,30.2916),(-93.253319,30.3034),(-93.262705,30.2992),(-93.266749,30.2869),(-93.255196,30.2855),(-93.244076,30.2921),(-93.241044,30.2829),(-93.222848,30.2673),(-93.210573,30.2623),(-93.241044,30.2592),(-93.249419,30.2525),(-93.251874,30.2343),(-93.269637,30.2268),(-93.277580,30.2278),(-93.284800,30.2154),(-93.296642,30.2145),(-93.330578,30.1730),(-93.345662,30.1427),(-93.692629,30.0621),(-93.707080,30.0924),(-93.680670,30.1081),(-93.707582,30.1321),(-93.705280,30.1242),(-93.729398,30.1146),(-93.725954,30.1017),(-93.728315,30.0937),(-93.732939,30.0951),(-93.738547,30.0859),(-93.729889,30.0768),(-93.731660,30.0723),(-93.724970,30.0642),(-93.725954,30.0539),(-93.740029,30.0630),(-93.745414,30.0672),(-93.744548,30.0692),(-93.738934,30.0752),(-93.745553,30.0751),(-93.757909,30.0665),(-93.751400,30.0624),(-93.734521,30.0467),(-93.749855,30.0302),(-93.780745,30.0396),(-93.806340,30.0488),(-93.808436,30.0428),(-93.825756,30.0512),(-93.842525,30.0815),(-93.849034,30.0813),(-93.850026,30.0736),(-93.828955,30.0453),(-93.801154,30.0345),(-93.766293,30.0206),(-93.831603,29.9898),(-93.874187,30.0292),(-93.887756,30.0272),(-93.877386,30.0160),(-93.886496,29.9806),(-93.967295,30.0187),(-94.012346,30.0471),(-94.044665,30.0832),(-94.095593,30.0946),(-94.106366,30.0815),(-94.082861,30.0734),(-94.066701,30.0700),(-94.044996,30.0556),(-94.031561,30.0205),(-93.979337,30.0053),(-93.958751,29.9965),(-93.934914,29.9871),(-93.913461,29.9646),(-93.898509,29.9730),(-93.879656,29.9687),(-93.868172,29.9655),(-93.885074,29.9310),(-93.939031,29.8799),(-93.954416,29.8530),(-94.003173,29.8808),(-94.029394,29.8902),(-94.053447,29.9004),(-94.071216,29.8844),(-94.054685,29.8716),(-94.034289,29.8840),(-93.996411,29.8736),(-93.987022,29.8278),(-93.963065,29.7753),(-94.022958,29.7590),(-94.020044,29.7334),(-93.925186,29.7497),(-93.880509,29.7092),(-93.990907,29.7106),(-94.325662,29.7387),(-94.322749,29.7146),(-94.397795,29.6385),(-94.406258,29.5802),(-94.454215,29.5428),(-94.532497,29.6391),(-94.588917,29.5894),(-94.671431,29.6041),(-94.673546,29.7322),(-94.634053,29.8050),(-94.696114,29.8454),(-94.722914,29.8081),(-94.784710,29.8015),(-94.810917,29.8224),(-94.839898,29.7831),(-94.840206,29.7520),(-94.886453,29.7159),(-94.899094,29.7025),(-94.882445,29.6832),(-94.904027,29.6717),(-94.933008,29.7009),(-94.925300,29.7301),(-94.945340,29.7362),(-94.957056,29.7137),(-94.980488,29.7169),(-94.970930,29.7392),(-94.983571,29.7448),(-94.993745,29.7317),(-95.008544,29.7303),(-95.052633,29.8125),(-95.050475,29.8395),(-95.079764,29.8395),(-95.089322,29.8983),(-95.103532,29.8881),(-95.102368,29.8573),(-95.108243,29.8425),(-95.103194,29.8343),(-95.094599,29.8198),(-95.094736,29.8152),(-95.098010,29.8112),(-95.097874,29.8045),(-95.087504,29.7975),(-95.072497,29.7954),(-95.086413,29.7908),(-95.094190,29.7890),(-95.103058,29.7834),(-95.104150,29.7779),(-95.104013,29.7673),(-95.101421,29.7595),(-95.113973,29.7471),(-95.128708,29.7404),(-95.148491,29.7421),(-95.172503,29.7506),(-95.205111,29.7446),(-95.212342,29.7332),(-95.226668,29.7279),(-95.242221,29.7262),(-95.259685,29.7300),(-95.268826,29.7326),(-95.276057,29.7398),(-95.291883,29.7541),(-95.295840,29.7464),(-95.285198,29.7424),(-95.278768,29.7236),(-95.257195,29.7216),(-95.240758,29.7144),(-95.230143,29.7213),(-95.213364,29.7230),(-95.202064,29.7326),(-95.190764,29.7406),(-95.171245,29.7412),(-95.155493,29.7320),(-95.142824,29.7308),(-95.121935,29.7317),(-95.104814,29.7364),(-95.080159,29.7546),(-95.061668,29.7293),(-95.064407,29.7162),(-95.040095,29.6963),(-95.017116,29.6853),(-95.009089,29.6774),(-94.988747,29.6804),(-95.001917,29.6734),(-95.017316,29.6519),(-95.018531,29.6258),(-95.005159,29.6170),(-94.991989,29.6008),(-95.004145,29.5895),(-95.016708,29.5663),(-95.027856,29.5628),(-95.039325,29.5618),(-95.047488,29.5660),(-95.044670,29.5721),(-95.043601,29.5800),(-95.037769,29.5843),(-95.037381,29.5920),(-95.040102,29.5969),(-95.043892,29.5916),(-95.051694,29.5840),(-95.051793,29.5788),(-95.057892,29.5716),(-95.068123,29.5714),(-95.083077,29.5558),(-95.088783,29.5445),(-95.086913,29.5375),(-95.092511,29.5380),(-95.102469,29.5514),(-95.104472,29.5510),(-95.106653,29.5458),(-95.098286,29.5399),(-95.103104,29.5361),(-95.114750,29.5189),(-95.116009,29.5068),(-95.104601,29.5089),(-95.098140,29.5133),(-95.087514,29.5220),(-95.089511,29.5256),(-95.089001,29.5275),(-95.084071,29.5307),(-95.069916,29.5303),(-95.069491,29.5328),(-95.072721,29.5384),(-95.070044,29.5407),(-95.064560,29.5416),(-95.059162,29.5450),(-95.061032,29.5478),(-95.049471,29.5483),(-95.047770,29.5412),(-95.040374,29.5405),(-95.039609,29.5440),(-95.034338,29.5410),(-95.027622,29.5371),(-95.027112,29.5415),(-95.033531,29.5452),(-95.031746,29.5498),(-95.030003,29.5450),(-95.024902,29.5447),(-95.022139,29.5468),(-95.021714,29.5438),(-95.012687,29.5315),(-95.000646,29.5175),(-94.961830,29.5040),(-94.915568,29.4929),(-94.921747,29.4817),(-94.936006,29.4879),(-94.939016,29.4827),(-94.962781,29.4701),(-94.971221,29.4688),(-94.975965,29.4643),(-94.980832,29.4589),(-94.989272,29.4610),(-95.013976,29.4687),(-95.031657,29.4612),(-95.049708,29.4600),(-95.051372,29.4563),(-95.060305,29.4590),(-95.070346,29.4538),(-95.078294,29.4439),(-95.084824,29.4442),(-95.100472,29.4339),(-95.097946,29.4300),(-95.090677,29.4353),(-95.083469,29.4372),(-95.069854,29.4456),(-95.061352,29.4536),(-95.051495,29.4509),(-95.045765,29.4543),(-95.030733,29.4561),(-95.015579,29.4544),(-95.015579,29.4612),(-95.007508,29.4589),(-94.978810,29.4485),(-94.967151,29.4610),(-94.956838,29.4349),(-94.980828,29.4210),(-94.980155,29.4122),(-94.958632,29.4155),(-94.944731,29.4054),(-94.934450,29.4148),(-94.936738,29.4192),(-94.929234,29.4213),(-94.925813,29.4165),(-94.922819,29.4212),(-94.920952,29.4159),(-94.912838,29.4150),(-94.893480,29.4199),(-94.892437,29.3933),(-94.895118,29.3805),(-94.894271,29.3671),(-94.915541,29.3689),(-94.921309,29.3649),(-94.941916,29.3465),(-94.948590,29.3414),(-94.961684,29.3460),(-94.976558,29.3486),(-94.993402,29.3531),(-95.005415,29.3549),(-95.008657,29.3516),(-95.012343,29.3515),(-95.013551,29.3493),(-95.007640,29.3480),(-95.002618,29.3520),(-94.978083,29.3433),(-94.972235,29.3363),(-94.961684,29.3326),(-94.949035,29.3321),(-94.949981,29.3298),(-94.953444,29.3037),(-95.002474,29.3120),(-95.013370,29.3436),(-95.043878,29.3493),(-95.024266,29.3237),(-95.009375,29.3056),(-95.022813,29.3012),(-95.074385,29.2565),(-95.168088,29.3329),(-95.160461,29.2977),(-95.139171,29.2786),(-95.094944,29.2338),(-95.157509,29.2117),(-95.233559,29.2786),(-95.251897,29.2508),(-95.228165,29.2334),(-95.226008,29.2178),(-95.197018,29.1874),(-95.289684,29.1116),(-95.306060,29.0807),(-95.271094,28.9899),(-95.287960,29.0047),(-95.331413,29.0277),(-95.381425,28.9774),(-95.396939,28.9344),(-95.392708,28.8914),(-95.418658,28.8648),(-95.487766,28.9102),(-95.542526,28.9612),(-95.564501,28.9906),(-95.578344,28.9832),(-95.570903,28.9733),(-95.560569,28.9397),(-95.524827,28.8942),(-95.495755,28.8944),(-95.456934,28.8691),(-95.466340,28.8617),(-95.696657,28.7690),(-95.763899,28.7880),(-95.949211,28.7212),(-96.012747,28.8432),(-95.997922,28.9910),(-96.014865,28.9934),(-96.024925,28.9489),(-96.059869,28.8872),(-96.034455,28.8464),(-96.011688,28.7704),(-95.992627,28.7012),(-96.086343,28.6626),(-96.174269,28.6787),(-96.179805,28.7266),(-96.160343,28.7528),(-96.144682,28.7591),(-96.144682,28.7846),(-96.137543,28.8054),(-96.140997,28.8104),(-96.138003,28.8159),(-96.135355,28.8335),(-96.140191,28.8238),(-96.146985,28.8182),(-96.150785,28.7921),(-96.164003,28.7813),(-96.161490,28.7721),(-96.152993,28.7718),(-96.152328,28.7661),(-96.193632,28.7656),(-96.188903,28.7986),(-96.195627,28.7979),(-96.199050,28.7739),(-96.205874,28.7716),(-96.198832,28.7607),(-96.186549,28.7609),(-96.181526,28.7532),(-96.193209,28.7481),(-96.204673,28.7338),(-96.207403,28.7283),(-96.204728,28.7165),(-96.212425,28.7095),(-96.211224,28.7016),(-96.228585,28.7028),(-96.237686,28.7120),(-96.245097,28.7164),(-96.279814,28.7517),(-96.295027,28.7465),(-96.295807,28.7328),(-96.273183,28.7113),(-96.299708,28.6966),(-96.316091,28.6641),(-96.337546,28.6589),(-96.378894,28.6849),(-96.380064,28.7376),(-96.413759,28.7780),(-96.386385,28.8053),(-96.399285,28.8271),(-96.407308,28.8227),(-96.399757,28.8055),(-96.422254,28.7747),(-96.401077,28.7551),(-96.422329,28.7375),(-96.428892,28.7762),(-96.459833,28.7701),(-96.451394,28.7140),(-96.432330,28.6992),(-96.431080,28.6723),(-96.434830,28.6421),(-96.463271,28.6926),(-96.483585,28.6783),(-96.479210,28.6534),(-96.553279,28.6759),(-96.551686,28.7430),(-96.538613,28.8788),(-96.600240,28.8649),(-96.589969,28.8199),(-96.596505,28.7364),(-96.651339,28.7467),(-96.695417,28.8118),(-96.731358,28.8344),(-96.739495,28.8237),(-96.721864,28.8133),(-96.709658,28.7767),(-96.675073,28.7425),(-96.668631,28.7143),(-96.689008,28.6980),(-96.623481,28.6435),(-96.630360,28.6221),(-96.629735,28.5938),(-96.653185,28.5996),(-96.672634,28.5804),(-96.623154,28.5565),(-96.578883,28.5739),(-96.519358,28.5255),(-96.568838,28.5098),(-96.574418,28.4817),(-96.525683,28.4523),(-96.475459,28.4954),(-96.428584,28.4568),(-96.633199,28.3623),(-96.732159,28.4352),(-96.819957,28.5654),(-96.882458,28.5360),(-96.833350,28.4509),(-96.867577,28.4395),(-96.863112,28.3973),(-96.805448,28.3596),(-96.825909,28.3115),(-96.812144,28.2509),(-96.886178,28.1883),(-96.911348,28.2873),(-96.982491,28.2072),(-96.968914,28.1949),(-96.992810,28.1468),(-96.993353,28.1801),(-97.003128,28.2186),(-97.123693,28.1605),(-97.206241,28.2141),(-97.232309,28.1818),(-97.207328,28.1736),(-97.173928,28.1310),(-97.232310,28.0731),(-97.215203,28.0654),(-97.164153,28.0330),(-97.200539,27.9671),(-97.182889,27.9626),(-97.145416,27.9906),(-97.136727,28.0170),(-97.110116,28.0259),(-97.099797,28.0486),(-97.111474,28.0584),(-97.079432,28.0769),(-97.069612,28.0838),(-97.055443,28.0891),(-97.045211,28.1007),(-97.043374,28.1154),(-97.025926,28.1093),(-97.035306,28.0923),(-97.038782,28.0759),(-97.036556,28.0669),(-97.034855,28.0551),(-97.036600,28.0516),(-97.038258,28.0548),(-97.040440,28.0540),(-97.039480,28.0524),(-97.036774,28.0492),(-97.039087,28.0505),(-97.040265,28.0489),(-97.035727,28.0463),(-97.033240,28.0503),(-97.034113,28.0517),(-97.033284,28.0504),(-97.035509,28.0466),(-97.044760,28.0359),(-97.048381,28.0284),(-97.047465,28.0270),(-97.050170,28.0265),(-97.051305,28.0211),(-97.056366,28.0110),(-97.058986,28.0027),(-97.066186,27.9936),(-97.075480,27.9968),(-97.080542,27.9901),(-97.080329,27.9878),(-97.103683,27.9580),(-97.161959,27.9909),(-97.164139,27.9943),(-97.167616,27.9935),(-97.164021,27.9908),(-97.110736,27.9584),(-97.134167,27.9181),(-97.150082,27.8958),(-97.181367,27.8529),(-97.195653,27.8491),(-97.216329,27.8731),(-97.239032,27.8857),(-97.322752,27.8944),(-97.382349,27.8873),(-97.434851,27.8765),(-97.483095,27.8944),(-97.521408,27.8861),(-97.528029,27.8610),(-97.516678,27.8376),(-97.479312,27.8188),(-97.455662,27.8217),(-97.418769,27.8167),(-97.406944,27.8229),(-97.398863,27.7845),(-97.382954,27.7473),(-97.345453,27.7194),(-97.331437,27.6805),(-97.343180,27.6755),(-97.356059,27.6423),(-97.371590,27.6480),(-97.363256,27.6272),(-97.335142,27.6207),(-97.433646,27.3502),(-97.450090,27.4041),(-97.524085,27.4833),(-97.596816,27.5197),(-97.625908,27.5108),(-97.564561,27.4563),(-97.528512,27.3929),(-97.605037,27.3350),(-97.655000,27.3761),(-97.778958,27.4720),(-97.799197,27.4114),(-97.773266,27.3923),(-97.717612,27.3631),(-97.701168,27.3159),(-97.754328,27.2937),(-97.745956,27.2404),(-97.670603,27.2540),(-97.605019,27.2106),(-97.468268,27.2255),(-97.514317,26.8277),(-97.446576,26.5357),(-97.380203,26.3681),(-97.432100,26.3551),(-97.464876,26.3378),(-97.485908,26.3451),(-97.525786,26.3307),(-97.545725,26.2993),(-97.591612,26.2873),(-97.593251,26.2342),(-97.579867,26.2349),(-97.576590,26.2653),(-97.570307,26.2805),(-97.556651,26.2739),(-97.543665,26.2871),(-97.521483,26.2947),(-97.521483,26.3199),(-97.488211,26.3315),(-97.476751,26.3212),(-97.444218,26.3205),(-97.419449,26.3394),(-97.352905,26.3477),(-97.324070,26.2463),(-97.304106,26.1640),(-97.288580,26.1275),(-97.303737,26.1215),(-97.297082,26.1026),(-97.267633,26.0845),(-97.244826,26.0803),(-97.226227,26.0740),(-97.212720,26.0497),(-97.304168,26.0137),(-97.309482,25.9990),(-97.406466,25.9538),(-97.403587,25.9444),(-97.394683,25.9490),(-97.342537,25.9652),(-97.296140,25.9899),(-97.180309,26.0538),(-97.167682,26.0077),(-97.166062,25.9604),(-97.158403,25.9617),(-97.160612,25.9506),(-97.155899,25.9495),(-97.146177,25.9566)])
Gulf_withorigin= gpd.GeoDataFrame({'geometry': [Gulf_withorigin_poly]})
# Gulf_withorigin_5_pts_poly = 'POLYGON ((-84.84686264156812 25.68340767152216, -87.43248750342957 30.94498325396067, -95.80214228446721 30.0359530662285, -98.3915463041922 26.769804868308682, -95.4673270575147 24.286120603664912, -84.84686264156812 25.68340767152216))'
Gulf_withorigin_5_pts_poly=Polygon([(-84.84686264156812,25.68340767152216),(-87.43248750342957,30.94498325396067),(-95.80214228446721,30.0359530662285),(-98.3915463041922,26.769804868308682),(-95.4673270575147,24.286120603664912),(-84.84686264156812,25.68340767152216)])
Gulf_withorigin_5_pts = gpd.GeoDataFrame({'geometry': [Gulf_withorigin_5_pts_poly]})

# Gulf_withorigin_12_pts = 'POLYGON ((-84.84856896756858 25.695741617325595, -87.17916383911246 30.452324640365944, -88.01903186224251 30.722395749561144, -89.65301634304046 30.32547021079138, -91.3126759908759 30.519766505652726, -93.85736740713483 30.260313341731944, -95.53720639388428 29.804942268773132, -95.08332787262616 29.38059169927938, -96.76296250517821 28.84621275467852, -97.84005543011887 27.476183331394836, -97.59767084364768 26.08365735241611, -95.46761369500034 24.28889336151191, -84.84856896756858 25.695741617325595))'
Gulf_withorigin_12_pts_poly=Polygon([(-84.8485,25.6957),(-87.1791,30.4523),(-88.0190,30.7223),(-89.6530,30.3254),(-91.3126,30.5197),(-93.8573,30.2603),(-95.5372,29.8049),(-95.0833,29.3805),(-96.7629,28.8462),(-97.8400,27.4761),(-97.5976,26.0836),(-95.4676,24.2888),(-84.8485,25.6957)])
Gulf_withorigin_12_pts= gpd.GeoDataFrame({'geometry': [Gulf_withorigin_12_pts_poly]})
# Gulf_withorigin_5_pts_poly = Polygon([(-84.84, 25.683), (-87.432, 30.944), (-95.8021, 30.035), (-98.3915, 26.769), (-95.467, 24.2861), (-84.8468, 25.683)])


## BLOBS THAT ARE ALL ONL THE COAST
#MatamorosBlob = 'POLYGON ((-97.23259063290548 26.07422217345797, -97.46224771270303 25.981357685684756, -97.39499099647664 25.914980954220738, -97.2112653326389 26.0020005889711, -97.23259063290548 26.07422217345797))'
MatamorosBlob=Polygon([(-97.2325,26.0742),(-97.4622,25.9813),(-97.3949,25.9149),(-97.2112,26.0020),(-97.2325,26.0742)])
MatamorosBlob = gpd.GeoDataFrame({'geometry': [MatamorosBlob]})

#CorpusBlob = 'POLYGON ((-97.51243643990766 27.80734415507696, -97.06610923723578 27.814454684222753, -96.84485190814578 28.10532390552312, -97.11551245229052 28.206340137571445, -97.51243643990766 27.80734415507696))'
CorpusBlob=Polygon([(-97.5124,27.8073),(-97.0661,27.8144),(-96.8448,28.1053),(-97.1155,28.2063),(-97.5124,27.8073)])
CorpusBlob = gpd.GeoDataFrame({'geometry':  [CorpusBlob]})

#VictoriaBlob = 'POLYGON ((-96.701912550505 28.178631236638182, -95.94638771869015 28.632867256013014, -95.97959760140681 28.942839513027266, -97.02072742458935 28.696974546580876, -96.701912550505 28.178631236638182))'
VictoriaBlob=Polygon([(-96.7019,28.1786),(-95.9463,28.6328),(-95.9795,28.9428),(-97.0207,28.6969),(-96.7019,28.1786)])
VictoriaBlob = gpd.GeoDataFrame({'geometry':  [VictoriaBlob]})

#HoustonBlob = 'POLYGON ((-95.35485397465607 28.89855833648224, -94.47848108960676 29.528788046606437, -94.86483393997872 29.407594621491782, -95.00589153012031 29.705946983017625, -95.29196087069126 29.738394905287052, -95.35485397465607 28.89855833648224))'
HoustonBlob=Polygon([(-95.3548,28.8985),(-94.4784,29.5287),(-94.8648,29.4075),(-95.0058,29.7059),(-95.2919,29.7383),(-95.3548,28.8985)])
HoustonBlob = gpd.GeoDataFrame({'geometry':  [HoustonBlob]})

#BeaumontBlob = 'POLYGON ((-94.10766005768463 30.094093689961753, -93.88811616856206 29.68568208677084, -92.53944082091348 29.86221065351242, -92.55705906221644 30.29048558924947, -94.10766005768463 30.094093689961753))'
BeaumontBlob=Polygon([(-94.1076,30.0940),(-93.8881,29.6856),(-92.5394,29.8622),(-92.5570,30.2904),(-94.1076,30.0940)])
BeaumontBlob = gpd.GeoDataFrame({'geometry':  [BeaumontBlob]})

#BatonRougeBlob = 'POLYGON ((-92.33271785451468 29.528762368360262, -91.62619193702291 29.78080242924962, -90.80005894878435 29.26600214951118, -90.41398010381532 29.343972579964536, -90.2039056916767 29.101345637777854, -89.55789542487304 29.339732120031684, -89.35845601364461 28.822528889892396, -89.1828626189764 29.23763442617603, -90.01177365764275 30.016560076104994, -91.22141704313462 30.551944912295767, -92.05047396907985 30.10091399835997, -92.33271785451468 29.528762368360262))'
BatonRougeBlob=Polygon([(-92.3327,29.5287),(-91.6261,29.7808),(-90.8000,29.2660),(-90.4139,29.3439),(-90.2039,29.1013),(-89.5578,29.3397),(-89.3584,28.8225),(-89.1828,29.2376),(-90.0117,30.0165),(-91.2214,30.5519),(-92.0504,30.1009),(-92.3327,29.5287)])
BatonRougeBlob = gpd.GeoDataFrame({'geometry':  [BatonRougeBlob]})

#BiloxiBlob = 'POLYGON ((-89.4661329795991 30.629862240410617, -89.36926583798825 30.2959068398294, -88.92171425242977 30.39188884923523, -88.68927911028184 30.360215073042568, -88.18267607674552 30.313270776031203, -88.12645779649921 30.341752322958058, -88.02084695547728 30.71362730232316, -89.4661329795991 30.629862240410617))'
BiloxiBlob=Polygon([(-89.4661,30.6298),(-89.3692,30.2959),(-88.9217,30.3918),(-88.6892,30.3602),(-88.1826,30.3132),(-88.1264,30.3417),(-88.0208,30.7136),(-89.4661,30.6298)])
BiloxiBlob = gpd.GeoDataFrame({'geometry': [BiloxiBlob]})

#DauphinBlob = 'POLYGON ((-88.06949528537193 30.247854272830082, -88.11160563267207 30.272120106038997, -88.20713060601955 30.249138233006136, -88.06949528537193 30.247854272830082))'
DauphinBlob=Polygon([(-88.0694,30.2478),(-88.1116,30.2721),(-88.2071,30.2491),(-88.0694,30.2478)])
DauphinBlob = gpd.GeoDataFrame({'geometry': [DauphinBlob]})

#GulfShoresBlob = 'POLYGON ((-88.01775249408088 30.221039760088928, -87.45348238733818 30.289426706597297, -87.48686163065229 30.34739175437315, -87.81660634479616 30.251277525955516, -88.02659722413568 30.23634224646591, -88.01775249408088 30.221039760088928))'
GulfShoresBlob=Polygon([(-88.0177,30.2210),(-87.4534,30.2894),(-87.4868,30.3473),(-87.8166,30.2512),(-88.0265,30.2363),(-88.0177,30.2210)])
GulfShoresBlob = gpd.GeoDataFrame({'geometry': [GulfShoresBlob]})

#MobileBayBlob = 'POLYGON ((-87.92276412807635 30.628234432949455, -87.90496337178624 30.543323828294263, -87.9402732848558 30.482987097593096, -87.90930891379693 30.41059032136188, -87.90097393124881 30.412387368985193, -87.92389513325656 30.486396957061302, -87.89097195219112 30.54312348761198, -87.92276412807635 30.628234432949455))'
MobileBayBlob=Polygon([(-87.92276,30.62823),(-87.90496,30.54332),(-87.94027,30.48298),(-87.90930,30.41059),(-87.90097,30.41238),(-87.92389,30.48639),(-87.89097,30.54312),(-87.92276,30.62823)])
MobileBayBlob = gpd.GeoDataFrame({'geometry': [MobileBayBlob]})

#GulfFinal = 'POLYGON ((-97.15283712203016 26.057398833940695, -87.07225636601888 28.720463567621906, -87.43428462975952 30.199438340751698, -89.03551348610372 30.63252316668084, -89.17446931734656 29.59587715514104, -90.44628500598762 29.289613352917996, -91.02129611298942 28.974193448175598, -91.55313232458553 29.513587232776004, -92.25570562256033 29.510171495745993, -93.05077904662467 29.72442677494432, -94.35583829044235 29.555562348541386, -95.32138595134722 28.892752839838238, -96.33403420026004 28.484736748047965, -97.15077887698884 27.701646888240504, -97.37618818234952 27.049704150693984, -97.15283712203016 26.057398833940695))'
GulfFinal=Polygon([(-97.15283,26.0573),(-87.07225,28.7204),(-87.43428,30.1994),(-89.03551,30.6325),(-89.17446,29.5958),(-90.44628,29.2896),(-91.02129,28.9741),(-91.55313,29.5135),(-92.25570,29.5101),(-93.05077,29.7244),(-94.35583,29.5555),(-95.32138,28.8927),(-96.33403,28.4847),(-97.15077,27.7016),(-97.37618,27.0497),(-97.15283,26.0573)])
GulfFinal = gpd.GeoDataFrame({'geometry': [GulfFinal]})

# Final Eliminate All But Coast-- to be used only after everything else has been cut out
#AwayFromCoast = 'POLYGON ((-97.21788050250058 25.87796703357563, -97.4440230018005 27.10697568689764, -97.13567431668646 27.933790186935823, -96.32191958175895 28.561740374233082, -94.37582180124227 29.628332170466393, -93.02976040963375 29.82048682994622, -90.92076683294566 29.386509066221322, -87.4968265121526 30.363538092259276, -87.4286095244991 30.94455080154404, -98.01979244016857 32.149366044190955, -98.41405820798954 26.679672951060198, -97.21788050250058 25.87796703357563))'
AwayFromCoast=Polygon([(-97.21788,25.87796),(-97.44402,27.10697),(-97.13567,27.93379),(-96.32191,28.56174),(-94.37582,29.62833),(-93.02976,29.82048),(-90.92076,29.38650),(-87.49682,30.36353),(-87.42860,30.94455),(-98.01979,32.14936),(-98.41405,26.67967),(-97.21788,25.87796)])
AwayFromCoast = gpd.GeoDataFrame({'geometry': [AwayFromCoast]})

## Function that creates Coastline and Just_Gulf data frames for a single day of AIS data

In [ ]:
## FUNCTION BASED ON CLEAN VERSION 
def process_data(AIS_Data_gdf):
    # Record the start time
    start_time = time.time()
    print("Starting function")


    #########################################################################################################################
    # Step 1: Remove points outside of 5 point version of Gulf_withorigin
    AIS_Data_gdf_B = AIS_Data_gdf[AIS_Data_gdf.geometry.within(Gulf_withorigin_5_pts.geometry.iloc[0])]
    
    #########################################################################################################################
    # Step 2: take points that remain, remove the ones in Gulf_9_points and put those points into a new dataframe called Gulf
    # The next 3 lines worked
    # points_within_gulf = AIS_Data_gdf_B[AIS_Data_gdf_B.geometry.within(Gulf_9_points.geometry.iloc[0])]
    # Gulf_df = gpd.GeoDataFrame(geometry=points_within_gulf.geometry)
    # AIS_Data_gdf_B = AIS_Data_gdf_B[~AIS_Data_gdf_B.geometry.isin(points_within_gulf.geometry)]
    Gulf_df = AIS_Data_gdf_B[AIS_Data_gdf_B.geometry.within(Gulf_9_points.geometry.iloc[0])]
    AIS_Data_gdf_B = AIS_Data_gdf_B[~AIS_Data_gdf_B.geometry.within(Gulf_9_points.geometry.iloc[0])]

    #########################################################################################################################
    # Step 3: Remove Coastal Points that contain all of the ports put these into a new df called Coast_df
    Coast_df1 = AIS_Data_gdf_B[AIS_Data_gdf_B.geometry.within(MatamorosBlob.geometry.iloc[0])]
    AIS_Data_gdf_B = AIS_Data_gdf_B[~AIS_Data_gdf_B.geometry.within(MatamorosBlob.geometry.iloc[0])]
    
    Coast_df2 = AIS_Data_gdf_B[AIS_Data_gdf_B.geometry.within(CorpusBlob.geometry.iloc[0])]
    AIS_Data_gdf_B = AIS_Data_gdf_B[~AIS_Data_gdf_B.geometry.within(CorpusBlob.geometry.iloc[0])]
    
    Coast_df3 = AIS_Data_gdf_B[AIS_Data_gdf_B.geometry.within(VictoriaBlob.geometry.iloc[0])]
    AIS_Data_gdf_B = AIS_Data_gdf_B[~AIS_Data_gdf_B.geometry.within(VictoriaBlob.geometry.iloc[0])]
    
    Coast_df7 = AIS_Data_gdf_B[AIS_Data_gdf_B.geometry.within(DauphinBlob.geometry.iloc[0])]
    AIS_Data_gdf_B = AIS_Data_gdf_B[~AIS_Data_gdf_B.geometry.within(DauphinBlob.geometry.iloc[0])]
    
    Coast_df8 = AIS_Data_gdf_B[AIS_Data_gdf_B.geometry.within(GulfShoresBlob.geometry.iloc[0])]
    AIS_Data_gdf_B = AIS_Data_gdf_B[~AIS_Data_gdf_B.geometry.within(GulfShoresBlob.geometry.iloc[0])]
    
    Coast_df4 = AIS_Data_gdf_B[AIS_Data_gdf_B.geometry.within(HoustonBlob.geometry.iloc[0])]
    AIS_Data_gdf_B = AIS_Data_gdf_B[~AIS_Data_gdf_B.geometry.within(HoustonBlob.geometry.iloc[0])]
    
    Coast_df5 = AIS_Data_gdf_B[AIS_Data_gdf_B.geometry.within(BeaumontBlob.geometry.iloc[0])]
    AIS_Data_gdf_B = AIS_Data_gdf_B[~AIS_Data_gdf_B.geometry.within(BeaumontBlob.geometry.iloc[0])]
    
    Coast_df6 = AIS_Data_gdf_B[AIS_Data_gdf_B.geometry.within(BatonRougeBlob.geometry.iloc[0])]
    AIS_Data_gdf_B = AIS_Data_gdf_B[~AIS_Data_gdf_B.geometry.within(BatonRougeBlob.geometry.iloc[0])]
    
    Coast_df9 = AIS_Data_gdf_B[AIS_Data_gdf_B.geometry.within(MobileBayBlob.geometry.iloc[0])]
    AIS_Data_gdf_B = AIS_Data_gdf_B[~AIS_Data_gdf_B.geometry.within(MobileBayBlob.geometry.iloc[0])]
    
    # Concatenate the DataFrames vertically
    Coast_df = [Coast_df1, Coast_df2, Coast_df3, Coast_df4, Coast_df5, Coast_df6, Coast_df7, Coast_df8, Coast_df9]
    Coast_df = pd.concat(Coast_df, ignore_index=True)
    
    #########################################################################################################################
    # Step 4: Add remaining points in the water to the Gulf data frame
    Gulf2 = AIS_Data_gdf_B[AIS_Data_gdf_B.geometry.within(GulfFinal.geometry.iloc[0])]
    AIS_Data_gdf_B = AIS_Data_gdf_B[~AIS_Data_gdf_B.geometry.within(GulfFinal.geometry.iloc[0])]
    Gulf_df = pd.concat([Gulf_df, Gulf2], ignore_index=True)
    
    #########################################################################################################################
    # Step 5: Use final section to take everything out of AIS_Data_gdf_B that is away from the coast that has not already been put into Coast
    AIS_Data_gdf_B = AIS_Data_gdf_B[~AIS_Data_gdf_B.geometry.within(AwayFromCoast.geometry.iloc[0])]
    Coast_df = pd.concat([Coast_df, AIS_Data_gdf_B], ignore_index=True)
    Coast_df = pd.concat([AIS_Data_gdf_B, Coast_df], ignore_index=True)

    return Gulf_df, Coast_df

    # THIS IS READY TO RETURN Coast_df AND Gulf_df
    
    # # Record the end time
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Elapsed time: {elapsed_time} seconds")

## Consolidate Just Gulf CSV's into a single data frame and save to csv

In [ ]:
## CREATE CSV FILES FOR EACH VESSEL TYPE
Gulfdir =  r"/scratch/user/r-woodward/AIS Data/CSV Files/Gulf"         # r"D:\AIS Data\CSV Files\Gulf"
Coastdir =  r"/scratch/user/r-woodward/AIS Data/CSV Files/Coastline"   #r"D:\AIS Data\CSV Files\Coastline"
Coastline_file_path = os.path.join(Coastdir, date_str + "_Coastline.csv")


## Consolidate all AIS csv files for the region into a single 
The csv files were concatenated using Linux code as developed in 
.\LinuxCommandsForConcatenation.xlsx
Commands are in the files
LinuxCommandsConcatenateAIStoCoastlinecsv.txt
LinuxCommandsConcatenateAIStoGulfcsv.txt


## The next cells contain the code to scrape data from the MarineTraffic web site to break VT #37 into two groups: Pleasure Craft and Sailing Vessels. 

### This should be done on my personal comptuer AND requires manual cleaning aftewards

### Extract the MMSI's for VT = 37 to look them up from MarineTraffic

In [ ]:
import os
import pandas as pd

Gulfdir =  r"/scratch/user/r-woodward/AIS Data/CSV Files/Gulf"         # r"D:\AIS Data\CSV Files\Gulf"
Gulf_file_path = os.path.join(Gulfdir, "AIS_Gulf_2019.csv")
VT37_file_path = os.path.join(Gulfdir, "VT37MMSIs.csv")
print(Gulf_file_path)
Gulf_Data_df = pd.read_csv(Gulf_file_path)

unique_mmsi_values = Gulf_Data_df[Gulf_Data_df['VesselType'] == 37]['MMSI'].unique()
df = pd.DataFrame({'MMSI': unique_mmsi_values})
df.to_csv(VT37_file_path, index=False)

print(df.head(4))

In [ ]:
unique_mmsi_values = Gulf_Data_df[Gulf_Data_df['VesselType'] == 37]['MMSI'].unique()

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import csv
import pandas as pd

#-------------------------------------------------------------------
# Input file -- CSV file that has MMSI's that have not previously been classifies
csv_file_path = r"S:\Spectus\Data\AIS Data\MMSINumbersTHREEMMSIs.csv"                ## CURRENTLY SET UP FOR DEBUGGING 
# csv_file_path = r"S:\Spectus\Data\AIS Data\VT37MMSINumbers.csv"   # Complete list from previous cell. Should not be used without cleanig for previously identified MMSI's 

# Output file
output_csv_file_path = r"S:\Spectus\Data\AIS Data\MMSINumbersResultsTEST.csv"        ## CURRENTLY SET UP FOR DEBUGGING 

#-------------------------------------------------------------------
# Define the list to store the results
results = []

# Read MMSI numbers from the CSV file and process them
with open(csv_file_path, mode='r') as file:
    for line in file:
        mmsi_numbers = line.strip()
        if mmsi_numbers:  # Check if the line is not empty
            scrape_mmsi_data(mmsi_numbers)
            time.sleep(5)  # Pause for 5 seconds between requests

# Convert the results list to a DataFrame
results_df = pd.DataFrame(results, columns=["MMSI", "Vessel_Name", "Vessel_Type", "Call_Sign", "URL"])

# Save the results DataFrame to a CSV file
results_df.to_csv(output_csv_file_path, index=False)

print(f"Results saved to {output_csv_file_path}")


## Function for scraping

In [ ]:
# Define a function to scrape data from a single URL and process it
def scrape_mmsi_data(mmsi):
    url = f"https://www.marinetraffic.com/en/ais/details/ships/mmsi:{mmsi}"
    headers = {
        "User-Agent": "Your User-Agent Here",  # Replace with your User-Agent
    }

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")

        # Extract the title field
        title_tag = soup.find("title")
        if title_tag:
            title = title_tag.text.strip()
            # Process the title
            if title.startswith("Ship "):
                title = title[5:]  # Remove the "Ship " prefix
                # Split the title into Vessel_Name, Vessel_Type, and Call_Sign
                parts = title.split("(")
                if len(parts) >= 2:
                    Vessel_Name = parts[0].strip()
                    Vessel_Type = parts[1].split(")")[0].strip()
                    Call_Sign = parts[-1].split("Call Sign ")[-1].strip()
                else:
                    Vessel_Name = Vessel_Type = Call_Sign = ""
                # Create the URL
                url = f"https://www.marinetraffic.com/en/ais/details/ships/mmsi:{mmsi}"
            else:
                Vessel_Name = Vessel_Type = Call_Sign = ""
                url = ""
            results.append((mmsi, Vessel_Name, Vessel_Type, Call_Sign, url))
            print(f"Scraped data for MMSI {mmsi}")
        else:
            print(f"Title not found for MMSI {mmsi}")
    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch data for MMSI {mmsi}: {str(e)}")

## Split the full Gulf data set into one for each VesselType

In [ ]:
import os
import pandas as pd

Gulfdir =  r"/scratch/user/r-woodward/AIS Data/CSV Files/Gulf"         # r"D:\AIS Data\CSV Files\Gulf"
Gulf_file_path = os.path.join(Gulfdir, "AIS_Gulf_2019.csv")
Gulf_Data_df = pd.read_csv(Gulf_file_path)

# List of VesselType values to split the data
vessel_types_to_split = [30, 31, 37, 60, 70, 80, 90]

# Iterate through the VesselType values
for vessel_type in vessel_types_to_split:
    # Create a new DataFrame containing only rows with the current VesselType
    subset_df = Gulf_Data_df[Gulf_Data_df['VesselType'] == vessel_type]
    
    # Define the output file path for the current VesselType
    output_file_path = os.path.join(Gulfdir, f"AIS_Gulf_2019_VesselType_{vessel_type}.csv")
    
    # Save the subset DataFrame to a CSV file
    subset_df.to_csv(output_file_path, index=False)


## Split the full Coastline data set into one for each VesselType

In [ ]:
import os
import pandas as pd

Coastlinedir =  r"/scratch/user/r-woodward/AIS Data/CSV Files/Coastline"         # r"D:\AIS Data\CSV Files\Gulf"
Coastline_file_path = os.path.join(Coastlinedir, "AIS_Coastline_2019.csv")
Coastline_Data_df = pd.read_csv(Coastline_file_path)

# List of VesselType values to split the data
vessel_types_to_split = [30, 31, 37, 60, 70, 80, 90]

# Iterate through the VesselType values
for vessel_type in vessel_types_to_split:
    print(vessel_type)
    # Create a new DataFrame containing only rows with the current VesselType
    subset_df = Coastline_Data_df[Coastline_Data_df['VesselType'] == vessel_type]
    
    # Define the output file path for the current VesselType
    output_file_path = os.path.join(Coastlinedir, f"AIS_Coastline_2019_VesselType_{vessel_type}.csv")
    
    # Save the subset DataFrame to a CSV file
    subset_df.to_csv(output_file_path, index=False)


## Break VT 37 into 2 groups and exclude vessels that can't be classified based on Marine Traffic data

In [ ]:
##  GULF FILES
# This divides VT 37 into two sub classes, 371 = Pleasure Boat (i.e. rec fishing vessel) and 372 = Sailing Vessel
# The final breakdown was carried out manually for boats with ambiguous vessel types. 
#   See S:\Spectus\Data\AIS Data\MMSINumbersResults.2019.xlsx
import os
import pandas as pd

Gulfdir =  r"/scratch/user/r-woodward/AIS Data/CSV Files/Gulf"         # r"D:\AIS Data\CSV Files\Gulf"

# Define the output file path for the current VesselType
VT37_file_path = os.path.join(Gulfdir, f"AIS_Gulf_2019_VesselType_37.csv")
VT37_df = pd.read_csv(VT37_file_path)
VT37_breakdown_file_path = os.path.join(Gulfdir, f"MMSI_VT37_Breakdown.2019.csv")
VT37_breakdown_df = pd.read_csv(VT37_breakdown_file_path)

# Filter rows with VT2 values 371 and 372 in VT37_breakdown_df
mmsi_371 = VT37_breakdown_df[VT37_breakdown_df['VT2'] == 371]['MMSI']
mmsi_372 = VT37_breakdown_df[VT37_breakdown_df['VT2'] == 372]['MMSI']

# Create VT371_df by filtering rows with MMSI values in mmsi_371
VT371_df = VT37_df[VT37_df['MMSI'].isin(mmsi_371)]
VT371_file_path = os.path.join(Gulfdir, f"AIS_Gulf_2019_VesselType_371.csv")
VT371_df.to_csv(VT371_file_path, index=False)

# Create VT372_df by filtering rows with MMSI values in mmsi_372
VT372_df = VT37_df[VT37_df['MMSI'].isin(mmsi_372)]
VT372_file_path = os.path.join(Gulfdir, f"AIS_Gulf_2019_VesselType_372.csv")
VT372_df.to_csv(VT372_file_path, index=False)


In [ ]:
##  COASTLINE FILES
# This divides VT 37 into two sub classes, 371 = Pleasure Boat (i.e. rec fishing vessel) and 372 = Sailing Vessel
# The final breakdown was carried out manually for boats with ambiguous vessel types. 
#   See S:\Spectus\Data\AIS Data\MMSINumbersResults.2019.xlsx
import os
import pandas as pd

Gulfdir =  r"/scratch/user/r-woodward/AIS Data/CSV Files/Gulf"        
Coastlinedir =  r"/scratch/user/r-woodward/AIS Data/CSV Files/Coastline"       

# Define the output file path for the current VesselType
VT37_file_path = os.path.join(Coastlinedir, f"AIS_Coastline_2019_VesselType_37.csv")
VT37_df = pd.read_csv(VT37_file_path)
VT37_breakdown_file_path = os.path.join(Gulfdir, f"MMSI_VT37_Breakdown.2019.csv")
VT37_breakdown_df = pd.read_csv(VT37_breakdown_file_path)

# Filter rows with VT2 values 371 and 372 in VT37_breakdown_df
mmsi_371 = VT37_breakdown_df[VT37_breakdown_df['VT2'] == 371]['MMSI']
mmsi_372 = VT37_breakdown_df[VT37_breakdown_df['VT2'] == 372]['MMSI']

# Create VT371_df by filtering rows with MMSI values in mmsi_371
print(371)
VT371_df = VT37_df[VT37_df['MMSI'].isin(mmsi_371)]
VT371_file_path = os.path.join(Coastlinedir, f"AIS_Coastline_2019_VesselType_371.csv")
VT371_df.to_csv(VT371_file_path, index=False)

# Create VT372_df by filtering rows with MMSI values in mmsi_372
print(372)
VT372_df = VT37_df[VT37_df['MMSI'].isin(mmsi_372)]
VT372_file_path = os.path.join(Coastlinedir, f"AIS_Coastline_2019_VesselType_372.csv")
VT372_df.to_csv(VT372_file_path, index=False)


## Main Indicators Cell that creates the trip indicators

In [ ]:
# THIS CELL IS USED TO CALCULATE FEATURES FOR EACH VESSEL TYPE. IT WAS SET UP TO BE RUN FOR A SINGLE VESSEL CLASS AT A TIME
# SO THAT SEPARATE FILES COULD BE RUN SIMULTANEOUSLY

# Values that need to be adjusted depending on ron and computer system in which we're operating
###############################################################################################
# Folders for input and output
input_folder =  r"/scratch/user/r-woodward/AIS Data/CSV Files/Gulf"     
Gulfdir =  input_folder
output_folder = r'/scratch/user/r-woodward/AIS Data/Indicators'


# 30: Fishing, 31: Towing, 37: Pleasure Craft, 60: Passenger, all ships of this type, 70: Cargo, all ships of this type, 
# 80: Tanker, all ships of this type, 90: Other Type, all ships of this type
# 33: Dredging or underwater operations  (5 vessels -- For debugging only)
# 99: A single MMSI from VT 30 for debugging purposes

# VT Values set for running one at a time
# VT_values = [30]
VT_values = [31]  
# VT_values = [70] 
# VT_values = [60] 
# VT_values = [80] 
# VT_values = [90]  
# VT_values = [371, 372]  
# VT_values = [999]  # For Debugging only

###############################################################################################
import math
from datetime import datetime

# Create a dictionary to store calculated distances
memo = {}


# Suppress warning message that are sent repeatedly
pd.set_option('mode.chained_assignment', None)  # Suppress the warning

# Loop over VT values
for VT in VT_values:
    # Construct the full path to the CSV file
    input_file_path  = os.path.join(Gulfdir, f"AIS_Gulf_2019_VesselType_{VT}.csv")
    # input_file_path = os.path.join(input_folder, f'VT{VT}_AIS_2019_06.csv')
    output_file_path = os.path.join(output_folder, f'Indicators2019VT{VT}.csv')

    # Read the CSV file into a DataFrame
    print("Reading ", input_file_path)
    AISdf = pd.read_csv(input_file_path)
    # Columns in these files: ['MMSI', 'BaseDateTime', 'lat', 'lon', 'VesselName', 'VesselType','geometry']
    # columns_to_drop = ['sog', 'cog', 'heading', 'vesselname', 'imo', 'callsign', 'vesseltype', 'status', 'length', 'width', 'draft', 'cargo', 'transceiverclass']
    columns_to_drop=['VesselName','VesselType','geometry']
    AISdf = AISdf.drop(columns=columns_to_drop)
    AISdf.rename(columns={'MMSI': 'mmsi', 'BaseDateTime': 'basedatetime'}, inplace=True)
   
    df = AISdf
    unique_ID = df.mmsi.unique()
    
    ivalue = 0
    # Check if the output file exists and if it does, remove those mmsi's from the list of unique ID'd to process
    if os.path.exists(output_file_path):
        # Read the existing CSV file into a DataFrame
        index_df = pd.read_csv(output_file_path)
        # List of MMSI values from index_df
        mmsi_values_in_index_df = index_df['mmsi'].unique()
        
        # Remove values from unique_ID that are in index_df
        unique_ID = [mmsi for mmsi in unique_ID if mmsi not in mmsi_values_in_index_df]
        ivalue = len(mmsi_values_in_index_df)
        print("STARTING ivalue", ivalue)

    
    print("Preparing", output_file_path)
    ###########################################
    
    index = []
    
    
    for value in unique_ID:
        indexOneValue = []
        current_time = datetime.now()
        formatted_time = current_time.strftime("%H:%M:%S")
        print("Start", value, formatted_time)
        ivalue = ivalue +1
        rows = df[df['mmsi'] == value]

        rows['event_timestamp'] = rows['basedatetime'].apply(AISdate_to_epoch)
        rows = rows.sort_values(by=['mmsi', 'event_timestamp'], ascending=[False, True])
        rows.index = range(len(rows))

        first_timestamp_mmmsi = rows['event_timestamp'].iloc[0]
        last_timestamp_mmmsi = rows['event_timestamp'].iloc[0]
        
        #ALL QID ALL TRIP VARS
        rows['time_diffs_minutes_all'] = rows["event_timestamp"].diff()/60.0
        rows['time_diffs_minutes_all'].iloc[0] = 0.0
    
        #trip number and break
        hour_break = 10
        trip_break = 60*hour_break
        rows['new_trip'] = (rows['time_diffs_minutes_all']>trip_break)*1
        rows['trip_num'] = 1+rows['new_trip'].cumsum() 
        rows['break_duration'] = (rows['time_diffs_minutes_all']*rows['new_trip'])
    
        #all the date variables
        rows['Hour'] = rows['event_timestamp'].apply(epoch_to_hour_of_day)
        rows['DOW'] = rows['event_timestamp'].apply(epoch_to_DOW)
        rows['month'] = rows['event_timestamp'].apply(monthofyear)
        rows['season'] = rows['month'].apply(month_to_season)
        rows['day_index'] = rows['event_timestamp'].apply(epoch_to_days_since_1_1_2019)
        rows['date_first_obsereved']= rows['day_index'].min()  
        rows['date_last_obsereved']= rows['day_index'].max()
        rows['date_count']=  rows['day_index'].nunique()
    
        #all the date variables
        rows['Hour'] = rows['event_timestamp'].apply(epoch_to_hour_of_day)
        rows['DOW'] = rows['event_timestamp'].apply(epoch_to_DOW)
        rows['month'] = rows['event_timestamp'].apply(monthofyear)
        rows['season'] = rows['month'].apply(month_to_season)
        rows['day_index'] = rows['event_timestamp'].apply(epoch_to_days_since_1_1_2019)
        rows['date_first_obsereved']= rows['day_index'].min()  
        rows['date_last_obsereved']= rows['day_index'].max()
        rows['date_count']=  rows['day_index'].nunique()
    
    ################################################################################################                    
    ######################## 1CID All TRIPs Indicator index and dataframe ################### 
    ################################################################################################
        all_trips_ping_counts= rows.mmsi.count()
        
        max_trip_num = max(rows.trip_num)
        
        rows['day_since_last_trip'] = (rows['time_diffs_minutes_all']/(24*60))*rows['new_trip']
        day_since_last_trip = rows.day_since_last_trip
        
        average_day_since_last_trip = sum(rows['day_since_last_trip'])/max_trip_num
            
        trips_per_day= (max_trip_num/all_trips_ping_counts)
           
        max_gap_between_trips= rows.day_since_last_trip.max()
        
        min_gap_between_trips = rows.day_since_last_trip.min()
        
        rows['Weekend_trips'] = (rows['DOW'] >= 5)*1
        Weekend_trips = (rows['Weekend_trips']*rows['new_trip']).sum()
        
        Weekday_trips= max_trip_num - Weekend_trips #62
    
    ################################################################################################        
        ############IMPLEMENTING CODE FOR 1 CID AND 1 TRIP############   
    ################################################################################################        
        unique_trip = rows.trip_num.unique()
        # print("Vessel type:", VT, "MMSI: ", value, "Trips: ", max(unique_trip))

        for tripnum in unique_trip:
            rows2 = rows[rows['trip_num'] == tripnum]
            # print(VT, value, tripnum, "before cleaning", len(rows2))
            rows2 = rows2.sort_values(by=["mmsi", "event_timestamp"], ascending=[False, True])

    
            ########################  ERROR CHECK #############################################################
            # Error checking for pings that have unreasonable shifts (>2km/minute)
            # Calculate distances and speeds
            counter = 0
            while True:
                # Find the speed to either side of a ping. If both of these are very large, this almost certainly indicates erroneous GPS coordinates
                cleaning_cols = ['dist_fwd', 'ping_speed_fwd']
                rows2['dist_fwd'] = haversine(rows2['lat'], rows2['lon'], rows2['lat'].shift(1), rows2['lon'].shift(1))
                # rows2['dist_bkwd'] = haversine(rows2['lat'], rows2['lon'], rows2['lat'].shift(-1), rows2['lon'].shift(-1))
                rows2['ping_speed_fwd'] = 60*abs(rows2['dist_fwd'] / (0.000001+rows2['event_timestamp'] - rows2['event_timestamp'].shift(1)))
                rows2['ping_speed_fwd'].iloc[0] = 0.0
                
                # rows2['ping_speed_bkwd'] = 60*abs(rows2['dist_bkwd'] / (0.000001+rows2['event_timestamp'].shift(-1) - rows2['event_timestamp']))
                # rows2['min_ping_speed'] = rows2[['ping_speed_fwd', 'ping_speed_bkwd']].min(axis = 1)
    
                condition1 = rows2['ping_speed_fwd'] <= 2.0
                condition2 = rows2['ping_speed_fwd'].shift(1) > 2.0
                
                filteredin = rows2[condition1 | condition2]

                # filteredin = rows2.loc[rows2['min_ping_speed'] <= 2.0 ]
                # filteredout = rows2.loc[rows2['min_ping_speed'] > 2.0]
                # print("filtered: ", len(filteredin), "rows2: " , len(rows2))
                if len(filteredin) == len(rows2):
                    # print(VT, value, tripnum, "Break criteria satisfied", len(rows2), len(filteredin))
                    rows2.drop(columns=cleaning_cols, inplace=True)
                    break
                    
                # Update rows2 if filteredin is smaller than rows2
                # print(VT, value, tripnum, "Break criteia NOT satisfied", len(rows2), len(filteredin))
                rows2 = filteredin

            ########################## END OF GPS ERROR CHECK ########################################################
            # print(VT, value, tripnum, "after cleaning")

            
            rows2.index = range(len(rows2))
       
            rows2['time_diffs_minutes'] = rows2["event_timestamp"].diff()/60.0
            rows2.loc[0, 'time_diffs_minutes'] = 0
            rows2['cumulative_time_minutes'] = rows2['time_diffs_minutes'].cumsum()
            rows2['cumulative_time_minutes'].iloc[0] = 0.0
    
            #One trip duration
            Trip_Duration_t = rows2.cumulative_time_minutes.max()
            #One trip ping count
            Trip_ping_count_t = rows2.mmsi.count()            

            #################  First check for whether this trip should be included in the analysis  ###
            # print(VT, value, tripnum, "Before if statement, Trip_Duration_t=", Trip_Duration_t, " Trip_ping_count_t = ", Trip_ping_count_t)
            if Trip_Duration_t > 60 and Trip_ping_count_t >=20:
                # print(VT, value, tripnum, "Inside if statement, Trip_Duration_t=", Trip_Duration_t, " Trip_ping_count_t = ", Trip_ping_count_t)
                # create array of aligned position obs and subsequent position obs
                posArray = rows2[["lat", "lon"]].to_numpy()
                laggedPosArray = np.vstack((posArray[0, :], posArray[:-1,:]))  # repeat starting obs
                pairedObsArray = np.hstack((laggedPosArray, posArray))
                
                # calc distaince diffs
                distDiffsList = [haversine(*row) for row in pairedObsArray]
                rows2['distance_diffs'] = pd.Series(distDiffsList)
                
                # cumulative distance
                rows2['cumulative_distance'] = rows2['distance_diffs'].cumsum()
        
                # distance from origin
                startLat = posArray[0, 0]
                startLon = posArray[0, 1]
                dfoList = [haversine(startLat, startLon, pos[0], pos[1])
                           for pos in posArray]
                rows2['distance_from_origin'] = pd.Series(dfoList)
                
                endlat = posArray[-1, 0]
                endlon = posArray[-1, 1]

                rows2['distance_from_coast'] = rows2.apply(lambda row: distance_to_coast_lat_lng(round(row['lat'], 2), round(row['lon'], 2)), axis=1)
                # rows2['distance_from_coast'] = rows2.apply(lambda row: distance_to_coast_lat_lng(row['lat'], row['lon']), axis=1)
                first_distance_from_coast_t = rows2['distance_from_coast'].iloc[0]
                # first_distance_from_coast_t = distance_to_coast_lat_lng(startLat, startLon)
                last_distance_from_coast_t = rows2['distance_from_coast'].iloc[-1]
                # last_distance_from_coast_t= distance_to_coast_lat_lng(endlat,endlon)
                max_distance_from_coast_t= rows2.distance_from_coast.max()
        #-----------------------------------------------------------------------------------------------            
                #Speed calculation
                x = rows2.cumulative_time_minutes
                xx = rows2.cumulative_time_minutes
                xx = np.append(-5,xx)
                xx = np.append(xx,xx[-1]+5)
                y = rows2.cumulative_distance
                yy = rows2.cumulative_distance
                yy = np.append(yy[0],yy)
                yy = np.append(yy,yy[-1])
                f_totdist = interpolate.interp1d(xx, yy)
        
                y = rows2.distance_from_origin
                yy = rows2.distance_from_origin
                yy = np.append(yy[0],yy)
                yy = np.append(yy,yy[-1])
                f_dist_to_origin = interpolate.interp1d(xx, yy)
        
                # forwardspeed and backwardspeed depend on the Time_step
                Time_step = 5
                stop_speed=0.025            ### 0.1 = 3.2 knots = 3.7 m.p.h
                trawl_speed = 0.1         ### 0.2 = 6.5 knots = 7.5 m.p.h
        
                def forwardspeed(x):
                    result = (f_totdist(x+Time_step)-f_totdist(x))/Time_step
                    return result
        
                def backwardspeed(x):
                    result = (f_totdist(x)-f_totdist(x-Time_step))/Time_step
                    return result
        
                # Use the  forwardspeed and backwardspeed functions, which use the linear sline that maps from a time to a total distance traveled
                # to calculate the forward speed and backward speed variables
                x = rows2.cumulative_time_minutes
                rows2['backwardspeed']=backwardspeed(x)
                rows2['forwardspeed']=forwardspeed(x)
                rows2['minspeed']=rows2[['forwardspeed', 'backwardspeed']].min(axis = 1)
                rows2['maxspeed']=rows2[['forwardspeed', 'backwardspeed']].max(axis = 1)
        
        #-----------------------------------------------------------------------------------------------
                # stops variables
                rows2['stop'] = 1*(rows2['minspeed'] <= stop_speed)
        
                rows2['first_ping_in_stop'] = rows2["stop"].diff()
                rows2['first_ping_in_stop'] =rows2['first_ping_in_stop'].replace(-1,0)
                rows2.at[0,'first_ping_in_stop']=rows2.at[0,"stop"]
        
                rows2['last_ping_in_stop'] = rows2["stop"].diff(periods=-1)
                rows2['last_ping_in_stop'] =rows2['last_ping_in_stop'].replace(-1,0)
                rows2.last_ping_in_stop.iloc[-1]=0
        
                rows2['time_in_stop'] =rows2['time_diffs_minutes']*rows2['stop'] #149
                rows2['weighted_dist_from_origin_in_stop'] =   rows2['time_in_stop']*rows2['distance_from_origin']   ### NEW - Delete comment after this has been checked
                
        
        #-----------------------------------------------------------------------------------------------
                # trawl variables
                rows2['ConForward']=1*((rows2['forwardspeed']-stop_speed)*(trawl_speed-rows2['forwardspeed'])>0)
                rows2['ConBackward']=1*((rows2['backwardspeed']-stop_speed)*(trawl_speed-rows2['backwardspeed'])>0)
                rows2['trawl']=rows2[['ConForward', 'ConBackward']].max(axis = 1)
                rows2 = rows2.drop(['ConForward','ConBackward'], axis =1)
        
                rows2['first_ping_in_trawl'] = rows2["trawl"].diff()
                rows2['first_ping_in_trawl'] =rows2['first_ping_in_trawl'].replace(-1,0)
                rows2.at[0,'first_ping_in_trawl']=rows2.at[0,"trawl"]
        
                rows2['last_ping_in_trawl'] = rows2["trawl"].diff(periods=-1)
                rows2['last_ping_in_trawl'] =rows2['last_ping_in_trawl'].replace(-1,0)
                rows2.last_ping_in_trawl.iloc[-1]=0
        
                rows2['time_in_trawl'] =rows2['time_diffs_minutes']*rows2['trawl']
                rows2['distance_in_trawl'] =rows2['distance_diffs']*rows2['trawl']
        
        #-----------------------------------------------------------------------------------------------
                # moving variables
                rows2['moving'] = 1*(rows2['maxspeed'] >= trawl_speed)
        
                rows2['first_ping_in_moving'] = rows2["moving"].diff()
                rows2['first_ping_in_moving'] =rows2['first_ping_in_moving'].replace(-1,0)
                rows2.at[0,'first_ping_in_moving']=rows2.at[0,"moving"]
        
                rows2['last_ping_in_moving'] = rows2["moving"].diff(periods=-1)
                rows2['last_ping_in_moving'] =rows2['last_ping_in_moving'].replace(-1,0)
                rows2.last_ping_in_moving.iloc[-1]=0
        
                rows2['time_moving'] =rows2['time_diffs_minutes']*rows2['moving']
                rows2['distance_moving'] =rows2['distance_diffs']*rows2['moving']
        
                rows2['trawl_or_moving']= 1*((rows2['moving'] ==1) | (rows2['trawl'] ==1))
                rows2['time_moving_or_trawl'] =rows2['time_diffs_minutes']*rows2['trawl_or_moving']
        
        #-----------------------------------------------------------------------------------------------
                # Create an empty list to store the new values
                # Loop to calculate cum_time_in_stop
                duration_of_stop = []
                old_value = 0
                new_value=0
                for row in rows2['time_in_stop']:
                    # Perform the desired operation on each row and append the result to the new_variable list
                    new_value = (row>0)*((row)+old_value)  
                    duration_of_stop.append(new_value)
                    old_value = new_value
        
                rows2['cum_time_in_stop'] =duration_of_stop
                rows2['cum_time_in_stop'] =rows2['cum_time_in_stop'] *rows2['last_ping_in_stop'] 
        
        #-----------------------------------------------------------------------------------------------
                # Loop to calculate cum_time_in_trawl
                duration_of_trawl = []
                old_value = 0
                new_value=0
                for row in rows2['time_in_trawl']:
                    # Perform the desired operation on each row and append the result to the new_variable list
                    new_value = (row>0)*((row)+old_value)   
                    duration_of_trawl.append(new_value)
                    old_value = new_value
        
                rows2['cum_time_in_trawl'] =duration_of_trawl
                rows2['cum_time_in_trawl'] =rows2['cum_time_in_trawl']*rows2['last_ping_in_trawl'] 
        
        #-----------------------------------------------------------------------------------------------            
                # Loop to calculate cum_time_in_move
                duration_of_move = []
                old_value = 0
                new_value=0
                for row in rows2['time_moving']:
                    # Perform the desired operation on each row and append the result to the new_variable list
                    new_value = (row>0)*((row)+old_value)   
                    duration_of_move.append(new_value)
                    old_value = new_value
                    
                rows2['cum_time_in_move'] =duration_of_move
                rows2['cum_time_in_move'] =rows2['cum_time_in_move'] *rows2['last_ping_in_moving'] 
        
        #----------------------------------------------------
        
                # Loop to calculate cum_dist_in_move
                distance_of_move = []
                old_value = 0
                new_value=0
                for row in rows2['distance_moving']:
                    # Perform the desired operation on each row and append the result to the new_variable list
                    new_value = (row>0)*((row)+old_value)   
                    distance_of_move.append(new_value)
                    old_value = new_value
        
                rows2['cum_dist_in_move'] = distance_of_move
                rows2['cum_dist_in_move'] = rows2['cum_dist_in_move'] *rows2['last_ping_in_moving']        
        
        ################################################################################################                    
                ######################## 1CID 1 TRIP Indicator index and DF ################### 
        ################################################################################################
                #Indicator variables            
                # Start and end times
                timestamp_start_t =rows2.event_timestamp.min() 
                timestamp_end_t =rows2.event_timestamp.max() 

                # One trip break duration
                break_duration_t = rows2.break_duration.max()
                # print(break_duration_t)
        
    
                #One trip total distance 
                Total_distance_traveled_t = rows2.cumulative_distance.max()
        
                #One trip distance between first and last ping in trip
                Distance_from_origin_t = rows2.distance_from_origin.iloc[-1]
        
                #One trip max distance from origin
                Max_distance_traveled_origin_t = rows2.distance_from_origin.max()
        
                #One trip pings per minute
                Pings_per_minute_t = Trip_ping_count_t/(Trip_Duration_t+.00000001) # by larger denominator we are generating wrong Pings_per_minute, but because we are droping trip duration less than 30 min.            
                
                #One trip greatest_time_step
                greatest_time_step_t=rows2.time_diffs_minutes.max()
                
                #One trip greatest_dist_step
                greatest_dist_step_t=rows2.distance_diffs.max()            
        
                #One trip contains weekend-  {5:'Sat',6:'Sun'}
                rows2['Weekend_trip_t']= (rows2['DOW'] >= 5)*1
                Weekend_trip_t = rows2.Weekend_trip_t.max() ## max(0,1)=1
                 
                Month_t = rows2['month'].max()    
                Season_t = rows2['season'].max()
        
                Start_day_num_t = rows2['day_index'].min() 
                End_day_num_t = rows2['day_index'].max()  
                Multi_day_t = End_day_num_t - Start_day_num_t
                
                
                #One trip time_in_stop
                time_stopped_t=rows2['time_in_stop'].sum()
            
                # One trip percentage of time stop
                pct_time_stopped= time_stopped_t/(Trip_Duration_t+.00000001) 
        
                #One trip number_of_stops
                number_of_stops_t = rows2['first_ping_in_stop'].sum()  # a np.float64
                
                #One trip longest_stop
                longest_stop_t = rows2['cum_time_in_stop'].max()
                # assert type(longest_stop_t) == np.float64
                
                #One trip shortest_stop
                rows2['shortest_stop_t']= rows2['cum_time_in_stop'].replace(0, 9999)
                shortest_stop_t= rows2['shortest_stop_t'].min()
          
                # One Trip weighted distance from origin in stops                      #### NOT CHECKED -- Delete this comment after it's confirmed
                dist_from_origin_during_stops = rows2['weighted_dist_from_origin_in_stop'].sum()/(time_stopped_t+0.000001)
            
                #One trip time_in_trawl
                time_trawling_t=rows2['time_in_trawl'].sum()
                
                # One trip percentage of time trawl
                pct_time_trawling= time_trawling_t/(Trip_Duration_t+.00000001)
        
                #One trip number_of_trawl
                number_of_trawl_t=rows2['first_ping_in_trawl'].sum()
        
                #One trip longest_trawl
                longest_trawl_t = rows2['cum_time_in_trawl'].max()
                
                #One trip shortest_trawl
                rows2['shortest_trawl_t']= rows2['cum_time_in_trawl'].replace(0, 9999)
                shortest_trawl_t= rows2['shortest_trawl_t'].min()
               
                #One trip distance_trawling
                distance_trawling_t=rows2['distance_in_trawl'].sum()            
                
                #One trip time_moving
                time_moving_t=rows2['time_moving'].sum()
                
                # One trip percentage of time moving
                pct_time_moving= time_moving_t/(Trip_Duration_t+.00000001) 
        
                #One trip number_of_moves
                number_of_moves_t=rows2['first_ping_in_moving'].sum()
        
                #One trip longest_move
                longest_move_t=rows2['cum_time_in_move'].max()
                
                #One trip shortest_move
                rows2['shortest_move_t']=rows2['cum_time_in_move'].replace(0, 9999)
                shortest_move_t = rows2['shortest_move_t'].min()
               
                #One trip distance_moving
                distance_moving_t=rows2['distance_moving'].sum()
                
                #One trip pings moving 
                Trip_ping_moving_t = rows2.moving.sum()
                
                #One trip pings moving per time moving 
                Trip_ping_moving_per_time_t = Trip_ping_moving_t/(time_moving_t +.00000001) 
                
                #One trip pings stop 
                Trip_ping_stop_t = rows2.stop.sum()
                
                #One trip pings stop per time stop 
                Trip_ping_stop_per_time_t = Trip_ping_stop_t/(time_stopped_t +.00000001) 
                
                time_moving_or_trawl = rows2['time_moving_or_trawl'].sum()           
                Trip_pings_mov_traw_per_time_t = rows2['trawl_or_moving'].sum()/(time_moving_or_trawl+.00000001)
                # if Trip_pings_mov_traw_per_time_t<.2:  # at least one ping every 5 min. on average when moving
                #     break
                # ------------- Loop to calculate euclidean distances between each move            
                # Efficiency of Movement
                # Find total distance traveled while moving relative to the Euclidean distance during moves.
                straightline=0
                startlon=rows2.at[0,'lon']
                startlat=rows2.at[0,'lat']
                endlon=rows2.at[0,'lon']
                endlat=rows2.at[0,'lat']
                i=0
                for i in range(0, len(rows2)):
                    if (rows2.at[i,'first_ping_in_moving']>0):
                        startlat=rows2.at[i,'lat']
                        startlon=rows2.at[i,'lon']
                    if (rows2.at[i,'last_ping_in_moving']==1):
                        endlat=rows2.at[i,'lat']
                        endlon=rows2.at[i,'lon']
                        straightline= straightline + haversine(startlat,startlon,endlat,endlon)        
                    else:
                        straightline= straightline
                # ---------------- End of loop
                # move_efficiency
                total_moving = rows2['distance_moving'].sum()
                move_efficiency_t= straightline/(total_moving+.0000001)
                
                # move_speed
                total_time_moving = rows2['time_moving'].sum()
                move_speed_t=total_moving/(total_time_moving+.0000001)
                maxspeed_t = rows2['maxspeed'].max()
    
        
        #             # ------------ Loop to generate CDF of speeds ------------------------------------------
                pdf_EQ_0_km_per_min_t=0
                pdf_GT_0_LE_0_010_km_per_min_t=0
                pdf_GT_0_010_LE_0_025_km_per_min_t=0
                pdf_GT_0_025_LE_0_050_km_per_min_t=0
                pdf_GT_0_05_LE_0_075_km_per_min_t=0
                pdf_GT_0_075_LE_0_1_km_per_min_t=0
                pdf_GT_0_1_LE_0_2_km_per_min_t=0
                pdf_GT_0_2_LE_0_3_km_per_min_t=0
                pdf_GT_0_3_LE_0_4_km_per_min_t=0
                pdf_GT_0_4_LE_0_5_km_per_min_t=0
                pdf_GT_0_5_LE_0_6_km_per_min_t=0
                pdf_GT_0_6_LE_0_7_km_per_min_t=0
                pdf_GT_0_7_LE_0_8_km_per_min_t=0
                pdf_GT_0_8_LE_0_9_km_per_min_t=0
                pdf_GT_0_9_LE_1_0_km_per_min_t=0
                pdf_GT_1_km_per_min_t=0
        
        
                triptime= Trip_Duration_t
        
                for i in range(0,100):
                    moment = (i/100)*triptime
                    speed=(backwardspeed(moment) + forwardspeed(moment))/2
                    if speed <=0:
                        pdf_EQ_0_km_per_min_t=0.01+pdf_EQ_0_km_per_min_t
                    if speed>0 and speed<=0.01:
                        pdf_GT_0_LE_0_010_km_per_min_t=0.01+pdf_GT_0_LE_0_010_km_per_min_t
                    if speed>0.01 and speed<=0.025:
                        pdf_GT_0_010_LE_0_025_km_per_min_t=0.01+pdf_GT_0_010_LE_0_025_km_per_min_t
                    if speed>0.025 and speed<=0.05:
                        pdf_GT_0_025_LE_0_050_km_per_min_t=0.01+pdf_GT_0_025_LE_0_050_km_per_min_t
                    if speed>0.05 and speed<=0.075:
                        pdf_GT_0_05_LE_0_075_km_per_min_t=0.01+pdf_GT_0_05_LE_0_075_km_per_min_t
                    if speed>0.075 and speed<=0.1:
                        pdf_GT_0_075_LE_0_1_km_per_min_t=0.01+pdf_GT_0_075_LE_0_1_km_per_min_t
                    if speed>0.1 and speed<=0.2:
                        pdf_GT_0_1_LE_0_2_km_per_min_t=0.01+pdf_GT_0_1_LE_0_2_km_per_min_t
                    if speed>0.2 and speed<=0.3:
                        pdf_GT_0_2_LE_0_3_km_per_min_t=0.01+pdf_GT_0_2_LE_0_3_km_per_min_t
                    if speed>0.3 and speed<=0.4:
                        pdf_GT_0_3_LE_0_4_km_per_min_t=0.01+pdf_GT_0_3_LE_0_4_km_per_min_t
                    if speed>0.4 and speed<=0.5:
                        pdf_GT_0_4_LE_0_5_km_per_min_t=0.01+pdf_GT_0_4_LE_0_5_km_per_min_t
                    if speed>0.5 and speed<=0.6:
                        pdf_GT_0_5_LE_0_6_km_per_min_t=0.01+pdf_GT_0_5_LE_0_6_km_per_min_t
                    if speed>0.6 and speed<=0.7:
                        pdf_GT_0_6_LE_0_7_km_per_min_t=0.01+pdf_GT_0_6_LE_0_7_km_per_min_t
                    if speed>0.7 and speed<=0.8:
                        pdf_GT_0_7_LE_0_8_km_per_min_t=0.01+pdf_GT_0_7_LE_0_8_km_per_min_t
                    if speed>0.8 and speed<=0.9:
                        pdf_GT_0_8_LE_0_9_km_per_min_t=0.01+pdf_GT_0_8_LE_0_9_km_per_min_t
                    if speed>0.9 and speed<=1:
                        pdf_GT_0_9_LE_1_0_km_per_min_t=0.01+pdf_GT_0_9_LE_1_0_km_per_min_t
                    if speed>1:
                        pdf_GT_1_km_per_min_t=0.01+pdf_GT_1_km_per_min_t
        
        ################################################################################################                    
                ######################## Indicator dataframes ################### 
        ################################################################################################
                # index.append((value, 
                indexOneValue.append((value, 
                          tripnum,
                          all_trips_ping_counts,
                          max_trip_num, 
                          trips_per_day, 
                          break_duration_t,
                          average_day_since_last_trip,
                          max_gap_between_trips,
                          min_gap_between_trips,
                          Weekend_trips,
                          Weekday_trips,
                          Trip_ping_count_t, 
                          Trip_Duration_t, 
                          Total_distance_traveled_t, 
                          Distance_from_origin_t,
                          startLat,
                          startLon,
                          endlat,
                          endlon,
                          first_distance_from_coast_t, 
                          last_distance_from_coast_t,
                          Max_distance_traveled_origin_t, 
                          Pings_per_minute_t, 
                          greatest_time_step_t, 
                          greatest_dist_step_t , 
                          Weekend_trip_t,
                          Month_t,
                          Season_t,
                          Start_day_num_t, 
                          End_day_num_t, 
                          Multi_day_t, 
                          time_stopped_t,
                          pct_time_stopped,
                          number_of_stops_t, 
                          longest_stop_t, 
                          shortest_stop_t, 
                          time_trawling_t,
                          pct_time_trawling,                                                    
                          number_of_trawl_t, 
                          longest_trawl_t, 
                          shortest_trawl_t,
                          distance_trawling_t, 
                          time_moving_t,
                          pct_time_moving,
                          number_of_moves_t,
                          longest_move_t,
                          shortest_move_t,
                          distance_moving_t,
                          dist_from_origin_during_stops,
                          Trip_pings_mov_traw_per_time_t, 
                          move_efficiency_t,
                          move_speed_t,
                          pdf_EQ_0_km_per_min_t,
                          pdf_GT_0_LE_0_010_km_per_min_t,
                          pdf_GT_0_010_LE_0_025_km_per_min_t,
                          pdf_GT_0_025_LE_0_050_km_per_min_t,
                          pdf_GT_0_05_LE_0_075_km_per_min_t,
                          pdf_GT_0_075_LE_0_1_km_per_min_t,
                          pdf_GT_0_1_LE_0_2_km_per_min_t,
                          pdf_GT_0_2_LE_0_3_km_per_min_t,
                          pdf_GT_0_3_LE_0_4_km_per_min_t,
                          pdf_GT_0_4_LE_0_5_km_per_min_t,
                          pdf_GT_0_5_LE_0_6_km_per_min_t,
                          pdf_GT_0_6_LE_0_7_km_per_min_t,
                          pdf_GT_0_7_LE_0_8_km_per_min_t,
                          pdf_GT_0_8_LE_0_9_km_per_min_t,
                          pdf_GT_0_9_LE_1_0_km_per_min_t,
                          pdf_GT_1_km_per_min_t,
                          maxspeed_t,
                          max_distance_from_coast_t,
                          timestamp_start_t, 
                          timestamp_end_t, 
                          ))
        
        #$$
            # index_df=pd.DataFrame(index, columns=['mmsi' , 
            indexOneValue_df=pd.DataFrame(indexOneValue, columns=['mmsi' , 
                                                  'Trip_number',
                                                  'all_trips_ping_counts',
                                                  'max_trip_num', 
                                                  'trips_per_day', 
                                                  'break_duration_t',
                                                  'average_day_since_last_trip',
                                                  'max_gap_between_trips',
                                                  'min_gap_between_trips',
                                                  'Weekend_trips',
                                                  'Weekday_trips',
                                                  'Trip_ping_count_t', 
                                                  'Trip_Duration_t', 
                                                  'Total_distance_traveled_t',
                                                  'Distance_from_origin_t',
                                                  'startLat',
                                                  'startLon',
                                                  'endlat',
                                                  'endlon',
                                                  'first_distance_from_coast_t', 
                                                  'last_distance_from_coast_t',
                                                  'Max_distance_traveled_origin_t', 
                                                  'Pings_per_minute_t',
                                                  'greatest_time_step_t',                   
                                                  'greatest_dist_step_t', 
                                                  'Weekend_trip_t',
                                                  'Month_t',
                                                  'Season_t',
                                                  'Start_day_num_t', 
                                                  'End_day_num_t', 
                                                  'Multi_day_t', 
                                                  'time_stopped_t',
                                                  'pct_time_stopped',
                                                  'number_of_stops_t', 
                                                  'longest_stop_t', 
                                                  'shortest_stop_t', 
                                                  'time_trawling_t',
                                                  'pct_time_trawling',
                                                  'number_of_trawl_t', 
                                                  'longest_trawl_t', 
                                                  'shortest_trawl_t',
                                                  'distance_trawling_t', 
                                                  'time_moving_t',
                                                  'pct_time_moving',
                                                  'number_of_moves_t',
                                                  'longest_move_t',
                                                  'shortest_move_t',
                                                  'distance_moving_t',
                                                  'dist_from_origin_during_stops',
                                                  'Trip_pings_mov_traw_per_time_t', 
                                                  'move_efficiency_t',
                                                  'move_speed_t',
                                                  'pdf_EQ_0_km_per_min_t',
                                                  'pdf_GT_0_LE_0_010_km_per_min_t',
                                                  'pdf_GT_0_010_LE_0_025_km_per_min_t',
                                                  'pdf_GT_0_025_LE_0_050_km_per_min_t',
                                                  'pdf_GT_0_05_LE_0_0.75_km_per_min_t',
                                                  'pdf_GT_0_07.5_LE_0_1_km_per_min_t',
                                                  'pdf_GT_0_1_LE_0_2_km_per_min_t',
                                                  'pdf_GT_0_2_LE_0_3_km_per_min_t',
                                                  'pdf_GT_0_3_LE_0_4_km_per_min_t',
                                                  'pdf_GT_0_4_LE_0_5_km_per_min_t',
                                                  'pdf_GT_0_5_LE_0_6_km_per_min_t',
                                                  'pdf_GT_0_6_LE_0_7_km_per_min_t',
                                                  'pdf_GT_0_7_LE_0_8_km_per_min_t',
                                                  'pdf_GT_0_8_LE_0_9_km_per_min_t',
                                                  'pdf_GT_0_9_LE_1_0_km_per_min_t',
                                                  'pdf_GT_1_km_per_min_t',
                                                  'maxspeed_t',
                                                  'max_distance_from_coast_t',
                                                  'timestamp_start_t', 
                                                  'timestamp_end_t', 
                                                 ])
        
    # for value in unique_ID:
        # for tripnum in unique_trip:
        indexOneValue_df['mmsi_index'] = pd.factorize(indexOneValue_df['mmsi'])[0]
        if ivalue==1:
            indexOneValue_df.to_csv(output_file_path, header=True, index=False)
        else:
            indexOneValue_df.to_csv(output_file_path, mode='a', header=False, index=False)
        file_size = os.path.getsize(output_file_path)
        current_time = datetime.now()
        formatted_time = current_time.strftime("%H:%M:%S")
        print("End ", value, formatted_time,  "trips: ",tripnum, f"File size: {file_size}", "Rows of df:", len(indexOneValue_df))
    print("Vessel Type ", VT, " is complete")

## Weather processing

### Loop to add average weather conditions to each point in each data frame based on the start date for the trip

In [ ]:
# Load data that were created in the code WeatherData.03.ipynb (not included here to keep code more readable)
weather_folder =  r"/scratch/user/r-woodward/AIS Data/Weather"     

Buoys_file_path  = os.path.join(weather_folder, f"Buoys.csv")
buoys_locations_df = pd.read_csv(Buoys_file_path)

Weather_file_path  = os.path.join(weather_folder, f"DailyWeatherData.csv")
daily_weather_averages = pd.read_csv(Weather_file_path)

indicator_folder = r'/scratch/user/r-woodward/AIS Data/Indicators'
# indicator_folder = r'D:\AIS Data\Indicators'

# 30: Fishing, 31: Towing, 37: Pleasure Craft, 60: Passenger, all ships of this type, 70: Cargo, all ships of this type, 
# 80: Tanker, all ships of this type, 90: Other Type, all ships of this type
# 33: Dredging or underwater operations  (5 vessels -- For debugging only)
# 99: A single MMSI from VT 30 for debugging purposes

# VT Values set for running one at a time
# VT_values = [30, 31, 70, 60, 80, 90, 371, 372]
# VT_values = [30, 31, 60, 70, 80, 371, 372]
VT_values = [90]

# VT_values = [999]  # For Debugging only

# Loop over VT values
for VT in VT_values:
    indicator_file_path = os.path.join(indicator_folder, f'Indicators2019VT{VT}.csv')
    indicator_file_path_new = os.path.join(indicator_folder, f'Indicators2019VT{VT}.B.csv')

    current_time = datetime.now()
    formatted_time = current_time.strftime("%H:%M:%S")
    
    if os.path.exists(indicator_file_path):
        print("Processing VT ", VT, "at ", formatted_time)
        Indicators_df = pd.read_csv(indicator_file_path)
    
        Indicators_df['Start_date'] = Indicators_df['timestamp_start_t'].apply(epoch_to_date)
        # Apply the function to each row in Indicators_df
        Indicators_df = Indicators_df.apply(apply_calculate_weighted_averages, axis=1)
        
        # # Save Indicators with weather data
        Indicators_df.to_csv(indicator_file_path_new, index=False)
    else:
        print("VT ", VT, "is missing. ", formatted_time)
        
    current_time = datetime.now()
    formatted_time = current_time.strftime("%H:%M:%S")
    print("Done with VT ", VT, "at ", formatted_time)



### Weather functions

In [ ]:
# Define a function to apply calculate_weighted_averages to each row
def apply_calculate_weighted_averages(row):
    date = row['Start_date']
    lat = row['startLat']
    lon = row['startLon']
    weighted_averages = calculate_weighted_averages(date, lat, lon, buoys_locations_df, daily_weather_averages)
    
    # Extract the values for the desired variables
    wspd = weighted_averages['WSPD']
    gst = weighted_averages['GST']
    wvht = weighted_averages['WVHT']
    atmp = weighted_averages['ATMP']
    
    # Assign the values to new columns in the DataFrame
    row['WSPD'] = wspd
    row['GST'] = gst
    row['WVHT'] = wvht
    row['ATMP'] = atmp
    
    return row

def calculate_weighted_averages(date, lat, lon, buoys_locations_df, daily_weather_averages):
    date_obj = pd.to_datetime(date).date()
    # Calculate distances to all stations
    buoys_locations_df1=buoys_locations_df
    buoys_locations_df1['distance'] = buoys_locations_df.apply(lambda row: haversine(lat, lon, row['lat'], row['lon']), axis=1)

    # Get the weather data for this date
    filtered_weather_data = daily_weather_averages[daily_weather_averages['date'] == date].copy()

    weighted_averages = {}
    variables_to_average = ['WSPD', 'GST', 'WVHT', 'ATMP']

    for var in variables_to_average:
        # First take out the case when no observations are available         
        if datetime(2021, 3, 10).date() <= date_obj <= datetime(2021, 3, 14).date() and var == 'WVHT':
        # if date_obj >= datetime.date(2021, 3, 10) and date_obj <= datetime.date(2021, 3, 14) and var == 'WVHT':
            # For the extended period when WVHT is not observed, use the average of all observations on 3/9 and 3/15
            weighted_averages[var] = 1.168570116
        else:
            # Find valid stations where the data exist
            valid_stations = filtered_weather_data[filtered_weather_data[var].notna()]
                
            # If only one valid station exists, use that station
            if len(valid_stations) == 1:            
                weighted_averages[var] = valid_stations[var].iloc[0]
                
            # If more than one station esists, use a linear interpolation of the nearest 2 stations
            elif len(valid_stations) > 1:
                nearest_stations = buoys_locations_df[buoys_locations_df['station_num'].isin(valid_stations['station_num'])].nsmallest(2, 'distance')
                # Calculate the weighted average for 'var' using the two nearest stations
                total_distance = nearest_stations['distance'].sum()
                nearest_stations['weight'] = 1 - (nearest_stations['distance'] / total_distance)
                merged_data = pd.merge(nearest_stations, valid_stations, on='station_num', how='inner')
                weighted_average = (merged_data['weight'] * merged_data[var]).sum()
                weighted_averages[var] = weighted_average
            

    return weighted_averages

## Identify the port types where the vessels spent time in the period before and after each trip

In [ ]:
current_time = datetime.now()
formatted_time = current_time.strftime("%H:%M:%S")
print("Start",  formatted_time)

# Record the starting time
Coastlinedir =  r"/scratch/user/r-woodward/AIS Data/CSV Files/Coastline"         # r"D:\AIS Data\CSV Files\Gulf"
Gulfdir =  r"/scratch/user/r-woodward/AIS Data/CSV Files/Gulf"         # r"D:\AIS Data\CSV Files\Gulf"
Indicatorsdir =  r"/scratch/user/r-woodward/AIS Data/Indicators"

# Load Ports list
PortsListWithH3_df = pd.read_csv(h3_file_path, encoding='latin-1')
empty_max_time_spent_H3_indices = create_emptyPortsDF()

# Define columns for max_time_spent_H3_indices_columns
max_time_spent_H3_indices_columns = [
        'mmsi',
        'Trip_number',
        'ConsolidatedPortType_1',
        'ConsolidatedPortType_2',
        'ConsolidatedPortType_3',
        'MarineTrafficPortType_1',
        'MarineTrafficPortType_2',
        'MarineTrafficPortType_3',
        'MarineTrafficPortType_4',
        'MarineTrafficPortType_5',
        'MarineTrafficPortType_6',
        'MarineTrafficPortType_7',
        'MarineTrafficPortType_8',
        'MarineTrafficPortType_9',
        'MarineTrafficPortType_10',
        'NearConsolidatedPortType_1',
        'NearConsolidatedPortType_2',
        'NearConsolidatedPortType_3'
    ]

# Define the list of file numbers you want to process
# Vessel_Types = [30, 31, 371, 372, 60, 70, 80, 90]
# Vessel_Types = [31, 371,60, 70, 80, 90]
Vessel_Types = [60, 70]

# Create an empty list to store the filtered DataFrames
# filtered_dfs = []

# # Convert the event_timestamp column to datetime
# coastline_df['event_timestamp'] = coastline_df['basedatetime'].apply(lambda x: AISdate_to_epoch(x))
# coastline_df['event_timestamp'] = pd.to_datetime(coastline_df['event_timestamp'], unit='s')


# Iterate through the list of file numbers
for VT in Vessel_Types:
    current_time = datetime.now()
    formatted_time = current_time.strftime("%H:%M:%S")
    print("Vessel type:", VT, "   starting at:", formatted_time)
    # File path for csv file that will store results for each mmsi-trip combination
    h3cells_file_path = os.path.join(Indicatorsdir, f'max_time_spent_H3_indices_All{VT}.csv')

    # Write the empty H3 indices to a csv file this will overwrite the one from the previous VT
    empty_max_time_spent_H3_indices.to_csv(h3cells_file_path, index=False)

    
    OK_to_process = True

    indicators_file_path = os.path.join(Indicatorsdir, f'Indicators2019VT{VT}.B.csv')
    if os.path.exists(indicators_file_path):
        indicators_df = pd.read_csv(indicators_file_path)
    else:
        print(indicators_file_path, " not found")
        OK_to_process = False
    
    Coastline_file_path = os.path.join(Coastlinedir, f'AIS_Coastline_2019_VesselType_{VT}.csv')
    if os.path.exists(Coastline_file_path):
        Coastline_df = pd.read_csv(Coastline_file_path)
        Coastline_df = Coastline_df.rename(columns={'MMSI': 'mmsi'})
    else:
        OK_to_process = False
        print(Coastline_file_path, " not found")

    # Initialize if max_time_spent_H3_indices_All for this Vessel Type
    max_time_spent_H3_indices_All = empty_max_time_spent_H3_indices  # pd.DataFrame()

    # Check if the file exists
    if OK_to_process:
        print("Start of processing")
        # Convert timestamp columns to datetime
        indicators_df['timestamp_start_t'] = pd.to_datetime(indicators_df['timestamp_start_t'], unit='s')
        indicators_df['timestamp_end_t'] = pd.to_datetime(indicators_df['timestamp_end_t'], unit='s')

        mmsi_list = indicators_df.mmsi.unique()
        # mmsi_list = [mmsi_list[33]]                      #  DEBUGGING TO JUST USE A SINGLE MMSI
        
        filtered_Coastline_df = Coastline_df[Coastline_df['mmsi'].isin(mmsi_list)]

#         # Iterate through each row in the indicators DataFrame and filter the coastline DataFrame
        immsi = -1
        for mmsi_num in mmsi_list:
            current_time = datetime.now()
            formatted_time = current_time.strftime("%H:%M:%S")
            print("mmsi #", immsi, " of ", len(mmsi_list), "at ", formatted_time)
            immsi =immsi+1
            Coastline2_df = filtered_Coastline_df[filtered_Coastline_df['mmsi'] == mmsi_num]
            
            indicators2_df = indicators_df[indicators_df['mmsi'] == mmsi_num]
            for index, row in indicators2_df.iterrows():
                # create filtered_rows where points before and afterthe trip will be placed
                filtered_coastline_rows = pd.DataFrame()

                # Set periods before and after the trip to evaluate
                nHours = 5
                start_time = row['timestamp_start_t'] - pd.Timedelta(hours=nHours)
                start_time = start_time.timestamp()
                start_of_trip = row['timestamp_start_t'] 
                start_of_trip = start_of_trip.timestamp()
                end_time = row['timestamp_end_t'] + pd.Timedelta(hours=nHours)
                end_time= end_time.timestamp()
                end_of_trip = row['timestamp_end_t'] 
                end_of_trip = end_of_trip.timestamp()
                
                # Now, you can filter the DataFrame
                Coastline2_df = Coastline2_df.copy()
                Coastline2_df.loc[:, 'event_timestamp'] = Coastline2_df['BaseDateTime'].apply(lambda x: AISdate_to_epoch(x))

                trip_number = row['Trip_number']  # Get the Trip_number
#             ###############################################################################
                ### First process the period before the trip
                # Filter the coastline DataFrame based on MMSI and time conditions
                filtered_df = Coastline2_df[(Coastline2_df['event_timestamp'] >= start_time) & (Coastline2_df['event_timestamp'] <= start_of_trip)]
                pingct = len(filtered_df)
                # # Create a copy of the filtered DataFrame and then add the 'Trip_number' column with the associated value
                filtered_df = filtered_df.copy()
                filtered_df['Trip_number'] = trip_number

                # sort and find the time differences   
                filtered_df = filtered_df.sort_values(by='event_timestamp')
                filtered_df['time_diff'] = 0
                filtered_df['time_diff'] = filtered_df['time_diff'].astype(int)
                filtered_df['time_diff'] = filtered_df['event_timestamp'].diff()
                # # Replace NaN values with 0
                filtered_df['time_diff'].fillna(0, inplace=True)

                # Append the filtered rows to the result
                filtered_coastline_rows = filtered_df
                # print("Before", len(filtered_coastline_rows))

                ###############################################################################
                ### Second process the period after the trip
                # Filter the coastline DataFrame based on MMSI and time conditions
                filtered_df = Coastline2_df[(Coastline2_df['event_timestamp'] >= end_of_trip) & (Coastline2_df['event_timestamp'] <= end_time)]
                pingct = pingct+len(filtered_df)
                # # Create a copy of the filtered DataFrame and then add the 'Trip_number' column with the associated value
                filtered_df = filtered_df.copy()
                filtered_df['Trip_number'] = trip_number

                # sort and find the time differences   
                filtered_df = filtered_df.sort_values(by='event_timestamp')
                filtered_df['time_diff'] = 0
                filtered_df['time_diff'] = filtered_df['time_diff'].astype(int)
                filtered_df['time_diff'] = filtered_df['event_timestamp'].diff()
                # # Replace NaN values with 0
                filtered_df['time_diff'].fillna(0, inplace=True)

                # Append the filtered rows to the result
                filtered_coastline_rows = pd.concat([filtered_coastline_rows, filtered_df], ignore_index=True)
                # print("pingct",pingct)

                # Initialize max_time_spent_H3_indices with an empty data frame
                # max_time_spent_H3_indices=empty_max_time_spent_H3_indices
                max_time_spent_H3_indices=pd.DataFrame(0, columns=max_time_spent_H3_indices_columns, index=range(1))
                max_time_spent_H3_indices['mmsi'] = mmsi_num
                max_time_spent_H3_indices['Trip_number']=trip_number

                # As long as the pingct > 0, process this coastline area, otherwise 
                if pingct > 0:
                    # find the h3 cell for each point
                    filtered_coastline_rows['h3cell'] = filtered_coastline_rows.apply(map_to_h3, axis=1)
    
                    # Find add up time_diff by h3cell and find the maximum value
                    grouped = filtered_coastline_rows.groupby('h3cell')['time_diff'].sum()
                    h3cell_with_max_sum = grouped.idxmax()
                    max_sum = grouped.max()
    
                    matching_ports_df = PortsListWithH3_df[PortsListWithH3_df['h3_cell'] == h3cell_with_max_sum]
                    # print(mmsi_num, "mmsi#", immsi, "    len(matching_ports_df)",len(matching_ports_df))

                    # Iterate through each unique 'MarineTrafficPortType_num' value in matching_ports_df
                    for port_type_num in matching_ports_df['MarineTrafficPortType_num'].unique():
                        # Construct the corresponding column name, e.g., 'MarineTrafficPortType_X'
                        column_name = f'MarineTrafficPortType_{port_type_num}'
                        max_time_spent_H3_indices.at[0, column_name] = 1

                    # Iterate through each unique 'MarineTrafficPortType_num' value in matching_ports_df
                    for port_type_num in matching_ports_df['ConsolidatedPortType_num'].unique():
                        # Construct the corresponding column name, e.g., 'MarineTrafficPortType_X'
                        column_name = f'ConsolidatedPortType_{port_type_num}'
                        max_time_spent_H3_indices.at[0, column_name] = 1

                    # Find nearby cells and see if any ports are within those cells
                    nearbycelllist = list(h3.k_ring(h3cell_with_max_sum, 1))
                    condition = (PortsListWithH3_df['h3_cell'].isin(nearbycelllist))
                    matching_ports_df = PortsListWithH3_df[condition]
                    for port_type_num in matching_ports_df['ConsolidatedPortType_num'].unique():
                        # Construct the corresponding column name, e.g., 'MarineTrafficPortType_X'
                        column_name = f'NearConsolidatedPortType_{port_type_num}'
                        max_time_spent_H3_indices.at[0, column_name] = 1

                # After each loop, append the row to the csv filte that stores all rows
                max_time_spent_H3_indices.to_csv(h3cells_file_path, index=False, mode='a', header=False)

    # After all mmsi's and trips are completed for a VT, read the full file and merge it with the Indicators file
    max_time_spent_H3_indices_All = pd.read_csv(h3cells_file_path, encoding='latin-1')

    # Add port indicators to Indicators DF and write to a new CSV File
    indicators_df = indicators_df.merge(max_time_spent_H3_indices_All, on=['mmsi', 'Trip_number'], how='left')

    # Save the resulting and now complete indicators file
    indicators_file_path = os.path.join(Indicatorsdir, f'Indicators2019VT{VT}.C.csv')
    indicators_df.to_csv(indicators_file_path, index=False)

# $$$$$$$$$  Time marker $$$$$$$$$$
current_time = datetime.now()
formatted_time = current_time.strftime("%H:%M:%S")
print("End time",  formatted_time)
# $$$$$$$$$  Time marker $$$$$$$$$$

# Random Forest Part

### Consololidate all complete indicators files with vt as new variable into a single CSV file for Random Forest

In [ ]:
Indicatorsdir =  r"/scratch/user/r-woodward/AIS Data/Indicators"
Vessel_Types = [30, 31, 371, 372, 60, 70, 80, 90]

# Initialize dfs where all the indicators will be placed temporarily
dfs = []

for VT in Vessel_Types:
    indicators_file_path = os.path.join(Indicatorsdir, f'Indicators2019VT{VT}.C.csv')
    df = pd.read_csv(indicators_file_path)
    df['vt'] = VT
    
    dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
indicators_file_path = os.path.join(Indicatorsdir, f'Indicators2019_All.C.csv')
AISIndicators_df = pd.concat(dfs, ignore_index=True)
AISIndicators_df.to_csv(indicators_file_path, index=False)

#### Summary stats for AIS data 

In [ ]:
import os
import pandas as pd

Indicatorsdir =  r"/scratch/user/r-woodward/AIS Data/Indicators"
indicators_file_path = os.path.join(Indicatorsdir, f'Indicators2019_All.C.csv')
# # Read df -- not needed if run sequentially
AISIndicators_df = pd.read_csv(indicators_file_path)

# Group by 'vt' and aggregate to get count of trips and number of unique 'mmsi'
summary_stats = AISIndicators_df.groupby('vt').agg(trips=('mmsi', 'count'), unique_mmsis=('mmsi', 'nunique'))

# Rename columns for clarity
summary_stats = summary_stats.rename(columns={'trips': 'Number of Trips', 'unique_mmsis': 'Number of Unique MMSIs'})

# Save summary_stats to a CSV file
summary_stats.to_csv('AIS_Summary_Stats.csv')

# Display the summary statistics
print(summary_stats)

In [ ]:
# Gulfdir =  r"/scratch/user/r-woodward/AIS Data/CSV Files/Gulf"         # r"D:\AIS Data\CSV Files\Gulf"
# Gulf_file_path = os.path.join(Gulfdir, "AIS_Gulf_2019.csv")
# AISIndicators_df = pd.read_csv(Gulf_file_path)

# Group by 'vt' and aggregate to get count of trips and number of unique 'mmsi'
# summary_stats_pings = AISIndicators_df.groupby('VesselType').agg(pings=('MMSI', 'count'), unique_mmsis=('MMSI', 'nunique'))

# Rename columns for clarity
summary_stats = summary_stats_pings.rename(columns={'pings': 'All Pings', 'unique_mmsis': 'MMSIs'})

# Save summary_stats to a CSV file
summary_stats_pings.to_csv('ALL_AIS_Summary_Stats.csv')

# Display the summary statistics
print(summary_stats_pings)

### Identify RF Classifier

In [ ]:
import datetime
import joblib

Indicatorsdir =  r"/scratch/user/r-woodward/AIS Data/Indicators"
indicators_file_path = os.path.join(Indicatorsdir, f'Indicators2019_All.C.csv')
# # Read df -- not needed if run sequentially
AISIndicators_df = pd.read_csv(indicators_file_path)

# Identify columns to be used in RF
X = AISIndicators_df[[
                        'trips_per_day', 
                        'break_duration_t',
                        'Weekday_trips', 
                        'Trip_Duration_t',
                        'Total_distance_traveled_t', 
                        'Distance_from_origin_t', 
                        'first_distance_from_coast_t',
                        'last_distance_from_coast_t', 
                        'Max_distance_traveled_origin_t',
                        'Weekend_trip_t', 
                        'time_stopped_t', 
                        'pct_time_stopped',
                        'number_of_stops_t', 
                        'longest_stop_t',
                        'shortest_stop_t',
                        'time_trawling_t',
                        'pct_time_trawling',
                        'number_of_trawl_t',
                        'longest_trawl_t', 
                        'shortest_trawl_t', 
                        'distance_trawling_t',
                        'time_moving_t', 
                        'pct_time_moving', 
                        'number_of_moves_t',
                        'longest_move_t', 
                        'shortest_move_t',
                        'distance_moving_t',
                        'dist_from_origin_during_stops', 
                        'Trip_pings_mov_traw_per_time_t',
                        'move_efficiency_t', 
                        'move_speed_t',
                        'maxspeed_t', 
                        'max_distance_from_coast_t',
                        'WSPD', 
                        'GST',
                        'WVHT', 
                        'ATMP'  #, 
                        # 'ConsolidatedPortType_1', 
                        # 'ConsolidatedPortType_2',
                        # 'ConsolidatedPortType_3', 
                        # 'MarineTrafficPortType_1',
                        # 'MarineTrafficPortType_2', 
                        # 'MarineTrafficPortType_3',
                        # 'MarineTrafficPortType_4', 
                        # 'MarineTrafficPortType_5',
                        # 'MarineTrafficPortType_6', 
                        # 'MarineTrafficPortType_7',
                        # 'MarineTrafficPortType_8', 
                        # 'MarineTrafficPortType_9',
                        # 'MarineTrafficPortType_10', 
                        # 'NearConsolidatedPortType_1',
                        # 'NearConsolidatedPortType_2', 
                        # 'NearConsolidatedPortType_3'
                        ]]

y = AISIndicators_df['vt']

identifiers = AISIndicators_df[['mmsi', 'Trip_number', 'vt']]

# Split the data into training and testing sets
#  NOTE THAT I HAVE MODIFIED THE TEST_SIZE TO EXPERIMENT -- 20% IS, ACCORDING TO CHAT GPT, THE DEFAULT TEST_SIZE
X_train, X_test, y_train, y_test, identifiers_train, identifiers_test = train_test_split(X, y, identifiers, test_size=0.2, random_state=49)

# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier on the training data
rf_classifier.fit(X_train, y_train)

###############  -- RF analysis is done --###############  

# Predict vessel types on the testing data
y_pred_test = rf_classifier.predict(X_test)

# Basic evaluation of model's accuracy on the test data
accuracy = accuracy_score(y_test, y_pred_test)
print(f"Accuracy: {accuracy:.4f}")

# Predict vessel types on the testing data
y_pred_train = rf_classifier.predict(X_train)
accuracy_train = accuracy_score(y_train, y_pred_train )
print(f"Accuracy in training: {accuracy_train:.4f}")

# Assuming you have trained rf_classifier and want to save it

# Save the trained Random Forest classifier to a file
import datetime

current_datetime = datetime.datetime.now()
date_string = current_datetime.strftime("%Y-%m-%d_%H-%M-%S")

classifier_file_path = os.path.join(Indicatorsdir, f'rf_model_{date_string}.pkl')
joblib.dump(rf_classifier, classifier_file_path)

### Create a single array with all the data, the predicted class and the probabilities of each class. This uses df's from the previous cell

In [ ]:
################################################################
# CREATE A DATA FRAME FOR THE TEST OBSERVATIONS FROM ABOVE
################################################################
# Create a combined data frame for all the test data
test_df = identifiers_test
# ### print
# print(1,test_df.shape)
test_df['group'] = "test"
test_df['vt_pred'] = y_pred_test
# ### print
# print(2,test_df.shape)

test_df.reset_index(drop=True, inplace=True)
predicted_proba_df = []
# predicted_proba_df.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)

predicted_probabilities = rf_classifier.predict_proba(X_test)
predicted_proba_df = pd.DataFrame(predicted_probabilities, columns=rf_classifier.classes_)
test_df = pd.concat([test_df, predicted_proba_df], axis=1)
test_df = pd.concat([test_df, X_test], axis=1)
# ### print
# print(3,test_df.shape)
# Export test_df to a CSV file
RF_Results_test = os.path.join(Indicatorsdir, f'Test_observations.csv')
test_df.to_csv(RF_Results_test, index=False)

################################################################
# CREATE A DATA FRAME FOR THE TRAINING OBSERVATIONS FROM ABOVE
################################################################
# Create a combined data frame for all the train data
train_df = identifiers_train
# ### print
# print(1,train_df.shape)
train_df['group'] = "train"
train_df['vt_pred'] = y_pred_train
# ### print
# print(2,train_df.shape)

train_df.reset_index(drop=True, inplace=True)
predicted_proba_df.reset_index(drop=True, inplace=True)
X_train.reset_index(drop=True, inplace=True)

predicted_probabilities = rf_classifier.predict_proba(X_train)
predicted_proba_df = pd.DataFrame(predicted_probabilities, columns=rf_classifier.classes_)
train_df = pd.concat([train_df, predicted_proba_df], axis=1)
train_df = pd.concat([train_df, X_train], axis=1)
# ### print
# print(3,train_df.shape)
# Export train_df to a CSV file
RF_Results_Train = os.path.join(Indicatorsdir, f'Train_observations.csv')
train_df.to_csv(RF_Results_Train, index=False)

################################################################
# PULL TEST AND TRAIN RESULTS TOGETHER INTO A SINGLE DATA FRAME AND EXPORT
################################################################
current_datetime = datetime.datetime.now()
date_string = current_datetime.strftime("%Y-%m-%d_%H-%M-%S")

full_df = pd.concat([train_df, test_df], axis=0)
RF_Results = os.path.join(Indicatorsdir, f'RandomForest_Predictions2019AISData{date_string}.csv')
full_df.to_csv(RF_Results, index=False)


### Areas for processing

In [ ]:
import pandas as pd

def create_emptyPortsDF():
    # Define the column names
    emptydf =pd.DataFrame()
    max_time_spent_H3_indices_columns = [
        'mmsi',
        'Trip_number',
        'ConsolidatedPortType_1',
        'ConsolidatedPortType_2',
        'ConsolidatedPortType_3',
        'MarineTrafficPortType_1',
        'MarineTrafficPortType_2',
        'MarineTrafficPortType_3',
        'MarineTrafficPortType_4',
        'MarineTrafficPortType_5',
        'MarineTrafficPortType_6',
        'MarineTrafficPortType_7',
        'MarineTrafficPortType_8',
        'MarineTrafficPortType_9',
        'MarineTrafficPortType_10',
        'NearConsolidatedPortType_1',
        'NearConsolidatedPortType_2',
        'NearConsolidatedPortType_3'
    ]

    # Create a DataFrame with the specified columns and initialize all values to 0
    emptydf = pd.DataFrame(0, columns=max_time_spent_H3_indices_columns, index=range(1))

    return emptydf

# Additional functions

## Function that creates Coastline and Just_Gulf data frames for a single day of AIS data

In [ ]:
## THIS VERSION OF THE CODE USES SPATIAL INDEXING AS RECOMMENDED BY CHATGPT

from shapely.wkt import loads
import geopandas as gpd
from geopandas import GeoDataFrame
import pandas as pd

def process_data(filtered_data, Gulf_withorigin_area, Gulf_area, alabama_island_area, galveston_island_area, biscayne_tx_area):
    print("Version with spatial indexing")
    # Create a GeoDataFrame from filtered_data
    gdf = GeoDataFrame(filtered_data, geometry=gpd.points_from_xy(filtered_data.lon, filtered_data.lat))

    # Build spatial indexes for the relevant areas
    Gulf_withorigin_area_sindex = Gulf_withorigin_area.sindex
    Gulf_area_sindex = Gulf_area.sindex
    alabama_island_area_sindex = alabama_island_area.sindex
    galveston_island_area_sindex = galveston_island_area.sindex
    biscayne_tx_area_sindex = biscayne_tx_area.sindex

    # Filter points in filtered_data that are within the Gulf_withorigin_area
    gulf_with_origin_union = Gulf_withorigin_area.unary_union
    gdf = gdf[gdf.geometry.within(gulf_with_origin_union)]

    # Create Coastline by excluding points in Gulf_area and other areas
    gulf_union = Gulf_area.unary_union
    other_areas_union = alabama_island_area.unary_union.union(galveston_island_area.unary_union).union(biscayne_tx_area.unary_union)
    
    Just_Gulf = gdf[gdf.geometry.within(gulf_union) & ~gdf.geometry.within(other_areas_union)]

    # Create Coastline by excluding points in Just_Gulf using spatial indexing
    Just_Gulf_sindex = Just_Gulf.sindex
    Coastline = gdf.loc[gdf.index.difference(Just_Gulf_sindex.intersection(gulf_union.sindex))]

    return Just_Gulf, Coastline


# Functions used for creating Indicators

## Distance Functions

In [ ]:
## THIS CODE CREATES DISTANCES BETWEEN EACH POINT AND CUMULATIVE TIME FOR A SINGLE TRIP
## IT ALSO CREATES GRAPHS FOR THAT TRIP


def haversine(lat1, lon1, lat2, lon2, to_radians=True, earth_radius=6371):
    if to_radians:
        lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])

    a = np.sin((lat2-lat1)/2.0)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2

    return earth_radius * 2 * np.arcsin(np.sqrt(a))

In [ ]:
# Adoption of the Haversine function to calculate as a function of vectors rather than 4 values 
def haversine_2(point1,point2):                  # This simply calls haversine for two points
    lat1 = point1[0]
    lat2 = point2[0]
    lon1 = point1[1]
    lon2 = point2[1]
    dist = haversine(lat1, lon1, lat2, lon2)
    return dist



# Single function that calculates distance the approximate distance from the coast
def distance_to_coast_lat_lng(lat, lng):           
    # coastal_points: sequence of points that are along the coast following the polygon used to identify trips in the Gulf
    coastal_points = [(25.9549238885927, -97.146341840721), (26.2105963742296, -97.1786084156481), (26.4392006473511, -97.2267948818937), (26.7131155219549, -97.3144205913896), (26.9409506241204, -97.3733392098903), (27.2381948299922, -97.3536997038123), (27.509565248758, -97.2499956220558), (27.8776471339216, -97.0077746348881), (28.0539999401663, -96.855022416767), (28.3280720859457, -96.4284079754034), (28.4969719023136, -96.1916422843358), (28.8635868411337, -95.3853300793036), (29.2108230117506, -94.9325270042829), (29.3231365306908, -94.7208564118929), (29.4049143402772, -94.7121277276709), (29.4827910105008, -94.5463471921147), (29.6592984918529, -94.0739071585848), (29.6792073576794, -93.9462501518341), (29.6630909605969, -93.826230743778), (29.7133276704338, -93.7574923555273), (29.7512255330155, -93.4399864669423), (29.7607479179518, -93.1998712835684), (29.6811534311097, -92.9380107569004), (29.5806227705967, -92.6565106907323), (29.5162520975399, -92.2642686354715), (29.5741538015925, -92.1475224839983), (29.5608678187247, -92.0318674180532), (29.4773159297318, -91.8529293914971), (29.4706667956712, -91.7951018585245), (29.5694090083582, -91.6969041610241), (29.6434023797613, -91.8496561349135), (29.7116557246348, -91.7492762663576), (29.7002833853108, -91.640167713579), (29.6196924687798, -91.6205281740788), (29.5181510398553, -91.5059641936618), (29.4754162216433, -91.4132219238004), (29.3746807827744, -91.3619409039944), (29.2243427925429, -91.2517412656884), (29.1044923894996, -90.7483800379835), (29.3044943130288, -90.5574400706218), (29.3301805074153, -90.4406939191486), (29.251198637516, -90.3915950703987), (29.2569102177412, -90.3206745110925), (29.1959702604612, -90.2977617150093), (29.0634922247878, -90.2584819182854), (29.1588178968435, -90.057722181173), (29.2997375962166, -89.8482337598385), (29.2768989322582, -89.6758422464489), (29.1788250621025, -89.4674449106425), (29.2064477008371, -89.3834316847671), (29.1559595208947, -89.3757940860725), (29.1511952028254, -89.4478057309062), (29.0062545714863, -89.4041623097948), (28.9489850882595, -89.4401681322116), (28.9021915070929, -89.3976157966283), (29.0224751597303, -89.2754142175166), (28.948030328351, -89.1401196120713), (29.034877420194, -89.1041137896545), (29.0406010378701, -89.0386486579873), (29.2083524286137, -88.954635072348), (29.4319111022806, -89.2448638227382), (29.4414136102532, -89.380157735065), (29.5316406824672, -89.4587158930652), (29.6872117675113, -89.2404987875087), (29.995091073275, -89.1464772356424), (30.1593768275681, -89.162843518559), (30.1744698581104, -89.2119423673089), (30.0706586329971, -89.3592389135599), (30.0914296066955, -89.4476168413101), (30.2791140829552, -89.3330528608931), (30.3803058562129, -88.9580522807037), (30.3671272452038, -88.8118468199808), (30.3304060353034, -88.7016471816741), (30.3680686334753, -88.6056316552292), (30.3360563490296, -88.585992115729), (30.3351146527177, -88.53143783934), (30.3125112244796, -88.4965231024513), (30.3247553955517, -88.4419688260617), (30.3887768822221, -88.3492265562003), (30.3482975768157, -88.1997478388939), (30.3125112244796, -88.1888369836164), (30.332289509439, -88.1408292203939), (30.2569223017108, -88.1190075098382), (30.2314728019458, -88.3437711285613), (30.2465547826037, -88.0786373453104), (30.2606920373758, -88.1091877400881), (30.337939714482, -88.1277361940602), (30.5436232982486, -88.0718067834058), (30.6450566630599, -88.0379831320445), (30.6506887357231, -87.9583338885163), (30.6121963694031, -87.9234191516275), (30.4157415341932, -87.921236980572), (30.3065333085294, -87.7750315198485), (30.2679047020761, -87.7652117500984), (30.2415154520466, -88.0368920465164), (30.2151191143229, -88.0259811912389), (30.238688023927, -87.6844707666455), (30.287692397843, -87.4542517202834), (30.297692397843, -87.4542517202834)]

    starting_point = (lat,lng)

#   Calculate the distance to every point along the coast
    distances = []
    for point in coastal_points:
        distance = haversine_2(starting_point, point)
        distances.append(distance)    

#   Find the shortest distance and its index
    hypotenuse1 = min(distances)
    minindex = distances.index(hypotenuse1)
    
#   Determine if the point after or before the current point is nearer and use those 2 points to calculate the distance
    hypotenuse2 = min(distances[minindex-1],distances[minindex+1])
    secindex = distances.index(hypotenuse2)

    base = haversine(coastal_points[minindex][0],coastal_points[minindex][1],coastal_points[secindex][0],coastal_points[secindex][1])
    
#   Correction for obtuse triangles that cause incorrect results
    hyp_rt_angle=math.sqrt(base*base+hypotenuse1*hypotenuse1)
    if hypotenuse2>hyp_rt_angle:
        dist_from_coast  = hypotenuse1
    else:     
        # First calculate the the area of a triangle using Heron's formula
        s = 0.5*(base + hypotenuse1 + hypotenuse2)
        area=math.sqrt(s*(s-hypotenuse1)*(s-hypotenuse2)*(s-base))
        # Then back out the distance using the formula Area = 1/2 base*height knowing that base = 50km
        dist_from_coast  = 2*area/base


    return dist_from_coast


## Date Functions

In [ ]:
# THIS CODE GENERATE THE SEASON BASED ON MONTH


def month_to_season(month_num):
    if (month_num == 11) | (month_num == 0) | (month_num == 1): # DEC-FEB
        season = 4
    if (month_num == 2) | (month_num == 3) | (month_num == 4): # MARCH - MAY
        season = 1
    if (month_num == 5) | (month_num == 6) | (month_num == 7): # JUNE - August 
        season = 2
    if (month_num == 8) | (month_num == 9) | (month_num == 10): # SEP- NOV
        season = 3
    return season

# check season function
season= month_to_season(1)
season

# note 0 equals Jan and 11 equals december 
def monthofyear(EpochTime):  # January = 0
    Base = 1546322400  # Jan. 1, 2019, 12:00 a.m. 
    Jan12021 = 731
    Jan12022 = 1096
    Jan12023 = 1461

    FebStart = 31
    MarStart = 59
    AprStart = 90
    MayStart = 120
    JunStart = 151
    JulStart = 181
    AugStart = 212
    SepStart = 243
    OctStart = 273
    NovStart = 304
    DecStart = 334
    leapyear2020 = 1582869600     # Feb 29, 2020

    days_since_2019 = epoch_to_days_since_1_1_2019(EpochTime)
    leapyearadjust = -1*(EpochTime >leapyear2020)
    year = int((days_since_2019+ leapyearadjust)/365)
    dayofyear = (days_since_2019) - 365*year +  leapyearadjust
    month = 1
    month = 1*(dayofyear>=FebStart) + \
            1*(dayofyear>=    MarStart ) + \
            1*(dayofyear>=    AprStart ) + \
            1*(dayofyear>=    MayStart ) + \
            1*(dayofyear>=    JunStart ) + \
            1*(dayofyear>=    JulStart ) + \
            1*(dayofyear>=    AugStart ) + \
            1*(dayofyear>=    SepStart ) + \
            1*(dayofyear>=    OctStart ) + \
            1*(dayofyear>=    NovStart ) + \
            1*(dayofyear>=    DecStart )
    return month
        

def epoch_to_season(EpochTime):
    month = monthofyear(EpochTime)
    season = month_to_season(month)
    return season

def epoch_to_DOW(EpochTime):
    Base = 1546322400  # Jan. 1, 2019, 12:00 a.m. 
    BaseDOW = 1
    DaySince = (EpochTime - Base) / (24*60*60)
    WeeksSinceBase = DaySince / 7
    DayOfWeek = BaseDOW + int((WeeksSinceBase - int(WeeksSinceBase)) * 7)
    return DayOfWeek

def epoch_to_days_since_1_1_2019(EpochTime):
    Base = 1546322400  # Jan. 1, 2019, 12:00 a.m. 
    DaySinceBase = int((EpochTime-Base) / (60*60*24))
    return DaySinceBase

def AISdate_to_epoch(AISDate):
    from datetime import datetime

    # date_string = "2019-06-01T16:14:14"

    # Define the format of the input date string
    date_format = "%Y-%m-%dT%H:%M:%S"

    # Convert the date string to a datetime object
    dt_object = datetime.strptime(AISDate, date_format)

    # Convert the datetime object to epoch time
    epoch_time = int(dt_object.timestamp())

    # print("Epoch Time:", epoch_time)
    return epoch_time

def epoch_to_hour_of_day(EpochTime):
    Base = 1546322400  # Jan. 1, 2019, 12:00 a.m. 
    # Daylight Savings Time points for US Central Time
    start2019 = 1552204800  
    end2019 = 1572768000
    start2020 = 1583654400
    end2020 = 1604217600
    start2021 = 1615708800
    end2021 = 1636272000
    start2022 = 1647158400
    end2022 = 1667721600
    start2023 = 1678608000
    end2023 = 1699171200

    
    DayLightSavingsAdjust = +1 * (EpochTime > start2019) + \
                            -1 * (EpochTime > end2019) + \
                            +1 * (EpochTime > start2020) + \
                            -1 * (EpochTime > end2020) + \
                            +1 * (EpochTime > start2021) + \
                            -1 * (EpochTime > end2021) + \
                            +1 * (EpochTime > start2022) + \
                            -1 * (EpochTime > end2022) + \
                            +1 * (EpochTime > start2023) + \
                            -1 * (EpochTime > end2023)
#    print(DayLightSavingsAdjust)
    DaysSince = ((EpochTime-Base) / (60*60*24))
    PortionOfDay = DaysSince - int(DaysSince)
    HourOfDay = int(PortionOfDay*24) + DayLightSavingsAdjust
    return HourOfDay

def epoch_to_date(epoch_time):
    # Convert epoch time to a datetime object
    dt = datetime.fromtimestamp(epoch_time)
    
    # Format the datetime as 'YYYY-MM-DD'
    formatted_date = dt.strftime('%Y-%m-%d')
    
    return formatted_date

## h3 functions

In [ ]:
import pandas as pd
import h3

# Function to map latitude and longitude to H3 cell
def map_to_h3(row):
    lat = row['lat']  # Replace 'lat' with the actual column name in your DataFrame
    lon = row['lon']  # Replace 'lon' with the actual column name in your DataFrame
    resolution = 8  # Adjust the H3 resolution as needed -- 8 is the level chosen for this analysis.
    h3_cell = h3.geo_to_h3(lat, lon, resolution)
    return h3_cell

# def map_to_h3(row):
#     return h3.geo_to_h3(row['latitude'], row['longitude'], resolution)


# Function to calculate the time spent in each H3 cell
def calculate_time_spent(group):
    group['time_diff'] = group['event_timestamp'].diff().fillna(pd.Timedelta(seconds=0))
    group['time_spent_seconds'] = group['time_diff'].cumsum().dt.total_seconds().astype(int)
    return group

In [ ]:
def calculate_time_spent(group):
    group['time_diff'] = group['event_timestamp'].diff().fillna(pd.Timedelta(seconds=0))
    group['time_diff'] = pd.to_timedelta(group['time_diff'])  # Ensure 'time_diff' is in Timedelta format
    group['time_spent_seconds'] = group['time_diff'].cumsum().dt.total_seconds().astype(int)
    return group['time_spent_seconds']

### Load Marine Traffic Ports and process to add H3_cells

In [ ]:
# Load CSV File of data scraped from MarineTraffic.com
# Specify the file path
AIS_dir =  r"/scratch/user/r-woodward/AIS Data/"         # r"D:\AIS Data\CSV Files\Gulf"
file_path = os.path.join(AIS_dir, f'MarineTrafficPorts.csv')
h3_file_path = os.path.join(AIS_dir, f'MarineTrafficPortsWithH3.csv')

# Load the CSV file into a DataFrame
PortsListdf = pd.read_csv(file_path, encoding='latin-1')

#
PortsListdf['lat'] = pd.to_numeric(PortsListdf['lat'], errors='coerce')
PortsListdf['lon'] = pd.to_numeric(PortsListdf['lon'], errors='coerce')

# Calculate h3 indices
PortsListdf['h3_cell'] = PortsListdf.apply(map_to_h3, axis=1)

# Save as csv file
PortsListdf.to_csv(h3_file_path, index=False)
PortsListWithH3_df = pd.read_csv(h3_file_path, encoding='latin-1')

In [ ]:
# import h3

# h3_cell
# # Define a function to get the neighbors of an h3_cell
# def get_neighbors(h3_cell):
#     return list(h3.k_ring(h3_cell, 1))  # Assuming 1-ring neighbors

# # Create new columns with initial values set to zero
# max_time_spent_H3_indices['NearConsolidatedPortType_1'] = 0
# max_time_spent_H3_indices['NearConsolidatedPortType_2'] = 0
# max_time_spent_H3_indices['NearConsolidatedPortType_3'] = 0

# # Iterate through each row in max_time_spent_H3_indices
# for index, row in max_time_spent_H3_indices.iterrows():
#     h3_cell = row['h3_cell']
    
#     # Get the neighboring h3_cells
#     neighbors = get_neighbors(h3_cell)
    
#     # Find matching rows in PortsListdf with ConsolidatedPortType_num equal to 1, 2, or 3 in the neighbors
#     matching_rows = PortsListdf[(PortsListdf['h3_cell'].isin(neighbors)) & (PortsListdf['ConsolidatedPortType_num'].isin([1, 2, 3]))]
    
#     # Update the corresponding NearConsolidatedPortType columns to 1 if there are matches
#     for port_type in [1, 2, 3]:
#         if port_type in matching_rows['ConsolidatedPortType_num'].values:
#             max_time_spent_H3_indices.at[index, f'NearConsolidatedPortType_{port_type}'] = 1

# # Now, max_time_spent_H3_indices contains the new columns NearConsolidatedPortType_1, NearConsolidatedPortType_2, and NearConsolidatedPortType_3.
